**1 Collecting Data**


## Improve thing
  1) Chowlin model to expand data frequency
  2) A. สร้าง Environment และ State Vector (ตามหัวข้อ 5. Method ใน Paper)แผนของคุณระบุว่าจะใช้ Reinforcement Learning (RL) ซึ่งต้องการ "สภาพแวดล้อม (Environment)" ในการตัดสินใจ คุณต้องเขียน Code เพื่อรวมข้อมูลจากสองส่วนเข้าด้วยกันเป็น $S_t$:Macro Signal ($m_t$): เอาผล Forecast หรือค่า ECT/Coefficients จากโมเดล ARDL-ECM ที่คุณทำเสร็จแล้วมาเป็น FeatureTechnical Signal ($b_t$): เอาความน่าจะเป็น ($P_{bull}, P_{bear}, P_{neutral}$) หรือสัญญาณจากโมเดล LSTM/GRU มาเป็น FeaturePortfolio State: สถานะปัจจุบัน (ถือเงินสด, ถือหุ้น, ต้นทุน)สิ่งที่ต้องทำ: สร้าง Class Environment (เช่นสืบทอดจาก gym.Env) ที่รับค่าเหล่านี้เข้าไปในแต่ละ Step
  3) B. พัฒนาส่วน Reinforcement Learning (PPO Algorithm)ใน Code ปัจจุบันยังไม่พบส่วนที่เป็น PPO (Proximal Policy Optimization) หรือ Agent ที่ตัดสินใจ "ซื้อ/ขาย/ถือ"Action: คุณต้องกำหนด Action Space เป็น {Buy, Hold, Sell}Reward Function: เขียนสมการ Reward ตามแผน: $\text{Reward} = \text{Portfolio Value} - (\lambda \times \text{Drawdown Penalty})$Implementation: ใช้ Library เช่น stable-baselines3 หรือ rllib เพื่อสร้าง PPO Agent แล้วสั่ง Train ด้วย Environment ที่สร้างในข้อ A
  4) C. เพิ่มโมเดล CNN (Optional ตามแผน)
ในแผนระบุว่าจะใช้ CNN (Convolutional Neural Networks) ร่วมกับ LSTM เพื่อจับ Pattern กราฟ แต่ใน Code พบแค่ LSTM/GRU


5) D. การรวมระบบและ Backtesting (Phase 4)
Rolling Window Training: ต้องเขียน Loop เพื่อให้โมเดล Train ข้อมูลปี 2010-2024 และ Test ปี 2025 (Out-of-sample)

Performance Metrics: Code ปัจจุบันอาจจะวัดแค่ความแม่นยำ (Accuracy/MSE) ของการทำนายราคา แต่ตามแผนคุณต้องวัด Trading Performance ของบอท ได้แก่:

Cumulative Return (กำไรสะสม)

Sharpe Ratio (ผลตอบแทนเทียบความเสี่ยง)

Maximum Drawdown (การขาดทุนสูงสุด)

In [2]:
!pip install fredapi

| Component               | Description                                    | Paper                 |
| ----------------------- | ---------------------------------------------- | --------------------- |
| Proxy selection         | Dynamic Factor / PCA on macro dataset          | Stock & Watson (2002) |
| Temporal disaggregation | Chow-Lin linear regression                     | Chow & Lin (1971)     |
| Extrapolation           | AR(1) residual correction (Fernandez model)    | Fernandez (1981)      |
| Ratio preservation      | Denton (1971) for % variables (e.g., Debt/GDP) | Denton (1971)         |

📘 Academic Framework (สำหรับ citation ใน report)

Chow & Lin (1971) – Best Linear Unbiased Interpolation by Related Series

Stock & Watson (2002) – Macroeconomic Forecasting Using Diffusion Indexes

Mariano & Murasawa (2003) – A New Coincident Index of Business Cycles Based on Monthly and Quarterly Series


GDP Data
(https://)https://lookerstudio.google.com/u/0/reporting/52a283c8-91f9-4b35-8157-2b2b42312602/page/p_l7cxsbayvc

In [3]:
# ================== Setup ==================
# pip install yfinance fredapi pandas pandas-datareader

import pandas as pd
import numpy as np
import yfinance as yf
from fredapi import Fred
from datetime import date

# ----- Parameters -----
START = "2013-01-01"
END   = date.today().isoformat()         # วันนี้
FRED_API_KEY = "c948956426006ca126a2dd3bd1f07cee"  # แนะนำอย่า hardcode key จริงลงงาน/ขึ้น GitHub

# FRED client (กันอินเทอร์เน็ตล่ม)
try:
    fred = Fred(api_key=FRED_API_KEY)
except Exception:
    fred = None

# ================== Helper Functions ==================
def get_fred_series(series_id: str, rename_to: str = None) -> pd.Series:
    """ดึงซีรีส์จาก FRED"""
    if fred is None:
        return pd.Series(dtype="float64", name=rename_to or series_id)
    s = fred.get_series(series_id)
    s.name = rename_to or series_id
    s.index = pd.to_datetime(s.index)
    return s.sort_index()

def expand_period_to_target_index(target_index: pd.DatetimeIndex,
                                  lowfreq_series: pd.Series,
                                  freq: str) -> pd.Series:
    """ขยายค่าความถี่ต่ำไปที่ target_index (เหมือนของเดิม)"""
    if lowfreq_series is None or lowfreq_series.empty:
        return pd.Series(index=target_index, dtype="float64", name=getattr(lowfreq_series, "name", None))
    s = lowfreq_series.dropna().copy()
    s.index = pd.PeriodIndex(s.index, freq=freq)
    df = pd.DataFrame(index=target_index)
    df["period"] = df.index.to_period(freq)
    out = df.join(s.rename(s.name), on="period")[s.name]
    out.name = s.name
    return out

# ================== 2) Macro: USA via FRED ==================
# คุณอยากได้ตัวแปร:
# ["inflation_rate"], ["GDP"], ["m2_growth"], ["bond_yield_spread"], ["dxy"],
# ["nonfarm_payrolls"], ["unemployment_rate"], ["vix_index"], ["ppi_inflation"],
# ["real_rate_10y"], ["m1_growth"]

# ดึง series (บางอันเป็น daily, monthly, quarterly)
cpi_index = get_fred_series("CPIAUCSL", "CPI_Index")     # monthly
ppi_index = get_fred_series("PPIACO", "PPI_Index")       # monthly


m2_level  = get_fred_series("M2SL", "M2_Level")          # weekly-ish
m1_level  = get_fred_series("M1SL", "M1_Level")          # weekly-ish

bond_spread = get_fred_series("T10Y2Y", "bond_yield_spread")  # daily
dxy_index   = get_fred_series("DTWEXBGS", "dxy")              # weekly-ish
payems      = get_fred_series("PAYEMS", "nonfarm_payrolls")   # monthly
unrate      = get_fred_series("UNRATE", "unemployment_rate")  # monthly
vix         = get_fred_series("VIXCLS", "vix_index")          # daily
real10y     = get_fred_series("DFII10", "real_rate_10y")      # daily

# ================== 1) Stock Prices ==================
SECTORS = {
    "Healthcare": ["AMGN","GILD","VRTX","REGN","BIIB","ILMN","IDXX","ISRG","ALGN"]
}

df_stocks = pd.DataFrame()
for sector, tickers in SECTORS.items():
    for ticker in tickers:
        try:
            data = yf.download(ticker, start=START, end=END, interval="1mo", auto_adjust=False, progress=False)['Close']
            if isinstance(data, pd.Series):
                df_stocks[f"Close_{ticker}"] = data
            else:
                df_stocks[f"Close_{ticker}"] = data[ticker]
        except Exception:
            df_stocks[f"Close_{ticker}"] = pd.Series(dtype="float64")

df_stocks = df_stocks.dropna(how="all")

# *** โครงสร้างเดิมของคุณ: ใช้ index จาก df_stocks เป็นแกน ***
# (ชื่อเดิม quarterly_index แต่จริง ๆ เป็น monthly จาก interval="1mo")
if not df_stocks.empty:
    monthly_index = df_stocks.index
else:
    monthly_index = pd.date_range(start=pd.to_datetime(START), end=pd.to_datetime(END), freq="MS")

# ================== 2) Align Macro (เหมือนเดิม) ==================
# ทำ index ซ้ำซ้อนให้สะอาด
def _dedupe(s: pd.Series) -> pd.Series:
    s = s.copy()
    s = s[~pd.Index(s.index).duplicated(keep="last")]
    return s.sort_index()

cpi_index  = _dedupe(cpi_index)
ppi_index  = _dedupe(ppi_index)
m2_level   = _dedupe(m2_level)
m1_level   = _dedupe(m1_level)
bond_spread= _dedupe(bond_spread)
dxy_index  = _dedupe(dxy_index)
payems     = _dedupe(payems)
unrate     = _dedupe(unrate)
vix        = _dedupe(vix)
real10y    = _dedupe(real10y)

# GDP เป็น quarterly -> ขยายไปตาม index ของหุ้น (monthly) แบบเดิม


# ที่เหลือใช้ reindex ffill เหมือนของเดิม
cpi_aligned   = cpi_index.reindex(monthly_index, method="ffill")
ppi_aligned   = ppi_index.reindex(monthly_index, method="ffill")
m2_aligned    = m2_level.reindex(monthly_index, method="ffill")
m1_aligned    = m1_level.reindex(monthly_index, method="ffill")
bond_aligned  = bond_spread.reindex(monthly_index, method="ffill")
dxy_aligned   = dxy_index.reindex(monthly_index, method="ffill")
payems_aligned= payems.reindex(monthly_index, method="ffill")
unrate_aligned= unrate.reindex(monthly_index, method="ffill")
vix_aligned   = vix.reindex(monthly_index, method="ffill")
real10y_aligned = real10y.reindex(monthly_index, method="ffill")


# ================== 4) Combine หุ้น + Macro (เหมือนเดิม) ==================
df = df_stocks.copy()
if df.empty:
    df = pd.DataFrame(index=monthly_index)

# inflation_rate: YoY จาก CPI index
df["inflation_rate"] = cpi_aligned.pct_change(12)




# m2_growth / m1_growth: ln(S_t / S_{t-1}) และแถวแรกเป็น 0
df["m2_growth"] = np.log(m2_aligned / m2_aligned.shift(1))
if len(df["m2_growth"]) > 0:
    df["m2_growth"].iloc[0] = 0.0

df["m1_growth"] = np.log(m1_aligned / m1_aligned.shift(1))
if len(df["m1_growth"]) > 0:
    df["m1_growth"].iloc[0] = 0.0

# bond_yield_spread, dxy, labor, risk
df["bond_yield_spread"] = bond_aligned
df["dxy"]               = dxy_aligned
df["nonfarm_payrolls"]  = payems_aligned
df["unemployment_rate"] = unrate_aligned
df["vix_index"]         = vix_aligned
df["real_rate_10y"]     = real10y_aligned

# ppi_inflation: YoY จาก PPI index
df["ppi_inflation"] = ppi_aligned.pct_change(12)

# ================== 5) ตรวจสอบผลลัพธ์ (เหมือนเดิม) ==================
df = df.iloc[25:]  # ตัดช่วงต้นเหมือนเดิม (กัน NaN จาก YoY/shift)

print(df)
print("\n=== HEAD ===")
print(df.head(10))
print("\n=== TAIL ===")
print(df.tail(10))
print("\n=== COLUMNS ===")
print(df.columns.tolist())
print("\n=== NA COUNTS ===")
print(df.isna().sum())

# (Optional) Save to CSV
# df.to_csv("us_healthcare_macro_2013_today_monthly_like_yours.csv", encoding="utf-8-sig")


            Close_AMGN  Close_GILD  Close_VRTX  Close_REGN  Close_BIIB  \
Date                                                                     
2015-02-01  157.720001  103.529999  119.430000  413.839996  409.589996   
2015-03-01  159.850006   98.129997  117.970001  451.480011  422.239990   
2015-04-01  157.910004  100.510002  123.279999  457.459991  373.929993   
2015-05-01  156.259995  112.269997  128.289993  512.559998  396.989990   
2015-06-01  153.520004  117.080002  123.480003  510.130005  403.940002   
...                ...         ...         ...         ...         ...   
2025-09-01  282.200012  111.000000  391.640015  562.270020  140.080002   
2025-10-01  298.429993  119.790001  425.570007  651.799988  154.270004   
2025-11-01  345.459991  125.839996  433.609985  780.190002  182.089996   
2025-12-01  327.309998  122.739998  453.359985  771.869995  175.990005   
2026-01-01  330.410004  124.910004  441.359985  733.039978  164.419998   

            Close_ILMN  Close_IDXX  C

/tmp/ipython-input-2876661565.py:132: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["inflation_rate"] = cpi_aligned.pct_change(12)
/tmp/ipython-input-2876661565.py:140: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this k

In [4]:
# ============================================================
# 📊 BLOCK 1.5 (USA): Chow-Lin Temporal Disaggregation (GDP Quarterly → Monthly)
#    Chow & Lin (1971) - Best Linear Unbiased Interpolation
#    Use FRED: GDP (quarterly) + INDPRO (monthly indicator)
# ============================================================

import numpy as np
import pandas as pd
from scipy import linalg  # not strictly required, but ok to keep

# ================== 1) Chow-Lin Function ==================
def chow_lin_disaggregate(y_low: pd.Series, X_high: pd.DataFrame,
                          agg_method: str = 'sum', rho: float = None) -> tuple:
    """
    Chow-Lin temporal disaggregation: Low-frequency → High-frequency

    Parameters
    ----------
    y_low : pd.Series
        Low-frequency series (e.g., Quarterly GDP) with DatetimeIndex
    X_high : pd.DataFrame
        High-frequency indicator(s) (e.g., Monthly INDPRO) with DatetimeIndex
    agg_method : str
        'sum' for flow variables (GDP), 'mean' for stock/average variables
    rho : float or None
        AR(1) autocorrelation parameter. If None, estimate from data.

    Returns
    -------
    (pd.Series, beta, rho)
    """

    # 1) Align data
    y_low = y_low.dropna().copy()
    X_high = X_high.dropna().copy()

    # Frequency ratio Q→M = 3 months per quarter
    n_high_per_low = 3

    # Overlap period
    quarters = y_low.index
    months = X_high.index

    min_date = max(quarters.min(), months.min().to_period('Q').to_timestamp())
    max_date = min(quarters.max(), months.max().to_period('Q').to_timestamp())

    y_low = y_low[(y_low.index >= min_date) & (y_low.index <= max_date)]

    # monthly span covering the low-frequency period
    month_start = y_low.index.min()
    month_end = (y_low.index.max() + pd.offsets.QuarterEnd()).to_period('M').to_timestamp()

    X_high = X_high[(X_high.index >= month_start) & (X_high.index <= month_end)]

    n_low = len(y_low)
    n_high = n_low * n_high_per_low

    # Trim / pad X_high to exact size
    X_high = X_high.iloc[:n_high]
    if len(X_high) < n_high:
        pad_size = n_high - len(X_high)
        last_idx = X_high.index[-1]
        new_idx = pd.date_range(start=last_idx + pd.offsets.MonthBegin(), periods=pad_size, freq='MS')
        pad_df = pd.DataFrame(index=new_idx, columns=X_high.columns)
        for col in X_high.columns:
            pad_df[col] = X_high[col].iloc[-1]
        X_high = pd.concat([X_high, pad_df])

    # 3) Aggregation matrix C (n_low x n_high)
    C = np.zeros((n_low, n_high))
    for i in range(n_low):
        start_col = i * n_high_per_low
        end_col = start_col + n_high_per_low
        if agg_method == 'sum':
            C[i, start_col:end_col] = 1.0
        elif agg_method == 'mean':
            C[i, start_col:end_col] = 1.0 / n_high_per_low
        else:  # 'last'
            C[i, end_col - 1] = 1.0

    # 4) X matrix (add constant)
    X = X_high.values
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    X = np.column_stack([np.ones(n_high), X])  # intercept

    # 5) Aggregate X to quarterly
    X_low = C @ X
    y = y_low.values.flatten()

    # 6) OLS on aggregated data
    beta_ols = np.linalg.lstsq(X_low, y, rcond=None)[0]
    u_low = y - X_low @ beta_ols

    # 7) Estimate rho if None
    if rho is None:
        if len(u_low) > 1:
            rho = np.corrcoef(u_low[:-1], u_low[1:])[0, 1]
            rho = float(np.clip(rho, -0.99, 0.99))
        else:
            rho = 0.0

    # 8) Covariance matrix V (AR1)
    V = np.fromfunction(lambda i, j: rho ** np.abs(i - j), (n_high, n_high), dtype=float)

    # 9) GLS
    V_low = C @ V @ C.T
    try:
        V_low_inv = np.linalg.inv(V_low)
    except np.linalg.LinAlgError:
        V_low_inv = np.linalg.pinv(V_low)

    XVX = X_low.T @ V_low_inv @ X_low
    XVy = X_low.T @ V_low_inv @ y
    try:
        beta_gls = np.linalg.solve(XVX, XVy)
    except np.linalg.LinAlgError:
        beta_gls = np.linalg.lstsq(XVX, XVy, rcond=None)[0]

    # 10) Preliminary high-frequency estimate
    p_high = X @ beta_gls

    # 11) Distribute residuals
    u_low_gls = y - X_low @ beta_gls
    VCt = V @ C.T
    try:
        dist_matrix = VCt @ np.linalg.inv(V_low)
    except np.linalg.LinAlgError:
        dist_matrix = VCt @ np.linalg.pinv(V_low)

    y_high = p_high + dist_matrix @ u_low_gls

    # 12) Output series
    result = pd.Series(y_high, index=X_high.index, name=y_low.name or 'GDP_Monthly')
    return result, beta_gls, rho


# ================== 2) Load US GDP Quarterly from FRED ==================
print("="*60)
print("📊 BLOCK 1.5 (USA): Chow-Lin Temporal Disaggregation")
print("="*60)

print("\n📥 Loading US GDP Quarterly from FRED...")

# GDP (US, quarterly, SAAR in billions of dollars in FRED)
gdp_quarterly = get_fred_series("GDP", "GDP_US_Q")

# Make sure it's quarter-start indexed (optional but helps consistent resample)
# FRED GDP usually timestamps quarter start already; still ok either way.
gdp_quarterly = gdp_quarterly.sort_index()

print(f"✅ US GDP Quarterly loaded: {len(gdp_quarterly)} quarters")
print(f"   Period: {gdp_quarterly.index.min().strftime('%Y-%m')} to {gdp_quarterly.index.max().strftime('%Y-%m')}")


# ================== 3) Prepare Monthly Indicator (INDPRO) ==================
print("\n📥 Loading monthly indicator (INDPRO) from FRED...")

# INDPRO: Industrial Production Index (monthly)
indpro = get_fred_series("INDPRO", "INDPRO")
indpro = indpro.sort_index()

# Align to month-start index (optional; depends on your df index style)
# If your main df uses month-start (YYYY-MM-01), this keeps it consistent:
indpro_ms = indpro.resample("MS").mean()  # monthly value placed on month-start
indpro_ms.name = "INDPRO"

print(f"✅ INDPRO Monthly loaded: {len(indpro_ms)} months")
print(f"   Period: {indpro_ms.index.min().strftime('%Y-%m')} to {indpro_ms.index.max().strftime('%Y-%m')}")

X_indicator = pd.DataFrame({"INDPRO": indpro_ms})


# ================== 4) Apply Chow-Lin (GDP Q → GDP M) ==================
print("\n🔄 Applying Chow-Lin disaggregation (US GDP Quarterly → Monthly)...")

gdp_monthly, beta_chowlin, rho_chowlin = chow_lin_disaggregate(
    y_low=gdp_quarterly,
    X_high=X_indicator,
    agg_method="sum",   # GDP is a flow variable
    rho=None
)

# Monthly GDP growth (log)
gdp_growth_monthly = np.log(gdp_monthly / gdp_monthly.shift(1))
gdp_growth_monthly.name = "gdp_growth"

print(f"\n✅ Chow-Lin completed!")
print(f"   - Estimated AR(1) rho: {rho_chowlin:.4f}")
print(f"   - Monthly GDP observations: {len(gdp_monthly)}")


# ================== 5) Validation (aggregate back to quarterly) ==================
print("\n🔍 Validation: Aggregating monthly back to quarterly...")

gdp_monthly_to_q = gdp_monthly.resample("QS").sum()

validation = pd.DataFrame({
    "Original_Q": gdp_quarterly,
    "ChowLin_Agg_Q": gdp_monthly_to_q
}).dropna()

validation["Error_%"] = (validation["ChowLin_Agg_Q"] - validation["Original_Q"]).abs() / validation["Original_Q"] * 100
print(f"✅ Mean Aggregation Error: {validation['Error_%'].mean():.6f}%")
print("   (Should be ~0% if Chow-Lin is working correctly)")


# ================== 6) Add to main df (if exists) ==================
print("\n📌 Adding GDP_Monthly and gdp_growth to main dataframe...")

if "df" in globals():
    df["GDP_Monthly"] = gdp_monthly.reindex(df.index)
    df["gdp_growth"]  = gdp_growth_monthly.reindex(df.index)

    # Optional: forward-fill small gaps
    df["GDP_Monthly"] = df["GDP_Monthly"].ffill()
    df["gdp_growth"]  = df["gdp_growth"].ffill()

    print("✅ Added to df:", ["GDP_Monthly", "gdp_growth"])
    print("df shape:", df.shape)
    print(df[["GDP_Monthly", "gdp_growth"]].dropna().tail(10))
else:
    gdp_output = pd.DataFrame({
        "GDP_Monthly": gdp_monthly,
        "gdp_growth": gdp_growth_monthly
    })
    print(gdp_output.tail(10))


print("\n" + "="*60)
print("✅ BLOCK 1.5 (USA) COMPLETED")
print("="*60)
print(f"""
Summary:
- Input:  US GDP Quarterly (FRED: GDP) - {len(gdp_quarterly)} quarters
- Output: US GDP Monthly - {len(gdp_monthly)} months
- Indicator used: INDPRO (Industrial Production Index)
- Method: Chow-Lin (1971) with AR(1) residual structure
- Estimated rho: {rho_chowlin:.4f}

New columns (if df exists):
- 'GDP_Monthly': Monthly GDP level (derived)
- 'gdp_growth' : Monthly GDP growth rate (log)
""")


📊 BLOCK 1.5 (USA): Chow-Lin Temporal Disaggregation

📥 Loading US GDP Quarterly from FRED...
✅ US GDP Quarterly loaded: 319 quarters
   Period: 1946-01 to 2025-07

📥 Loading monthly indicator (INDPRO) from FRED...
✅ INDPRO Monthly loaded: 1284 months
   Period: 1919-01 to 2025-12

🔄 Applying Chow-Lin disaggregation (US GDP Quarterly → Monthly)...

✅ Chow-Lin completed!
   - Estimated AR(1) rho: 0.9900
   - Monthly GDP observations: 945

🔍 Validation: Aggregating monthly back to quarterly...
✅ Mean Aggregation Error: 0.000000%
   (Should be ~0% if Chow-Lin is working correctly)

📌 Adding GDP_Monthly and gdp_growth to main dataframe...
✅ Added to df: ['GDP_Monthly', 'gdp_growth']
df shape: (132, 21)
             GDP_Monthly  gdp_growth
Date                                
2025-04-01  10097.755060    0.004790
2025-05-01  10144.625466    0.004631
2025-06-01  10243.348473    0.009685
2025-07-01  10349.465335    0.010306
2025-08-01  10370.101168    0.001992
2025-09-01  10375.522497    0.0005

**2 Data preprocessing**

In [5]:
# =====================================================
# ⚙️ BLOCK 2: Prepare Combined Data (No PCA, No LASSO)
# =====================================================

import numpy as np
import pandas as pd

# ======== 1) Filter & split ========
df_filtered = df.iloc[14:].copy()  # ตัดช่วงแรกที่ NA เยอะ

# เลือกหุ้น (ตัวอย่าง 8 ตัวแรก)
stock_columns = [c for c in df_filtered.columns if c.startswith("Close_")]
selected_stock_data = df_filtered[stock_columns]

# เลือกตัวแปร macro ที่มีอยู่จริง

macro_columns = [
    "inflation_rate",
    "ppi_inflation",
    "m2_growth",
    "m1_growth",
    "bond_yield_spread",
    "dxy",
    "nonfarm_payrolls",
    "unemployment_rate",
    "vix_index",
    "real_rate_10y",
    "gdp_growth"
]
macro_columns = [c for c in macro_columns if c in df_filtered.columns]
selected_macro_data = df_filtered[macro_columns]

# ======== 2) Combine + clean ========
combined_df = pd.concat([selected_stock_data, selected_macro_data], axis=1)
combined_df = combined_df.ffill().replace([np.inf, -np.inf], np.nan).dropna(how="any")

print("✅ Combined data shape:", combined_df.shape)
print("✅ Stocks:", len(stock_columns), "| Macros:", len(macro_columns))

# ======== 3) Log-transform หุ้นและสร้าง Δlog ========
combined_dfs = combined_df.copy()
for c in stock_columns:
    combined_dfs[f"Logclose_{c.replace('Close_','')}"] = np.log(combined_df[c])

combined_dfs_diff = combined_dfs.copy()
for c in stock_columns:
    stock_name = c.replace("Close_","")
    combined_dfs_diff[f"D_Logclose_{stock_name}"] = combined_dfs[f"Logclose_{stock_name}"].diff()

# ======== 4) Drop แถวแรกของ differenced ========
combined_dfs_diff = combined_dfs_diff.dropna()

print("✅ Differenced data ready:", combined_dfs_diff.shape)

# ======== 5) ตัวอย่างผลลัพธ์ ========
print("\n=== Combined_dfs (Levels) ===")
display(combined_dfs.head())

print("\n=== Combined_dfs_diff (Differenced) ===")
display(combined_dfs_diff.head())


✅ Combined data shape: (118, 20)
✅ Stocks: 9 | Macros: 11
✅ Differenced data ready: (117, 38)

=== Combined_dfs (Levels) ===


,Close_AMGN,Close_GILD,Close_VRTX,Close_REGN,Close_BIIB,Close_ILMN,Close_IDXX,Close_ISRG,Close_ALGN,inflation_rate,...,gdp_growth,Logclose_AMGN,Logclose_GILD,Logclose_VRTX,Logclose_REGN,Logclose_BIIB,Logclose_ILMN,Logclose_IDXX,Logclose_ISRG,Logclose_ALGN
Date,,,,,,,,,,,,,,,,,,,,,
2016-04-01,158.300003,88.209999,84.339996,376.709991,274.989990,131.313232,84.349998,69.595558,72.190002,0.011726,...,0.008352,5.064492,4.479720,4.434856,5.931476,5.616735,4.877586,4.434975,4.242701,4.279302
2016-05-01,157.949997,87.059998,93.150002,398.929993,289.730011,140.885208,87.570000,70.523331,78.830002,0.010785,...,0.001967,5.062279,4.466598,4.534211,5.988786,5.668949,4.947945,4.472438,4.255944,4.367294
2016-06-01,152.149994,83.419998,86.019997,349.230011,241.820007,136.556427,92.860001,73.489998,80.550003,0.010793,...,0.007107,5.024867,4.423888,4.454580,5.855731,5.488194,4.916738,4.531093,4.297149,4.388878
2016-07-01,172.029999,79.470001,97.000000,425.119995,289.929993,161.819061,93.790001,77.306664,89.150002,0.008684,...,0.002690,5.147669,4.375380,4.574711,6.052371,5.669639,5.086479,4.541058,4.347780,4.490320
2016-08-01,170.059998,78.379997,94.510002,392.549988,305.630005,163.754868,112.680000,76.268890,92.900002,0.010553,...,0.001123,5.136151,4.361569,4.548706,5.972664,5.722375,5.098371,4.724552,4.334265,4.531524



=== Combined_dfs_diff (Differenced) ===


,Close_AMGN,Close_GILD,Close_VRTX,Close_REGN,Close_BIIB,Close_ILMN,Close_IDXX,Close_ISRG,Close_ALGN,inflation_rate,...,Logclose_ALGN,D_Logclose_AMGN,D_Logclose_GILD,D_Logclose_VRTX,D_Logclose_REGN,D_Logclose_BIIB,D_Logclose_ILMN,D_Logclose_IDXX,D_Logclose_ISRG,D_Logclose_ALGN
Date,,,,,,,,,,,,,,,,,,,,,
2016-05-01,157.949997,87.059998,93.150002,398.929993,289.730011,140.885208,87.570000,70.523331,78.830002,0.010785,...,4.367294,-0.002213,-0.013123,0.099355,0.057310,0.052215,0.070360,0.037464,0.013243,0.087992
2016-06-01,152.149994,83.419998,86.019997,349.230011,241.820007,136.556427,92.860001,73.489998,80.550003,0.010793,...,4.388878,-0.037412,-0.042709,-0.079631,-0.133055,-0.180756,-0.031208,0.058655,0.041206,0.021584
2016-07-01,172.029999,79.470001,97.000000,425.119995,289.929993,161.819061,93.790001,77.306664,89.150002,0.008684,...,4.490320,0.122802,-0.048508,0.120131,0.196641,0.181446,0.169741,0.009965,0.050631,0.101442
2016-08-01,170.059998,78.379997,94.510002,392.549988,305.630005,163.754868,112.680000,76.268890,92.900002,0.010553,...,4.531524,-0.011518,-0.013811,-0.026005,-0.079708,0.052736,0.011892,0.183494,-0.013515,0.041203
2016-09-01,166.809998,79.120003,87.209999,402.019989,313.029999,176.712067,112.730003,80.536667,93.750000,0.015486,...,4.540632,-0.019296,0.009397,-0.080387,0.023838,0.023924,0.076151,0.000444,0.054447,0.009108


In [6]:
# ======== Filter sector log-prices + macro variables ========

# Macro columns ที่มีอยู่จริงใน Excel + yfinance
macro_columns = [
    "inflation_rate",
    "ppi_inflation",
    "m2_growth",
    "m1_growth",
    "bond_yield_spread",
    "dxy",
    "nonfarm_payrolls",
    "unemployment_rate",
    "vix_index",
    "real_rate_10y",
    "gdp_growth"
]



# ======== ตรวจสอบว่า df ถูกสร้างแล้วจาก Block ก่อนหน้า ========
if 'df' not in globals():
    raise RuntimeError("❌ Variable 'df' not found. Please run the data-preparation block first.")

df_filtered_macro = df

# เก็บผลลัพธ์แยก sector
combined_dfs = {}

for sector, tickers_list in SECTORS.items():
    sector_frames = []

    for ticker in tickers_list:
        colname = f"Close_{ticker}"
        if colname not in df.columns:
            print(f"⚠️ {colname} not found in df. Skipping...")
            continue

        # ✅ ใช้ Logclose แทน log-return
        tmp = df[[colname]].copy()
        tmp[f"Logclose_{ticker}"] = np.log(tmp[colname])
        tmp = tmp[[f"Logclose_{ticker}"]]

        # align index ให้ตรงกับ macro
        tmp = tmp.loc[df_filtered_macro.index]
        sector_frames.append(tmp)

    if len(sector_frames) == 0:
        continue

    # รวม Logclose ของหุ้นทุกตัวใน sector
    sector_data = pd.concat(sector_frames, axis=1)

    # รวมกับ macro
    combined_df = pd.concat([sector_data, df_filtered_macro], axis=1).ffill()

    # ตัดข้อมูลปี 2025 (ป้องกัน incomplete data)
    combined_df = combined_df[combined_df.index.year != 2025]

    combined_dfs[sector] = combined_df

# ======== แสดงตัวอย่างผลลัพธ์ ========
for sector, combined_df in combined_dfs.items():
    print(f"\n📊 Combined data for sector: {sector}")
    cols_to_show = [c for c in combined_df.columns if c.startswith("Logclose_")] + macro_columns
    cols_to_show_present = [col for col in cols_to_show if col in combined_df.columns]
    display(combined_df[cols_to_show_present].head())



📊 Combined data for sector: Healthcare


,Logclose_AMGN,Logclose_GILD,Logclose_VRTX,Logclose_REGN,Logclose_BIIB,Logclose_ILMN,Logclose_IDXX,Logclose_ISRG,Logclose_ALGN,inflation_rate,ppi_inflation,m2_growth,m1_growth,bond_yield_spread,dxy,nonfarm_payrolls,unemployment_rate,vix_index,real_rate_10y,gdp_growth
Date,,,,,,,,,,,,,,,,,,,,
2015-02-01,5.060821,4.639861,4.782730,6.025479,6.015157,5.247741,4.362015,4.017384,4.049173,-0.000870,-0.070977,0.009868,0.023276,1.21,105.5884,140827.0,5.5,20.97,0.03,0.002258
2015-03-01,5.074236,4.586293,4.770430,6.112531,6.045574,5.196194,4.346917,4.027393,3.985088,-0.000220,-0.074879,-0.000160,-0.007965,1.37,105.5981,140923.0,5.4,13.34,0.17,0.006301
2015-04-01,5.062025,4.610257,4.814458,6.125689,5.924069,5.188678,4.138122,4.009311,4.074822,-0.001040,-0.083533,0.002844,0.000500,1.32,107.0600,141196.0,5.4,15.11,0.07,0.005281
2015-05-01,5.051521,4.720907,4.854293,6.239418,5.983911,5.300649,4.216562,3.992578,4.105449,0.000350,-0.070192,0.002301,-0.008159,1.52,105.6382,141538.0,5.6,12.70,0.18,0.002519
2015-06-01,5.033831,4.762857,4.816079,6.234666,6.001266,5.358530,4.161068,3.985893,4.138521,0.001796,-0.064810,0.003371,0.012181,1.55,107.2238,141709.0,5.3,13.97,0.39,-0.000021


In [7]:
# Select the dataframe for the first stock (e.g., 'BBL') from the combined_dfs dictionary
# This cell is intended to display example dataframes from combined_dfs,
# which are structured by sector, not individual tickers.
# Iterate through the combined_dfs dictionary using sector names as keys.

if 'combined_dfs' in globals() and combined_dfs:
    print("Displaying head of combined dataframes for each sector:")
    for sector, combined_df_example in combined_dfs.items():
        print(f"\n--- Combined dataframe for {sector} ---")
        # Display the columns of the example combined dataframe
        print(f"Columns in the combined dataframe for {sector}:")
        print(combined_df_example.columns)

        # Display the head of the example combined dataframe
        print(f"\nHead of the combined dataframe for {sector}:")
        display(combined_df_example.head())
else:
    print("The 'combined_dfs' dictionary is not available or is empty. Please run the data processing cell first.")

Displaying head of combined dataframes for each sector:

--- Combined dataframe for Healthcare ---
Columns in the combined dataframe for Healthcare:
Index(['Logclose_AMGN', 'Logclose_GILD', 'Logclose_VRTX', 'Logclose_REGN',
       'Logclose_BIIB', 'Logclose_ILMN', 'Logclose_IDXX', 'Logclose_ISRG',
       'Logclose_ALGN', 'Close_AMGN', 'Close_GILD', 'Close_VRTX', 'Close_REGN',
       'Close_BIIB', 'Close_ILMN', 'Close_IDXX', 'Close_ISRG', 'Close_ALGN',
       'inflation_rate', 'm2_growth', 'm1_growth', 'bond_yield_spread', 'dxy',
       'nonfarm_payrolls', 'unemployment_rate', 'vix_index', 'real_rate_10y',
       'ppi_inflation', 'GDP_Monthly', 'gdp_growth'],
      dtype='object')

Head of the combined dataframe for Healthcare:


,Logclose_AMGN,Logclose_GILD,Logclose_VRTX,Logclose_REGN,Logclose_BIIB,Logclose_ILMN,Logclose_IDXX,Logclose_ISRG,Logclose_ALGN,Close_AMGN,...,m1_growth,bond_yield_spread,dxy,nonfarm_payrolls,unemployment_rate,vix_index,real_rate_10y,ppi_inflation,GDP_Monthly,gdp_growth
Date,,,,,,,,,,,,,,,,,,,,,
2015-02-01,5.060821,4.639861,4.782730,6.025479,6.015157,5.247741,4.362015,4.017384,4.049173,157.720001,...,0.023276,1.21,105.5884,140827.0,5.5,20.97,0.03,-0.070977,6013.027543,0.002258
2015-03-01,5.074236,4.586293,4.770430,6.112531,6.045574,5.196194,4.346917,4.027393,3.985088,159.850006,...,-0.007965,1.37,105.5981,140923.0,5.4,13.34,0.17,-0.074879,6051.033159,0.006301
2015-04-01,5.062025,4.610257,4.814458,6.125689,5.924069,5.188678,4.138122,4.009311,4.074822,157.910004,...,0.000500,1.32,107.0600,141196.0,5.4,15.11,0.07,-0.083533,6083.074315,0.005281
2015-05-01,5.051521,4.720907,4.854293,6.239418,5.983911,5.300649,4.216562,3.992578,4.105449,156.259995,...,-0.008159,1.52,105.6382,141538.0,5.6,12.70,0.18,-0.070192,6098.418916,0.002519
2015-06-01,5.033831,4.762857,4.816079,6.234666,6.001266,5.358530,4.161068,3.985893,4.138521,153.520004,...,0.012181,1.55,107.2238,141709.0,5.3,13.97,0.39,-0.064810,6098.290769,-0.000021


** 3 Econometric **

In [8]:
# ============================================================
# 🧠 MULTI-STOCK PIPELINE (Logclose + Dynamic Alignment)
# ============================================================

import re
import numpy as np
import pandas as pd

# --- Safety check ---
if 'combined_dfs' not in globals() or not isinstance(combined_dfs, dict) or not combined_dfs:
    raise RuntimeError("❌ combined_dfs not found or empty. Run previous macro/stock preparation cells first.")

if 'SECTORS' not in globals():
    raise RuntimeError("❌ SECTORS not defined. Make sure the sector mapping dict is loaded.")

# ============================================================
# 1️⃣ Helper functions
# ============================================================
def _norm(s: str) -> str:
    return re.sub(r'[^a-z0-9]+', '', str(s).lower())

PRICE_TOKENS = ("logclose","close","adjclose","price","px","pxlast","last")

def _find_price_like_col(df_one: pd.DataFrame, ticker: str) -> str | None:
    """ค้นหาคอลัมน์ราคาหรือ logclose ที่ตรงกับชื่อหุ้น"""
    if df_one is None or df_one.empty:
        return None
    exacts = [
        f"Logclose_{ticker}",
        f"Close_{ticker}",
        f"close_{ticker}",
        f"AdjClose_{ticker}",
        f"Adj_Close_{ticker}"
    ]
    for c in exacts:
        if c in df_one.columns:
            return c
    generics = ["Logclose", "Close", "Adj Close", "Price", "PX", "PX_LAST", "Last"]
    for c in generics:
        if c in df_one.columns:
            return c
    numeric_cols = [c for c in df_one.columns if pd.api.types.is_numeric_dtype(df_one[c])]
    want = {_norm(t)+_norm(ticker) for t in PRICE_TOKENS}
    best, best_nn = None, -1
    for c in numeric_cols:
        nc = _norm(c)
        if any(tok in nc for tok in want) or any(t in nc for t in PRICE_TOKENS):
            nn = df_one[c].notna().sum()
            if nn > best_nn:
                best, best_nn = c, nn
    return best

def _align(s: pd.Series, idx: pd.Index) -> pd.Series:
    if s is None:
        return None
    s = s.copy()
    s.index = pd.to_datetime(s.index)
    return s.reindex(idx)

# ============================================================
# 2️⃣ Build base index (union of all sectors)
# ============================================================
union_idx = None
for d in combined_dfs.values():
    idx = pd.to_datetime(d.index)
    union_idx = idx if union_idx is None else union_idx.union(idx)
if 'df' in globals() and isinstance(df, pd.DataFrame) and not df.empty:
    union_idx = (pd.to_datetime(df.index) if union_idx is None else union_idx.union(pd.to_datetime(df.index)))
if union_idx is None:
    raise RuntimeError("❌ No index found to build panel.")
union_idx = union_idx.sort_values()

# Base DataFrame
df_master = (df.reindex(union_idx).copy()
             if 'df' in globals() and isinstance(df, pd.DataFrame)
             else pd.DataFrame(index=union_idx))

# ============================================================
# 3️⃣ Collect Logclose per ticker
# ============================================================
TARGET_TICKERS = [ticker for tickers in SECTORS.values() for ticker in tickers]
source_used, missing = {}, []

for ticker in TARGET_TICKERS:
    dsec = None
    for sec, d in combined_dfs.items():
        if f"Logclose_{ticker}" in d.columns or f"Close_{ticker}" in d.columns:
            dsec = d
            break

    price, src, col_used = None, None, None
    if dsec is not None:
        pcol = _find_price_like_col(dsec, ticker)
        if pcol is not None:
            price = _align(pd.to_numeric(dsec[pcol], errors='coerce'), union_idx)
            src, col_used = 'combined', pcol

    if price is None and 'df' in globals() and isinstance(df, pd.DataFrame):
        pcol_g = _find_price_like_col(df, ticker)
        if pcol_g is not None:
            price = _align(pd.to_numeric(df[pcol_g], errors='coerce'), union_idx)
            src, col_used = 'global', pcol_g

    if price is None:
        missing.append(ticker)
        source_used[ticker] = None
        continue

    logc = f"Logclose_{ticker}"
    df_master[logc] = price.astype(float)
    source_used[ticker] = (src, col_used)

print("=== ✅ Logclose columns collected ===")
print(f"Total: {len([c for c in df_master.columns if 'Logclose_' in c])} stocks")

# ============================================================
# 4️⃣ Dynamic Start Date per Ticker
# ============================================================
logclose_cols = [f"Logclose_{t}" for t in TARGET_TICKERS if f"Logclose_{t}" in df_master.columns]

first_valid_dates = {}
for c in logclose_cols:
    valid = df_master[c].first_valid_index()
    first_valid_dates[c] = valid

print("\n📅 First available data per stock:")
for k, v in first_valid_dates.items():
    print(f"{k:25s}: {v}")

# --- สร้าง Dynamic Panel ---
df_dynamic = pd.DataFrame(index=df_master.index)
for c in logclose_cols:
    start_date = first_valid_dates[c]
    if start_date is not None:
        tmp = df_master.loc[start_date:, c]
        df_dynamic[c] = tmp

# --- เติมค่าหลังเริ่มต้นด้วย ffill 1 step (เพื่อความต่อเนื่องเล็กน้อย) ---
df_dynamic = df_dynamic.sort_index().ffill(limit=1)

# ============================================================
# 5️⃣ Final Panel
# ============================================================
df = df_dynamic.copy()

print(f"\n✅ Dynamic Panel Ready: shape={df.shape}")
print(f"🕐 Date range: {df.index.min().date()} → {df.index.max().date()}")
if missing:
    print(f"\n⚠️ Missing Logclose for: {missing}")

# --- Preview ---
try:
    from IPython.display import display
    display(df.iloc[:10, :6])
except Exception:
    print(df.iloc[:10, :6])


=== ✅ Logclose columns collected ===
Total: 9 stocks

📅 First available data per stock:
Logclose_AMGN            : 2015-02-01 00:00:00
Logclose_GILD            : 2015-02-01 00:00:00
Logclose_VRTX            : 2015-02-01 00:00:00
Logclose_REGN            : 2015-02-01 00:00:00
Logclose_BIIB            : 2015-02-01 00:00:00
Logclose_ILMN            : 2015-02-01 00:00:00
Logclose_IDXX            : 2015-02-01 00:00:00
Logclose_ISRG            : 2015-02-01 00:00:00
Logclose_ALGN            : 2015-02-01 00:00:00

✅ Dynamic Panel Ready: shape=(132, 9)
🕐 Date range: 2015-02-01 → 2026-01-01


,Logclose_AMGN,Logclose_GILD,Logclose_VRTX,Logclose_REGN,Logclose_BIIB,Logclose_ILMN
Date,,,,,,
2015-02-01,5.060821,4.639861,4.782730,6.025479,6.015157,5.247741
2015-03-01,5.074236,4.586293,4.770430,6.112531,6.045574,5.196194
2015-04-01,5.062025,4.610257,4.814458,6.125689,5.924069,5.188678
2015-05-01,5.051521,4.720907,4.854293,6.239418,5.983911,5.300649
2015-06-01,5.033831,4.762857,4.816079,6.234666,6.001266,5.358530
2015-07-01,5.173831,4.769497,4.905275,6.316551,5.764501,5.362825
2015-08-01,5.022432,4.654627,4.848273,6.241250,5.694742,5.258680
2015-09-01,4.929570,4.586904,4.645736,6.142338,5.676103,5.141846
2015-10-01,5.063734,4.683334,4.826232,6.323265,5.671638,4.937186


In [9]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 12.0 MB/s eta 0:00:00


In [10]:
# ============================================================
# ⚙️ Build Selectively-Differenced Panel (Diff only I(1))
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# --- Check availability ---
if 'combined_dfs' not in globals() or not isinstance(combined_dfs, dict) or not combined_dfs:
    raise RuntimeError("❌ combined_dfs not found or empty. Please run previous block first.")

# --- Helper function: classify I(0) vs I(1) ---
def classify_order(series, signif=0.05):
    series = series.dropna()
    if len(series) < 10:
        return None
    try:
        adf_result = adfuller(series, autolag='AIC')
        pval = adf_result[1]
        return "I(0)" if pval < signif else "I(1)"
    except Exception:
        return None

# --- Detect integration order for all macros ---
macro_candidates = [
    "inflation_rate",
    "ppi_inflation",
    "m2_growth",
    "m1_growth",
    "bond_yield_spread",
    "dxy",
    "nonfarm_payrolls",
    "unemployment_rate",
    "vix_index",
    "real_rate_10y",
    "gdp_growth"
]

macro_orders = {}
for sector, df_sec in combined_dfs.items():
    for m in macro_candidates:
        if m in df_sec.columns and m not in macro_orders:
            order = classify_order(df_sec[m])
            if order:
                macro_orders[m] = order

print("=== Integration Order Detected ===")
for k, v in macro_orders.items():
    print(f"{k:25s} → {v}")

# --- Build differenced dataset selectively ---
combined_dfs_selective_diff = {}

for sector, df_sec in combined_dfs.items():
    df_sec = df_sec.copy()
    out = pd.DataFrame(index=df_sec.index)

    # 1️⃣ หุ้น: ใช้ Δlog เสมอ
    for col in df_sec.columns:
        if col.startswith("Logclose_"):
            out[col] = df_sec[col].diff()

    # 2️⃣ Macro: diff เฉพาะ I(1)
    for m in macro_candidates:
        if m not in df_sec.columns:
            continue
        if macro_orders.get(m, "I(1)") == "I(1)":
            out[m] = df_sec[m].diff()
        else:  # I(0)
            out[m] = df_sec[m]

    # 3️⃣ Drop NA
    combined_dfs_selective_diff[sector] = out.dropna(how='all')

print("\n✅ Built 'combined_dfs_selective_diff' successfully.")
print("Diffed Macros (I(1)):", [k for k, v in macro_orders.items() if v == "I(1)"])
print("Kept in Levels (I(0)):", [k for k, v in macro_orders.items() if v == "I(0)"])


=== Integration Order Detected ===
inflation_rate            → I(1)
ppi_inflation             → I(1)
m2_growth                 → I(0)
m1_growth                 → I(0)
bond_yield_spread         → I(1)
dxy                       → I(1)
nonfarm_payrolls          → I(1)
unemployment_rate         → I(0)
vix_index                 → I(0)
real_rate_10y             → I(1)
gdp_growth                → I(0)

✅ Built 'combined_dfs_selective_diff' successfully.
Diffed Macros (I(1)): ['inflation_rate', 'ppi_inflation', 'bond_yield_spread', 'dxy', 'nonfarm_payrolls', 'real_rate_10y']
Kept in Levels (I(0)): ['m2_growth', 'm1_growth', 'unemployment_rate', 'vix_index', 'gdp_growth']


In [11]:
# ============================================================
# 🔍 CHECK DATA CONSISTENCY BEFORE ARDL (Selective Diff Version)
# ============================================================

import pandas as pd

print("=== Checking macro variable presence in combined_dfs_selective_diff ===\n")

# List ของตัวแปรที่ ARDL ต้องใช้ (ปรับให้ตรงกับ macro_vars จริง)
macro_vars = [
    "inflation_rate",
    "ppi_inflation",
    "m2_growth",
    "m1_growth",
    "bond_yield_spread",
    "dxy",
    "nonfarm_payrolls",
    "unemployment_rate",
    "vix_index",
    "real_rate_10y",
    "gdp_growth"
]

summary_records = []

for sector, df_sec in combined_dfs_selective_diff.items():
    present = [m for m in macro_vars if m in df_sec.columns]
    missing = [m for m in macro_vars if m not in df_sec.columns]
    logclose_cols = [c for c in df_sec.columns if c.startswith("Logclose_")]

    print(f"📂 Sector: {sector}")
    print(f"  • Stocks found: {len(logclose_cols)} ({', '.join(logclose_cols[:5])}{'...' if len(logclose_cols)>5 else ''})")
    print(f"  • Macros present ({len(present)}): {present}")
    print(f"  • Macros missing ({len(missing)}): {missing}")
    print("-" * 80)

    summary_records.append({
        "Sector": sector,
        "N_stocks": len(logclose_cols),
        "Macros_present": len(present),
        "Macros_missing": len(missing),
        "Missing_list": ", ".join(missing)
    })

# รวมเป็นตารางภาพรวม
df_summary = pd.DataFrame(summary_records)
print("\n=== Summary of Macro Coverage per Sector ===")
display(df_summary)

# # ตัวอย่าง dataframe จริง
# sector_to_inspect = list(combined_dfs_selective_diff.keys())[0]
# print(f"\n\n=== Example: head() of combined_dfs_selective_diff['{sector_to_inspect}'] ===")
# display(combined_dfs_selective_diff[sector_to_inspect].head(10))

# ============================================================
# 🧹 CLEAN: Drop first NaN row from all selective-diff dataframes
# ============================================================

for sector in combined_dfs_selective_diff:
    df_tmp = combined_dfs_selective_diff[sector].copy()

    # ลบแถวแรกที่เป็น NaN ทั้งหมด
    df_tmp = df_tmp.dropna(how="all")

    # ลบ NaN เฉพาะคอลัมน์หลัก (ถ้าบาง macro เริ่มช้า)
    df_tmp = df_tmp.dropna(thresh=int(0.8 * len(df_tmp.columns)))  # อย่างน้อย 80% ต้องไม่ NA

    combined_dfs_selective_diff[sector] = df_tmp

print("✅ Removed first NaN row and cleaned selective-diff dataframes successfully.\n")

# ตรวจสอบตัวอย่างหลัง clean
sector_to_inspect = list(combined_dfs_selective_diff.keys())[0]
print(f"=== Example after cleaning: '{sector_to_inspect}' ===")
display(combined_dfs_selective_diff[sector_to_inspect].head(10))



=== Checking macro variable presence in combined_dfs_selective_diff ===

📂 Sector: Healthcare
  • Stocks found: 9 (Logclose_AMGN, Logclose_GILD, Logclose_VRTX, Logclose_REGN, Logclose_BIIB...)
  • Macros present (11): ['inflation_rate', 'ppi_inflation', 'm2_growth', 'm1_growth', 'bond_yield_spread', 'dxy', 'nonfarm_payrolls', 'unemployment_rate', 'vix_index', 'real_rate_10y', 'gdp_growth']
  • Macros missing (0): []
--------------------------------------------------------------------------------

=== Summary of Macro Coverage per Sector ===


,Sector,N_stocks,Macros_present,Macros_missing,Missing_list
0,Healthcare,9,11,0,


✅ Removed first NaN row and cleaned selective-diff dataframes successfully.

=== Example after cleaning: 'Healthcare' ===


,Logclose_AMGN,Logclose_GILD,Logclose_VRTX,Logclose_REGN,Logclose_BIIB,Logclose_ILMN,Logclose_IDXX,Logclose_ISRG,Logclose_ALGN,inflation_rate,ppi_inflation,m2_growth,m1_growth,bond_yield_spread,dxy,nonfarm_payrolls,unemployment_rate,vix_index,real_rate_10y,gdp_growth
Date,,,,,,,,,,,,,,,,,,,,
2015-03-01,0.013415,-0.053568,-0.012300,0.087052,0.030417,-0.051546,-0.015098,0.010010,-0.064085,0.000650,-0.003902,-0.000160,-0.007965,0.16,0.0097,96.0,5.4,13.34,0.14,0.006301
2015-04-01,-0.012211,0.023964,0.044028,0.013158,-0.121505,-0.007516,-0.208795,-0.018082,0.089734,-0.000820,-0.008654,0.002844,0.000500,-0.05,1.4619,273.0,5.4,15.11,-0.10,0.005281
2015-05-01,-0.010504,0.110649,0.039835,0.113728,0.059842,0.111971,0.078440,-0.016733,0.030627,0.001391,0.013341,0.002301,-0.008159,0.20,-1.4218,342.0,5.6,12.70,0.11,0.002519
2015-06-01,-0.017690,0.041951,-0.038214,-0.004752,0.017355,0.057881,-0.055494,-0.006686,0.033072,0.001445,0.005382,0.003371,0.012181,0.03,1.5856,171.0,5.3,13.97,0.21,-0.000021
2015-07-01,0.140000,0.006640,0.089196,0.081885,-0.236765,0.004296,0.125686,0.095723,-0.000159,0.000461,-0.002978,0.003634,0.006711,0.19,-0.2137,282.0,5.2,16.09,0.15,0.005345
2015-08-01,-0.151399,-0.114871,-0.057002,-0.075301,-0.069760,-0.104145,-0.017476,-0.042569,-0.102353,0.000156,-0.005158,0.003711,-0.006877,-0.21,1.7718,134.0,5.1,12.12,-0.08,-0.000053
2015-09-01,-0.092862,-0.067722,-0.202537,-0.098912,-0.018639,-0.116835,0.038160,-0.105959,0.002823,-0.002325,-0.010871,0.005048,0.007832,-0.06,1.6687,150.0,5.0,31.40,0.14,0.001525
2015-10-01,0.134164,0.096430,0.180495,0.180927,-0.004465,-0.204660,-0.078854,0.077472,0.142607,0.001188,0.005647,0.004212,-0.006638,-0.06,0.5933,304.0,5.0,22.55,-0.01,0.002076
2015-11-01,0.018292,-0.020273,0.036368,-0.023397,-0.012644,0.249591,0.031557,0.046082,0.019365,0.003087,0.002512,0.007452,0.018614,0.00,-0.8360,229.0,5.1,15.07,0.04,-0.001593


# ARDL + ECM

In [67]:
# ============================================================
# ✅ RUN 3 ARDL-ECM LAG REGIMES FOR *ALL STOCKS* (ALL SECTORS)
#    FIXED / ACF-PACF / AIC-greedy
#    Walk-forward: ECM vs AR(p) vs RW + DM tests
#    + BUILD df_alpha / df_beta / df_gamma FOR DOWNSTREAM FORECAST
#    Export Excel
# ============================================================

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from scipy import stats
from statsmodels.tsa.stattools import acf, pacf

# -----------------------------
# 0) SAFETY: which dataset?
# -----------------------------
USE_DATA = "combined_dfs"  # or "combined_dfs_selective_diff"

if USE_DATA not in globals() or not isinstance(globals()[USE_DATA], dict) or not globals()[USE_DATA]:
    raise RuntimeError(f"❌ {USE_DATA} not found or empty.")
sector_dfs = globals()[USE_DATA]

# -----------------------------
# CONFIG
# -----------------------------
macro_vars = [
    "inflation_rate","ppi_inflation","m2_growth","m1_growth","bond_yield_spread","dxy",
    "nonfarm_payrolls","unemployment_rate","vix_index","real_rate_10y","gdp_growth"
]

# AIC search bounds (fast)
CRITERION = "AIC"     # "AIC" or "BIC"
P_MIN, P_MAX = 1, 2
Q_MAX_AIC = 2

# FIXED lag regime
P_FIXED = 2
Q_FIXED = 6
Q_CAP   = 6          # cap for ACF/PACF q selection too

# Walk-forward evaluation
TRAIN_END = "2020-12-01"
TEST_END  = None
MIN_TRAIN = 60
REFIT_EVERY = 6
MAX_TEST_POINTS = 120

EXPORT_PATH = "ALL_SECTORS_3MODELS_ECM_vs_AR_vs_RW.xlsx"

# ============================================================
# 1) Utilities (columns + metrics + DM)
# ============================================================
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d.columns = d.columns.astype(str).str.strip()
    return d

def _safe_mape(y_true, y_pred, eps=1e-8):
    denom = np.maximum(np.abs(y_true), eps)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

def metrics(y_true, y_pred) -> dict:
    err = y_true - y_pred
    return {
        "MAE": float(np.mean(np.abs(err))),
        "RMSE": float(np.sqrt(np.mean(err**2))),
        "MAPE(%)": float(_safe_mape(y_true, y_pred)),
        "n_test": int(len(y_true))
    }

def dm_test(e1: np.ndarray, e2: np.ndarray, power: int = 2):
    e1 = np.asarray(e1); e2 = np.asarray(e2)
    if power == 1:
        d = np.abs(e1) - np.abs(e2)
    else:
        d = (e1**2) - (e2**2)
    d = d[~np.isnan(d)]
    T = len(d)
    if T < 10:
        return np.nan, np.nan
    mean_d = d.mean()
    var_d = d.var(ddof=1)
    if var_d <= 1e-12:
        return np.nan, np.nan
    dm_stat = mean_d / np.sqrt(var_d / T)
    pval = 2 * (1 - stats.t.cdf(np.abs(dm_stat), df=T-1))
    return float(dm_stat), float(pval)

# ============================================================
# 2) ECM (manual OLS)
# ============================================================
def build_ecm_ols(y: pd.Series, X: pd.DataFrame, p: int, q_dict: dict, add_const: bool = True):
    y = y.astype(float)
    X = X.astype(float)

    dy = y.diff()
    Z = pd.DataFrame(index=y.index)

    if add_const:
        Z["const"] = 1.0

    # levels (t-1)
    Z[f"L1.{y.name}"] = y.shift(1)
    for m in X.columns:
        Z[f"L1.{m}"] = X[m].shift(1)

    # Δy lags 1..p-1
    for i in range(1, max(p, 1)):
        if i <= p - 1:
            Z[f"D.L{i}.{y.name}"] = dy.shift(i)

    # Δx lags 0..q_m-1
    for m in X.columns:
        q_m = int(q_dict.get(m, 0))
        dx = X[m].diff()
        for k in range(0, q_m):
            Z[f"D.{m}" if k == 0 else f"D.L{k}.{m}"] = dx.shift(k)

    df_all = pd.concat([dy.rename("D.y"), Z], axis=1).dropna()
    if df_all.shape[0] < 30:
        raise ValueError("Too few observations after alignment.")

    y_dep = df_all["D.y"]
    X_reg = df_all.drop(columns=["D.y"])
    res = sm.OLS(y_dep, X_reg).fit()
    return res, X_reg

def get_ic(res, criterion="AIC"):
    return res.aic if str(criterion).upper() == "AIC" else res.bic

# ============================================================
# 3) AIC Greedy selector (fast)
# ============================================================
def greedy_select_q_AIC(y, X, p, q_max=2, add_const=True, criterion="AIC"):
    q_dict = {m: 0 for m in X.columns}
    best_res, _ = build_ecm_ols(y, X, p=p, q_dict=q_dict, add_const=add_const)
    best_ic = get_ic(best_res, criterion=criterion)

    for _ in range(2):
        improved = False
        for m in X.columns:
            cur_q = q_dict[m]
            best_local_q = cur_q
            best_local_ic = best_ic
            best_local_res = best_res

            for q_try in range(0, q_max + 1):
                if q_try == cur_q:
                    continue
                q_tmp = dict(q_dict); q_tmp[m] = q_try
                try:
                    res_try, _ = build_ecm_ols(y, X, p=p, q_dict=q_tmp, add_const=add_const)
                    ic_try = get_ic(res_try, criterion=criterion)
                    if ic_try < best_local_ic - 1e-9:
                        best_local_ic = ic_try
                        best_local_q = q_try
                        best_local_res = res_try
                except Exception:
                    continue

            if best_local_q != cur_q:
                q_dict[m] = best_local_q
                best_ic = best_local_ic
                best_res = best_local_res
                improved = True

        if not improved:
            break

    return q_dict, best_res, best_ic

def select_pq_AIC_fast(y, X, p_min=1, p_max=2, q_max=2, add_const=True, criterion="AIC"):
    best = None
    for p in range(p_min, p_max + 1):
        try:
            q_dict, res, ic = greedy_select_q_AIC(y, X, p=p, q_max=q_max, add_const=add_const, criterion=criterion)
            if (best is None) or (ic < best["ic"]):
                best = {"p": p, "q_dict": q_dict, "res": res, "ic": ic}
        except Exception:
            continue
    if best is None:
        raise ValueError("cannot select ARDL order by AIC")
    return best["p"], best["q_dict"], best["res"], best["ic"]

# ============================================================
# 4) ACF/PACF selector
# ============================================================
def _sig_lags_from_acf(series: pd.Series, max_lag: int, alpha: float = 0.05):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if len(s) < max(30, max_lag + 5):
        return []
    acf_vals, confint = acf(s, nlags=max_lag, alpha=alpha, fft=True)
    sig = []
    for k in range(1, max_lag + 1):
        lo, hi = confint[k]
        if not (lo <= 0 <= hi):
            sig.append(k)
    return sig

def _choose_p_from_pacf_dy(y: pd.Series, p_min: int, p_max: int, alpha: float = 0.05):
    dy = pd.to_numeric(y, errors="coerce").diff().dropna()
    if len(dy) < max(30, p_max + 5):
        return p_min
    pacf_vals, confint = pacf(dy, nlags=p_max, alpha=alpha, method="ywm")
    sig_lags = []
    for k in range(1, p_max + 1):
        lo, hi = confint[k]
        if not (lo <= 0 <= hi):
            sig_lags.append(k)
    p = (max(sig_lags) + 1) if sig_lags else p_min
    return int(np.clip(p, p_min, p_max))

def _choose_q_from_acf_dx(x: pd.Series, q_cap: int, alpha: float = 0.05):
    dx = pd.to_numeric(x, errors="coerce").diff()
    sig_lags = _sig_lags_from_acf(dx, max_lag=q_cap, alpha=alpha)
    if not sig_lags:
        return 0
    q = max(sig_lags) + 1
    return int(np.clip(q, 0, q_cap))

def select_pq_ACF(y: pd.Series, X: pd.DataFrame, p_min=1, p_max=2, q_cap=6, alpha=0.05, add_const=True, criterion="AIC"):
    p_sel = _choose_p_from_pacf_dy(y, p_min, p_max, alpha=alpha)
    q_dict = {m: _choose_q_from_acf_dx(X[m], q_cap=q_cap, alpha=alpha) for m in X.columns}
    res, _ = build_ecm_ols(y, X, p=p_sel, q_dict=q_dict, add_const=add_const)
    ic = get_ic(res, criterion=criterion)
    return p_sel, q_dict, res, ic

# ============================================================
# 5) FIXED selector
# ============================================================
def select_pq_FIXED(y: pd.Series, X: pd.DataFrame, p_fixed=2, q_fixed=6, add_const=True, criterion="AIC"):
    q_dict = {m: int(q_fixed) for m in X.columns}
    res, _ = build_ecm_ols(y, X, p=p_fixed, q_dict=q_dict, add_const=add_const)
    ic = get_ic(res, criterion=criterion)
    return int(p_fixed), q_dict, res, ic

# ============================================================
# 6) Walk-forward eval
# ============================================================
def fit_ecm_from_train(df_train: pd.DataFrame, y_col: str, x_cols: list, p: int, q_dict: dict):
    y = df_train[y_col].copy(); y.name = y_col
    X = df_train[x_cols].copy()
    res, _ = build_ecm_ols(y, X, p=p, q_dict=q_dict, add_const=True)
    return res

def ar_one_step_forecast(y_train: pd.Series, y_all: pd.Series, i: int, p: int):
    ytr = y_train.dropna()
    if len(ytr) < max(p + 5, 20):
        return float(y_all.shift(1).iloc[i])
    fit = AutoReg(ytr, lags=p, old_names=False).fit()
    fc = fit.predict(start=len(ytr), end=len(ytr))
    return float(fc.iloc[0])

def rw_one_step_forecast(y_all: pd.Series, i: int):
    return float(y_all.shift(1).iloc[i])

def ecm_one_step_forecast_from_params(df_all: pd.DataFrame, i: int, y_col: str, x_cols: list, p: int, q_dict: dict, ecm_fit):
    params = ecm_fit.params
    row = {}

    if "const" in params.index:
        row["const"] = 1.0

    row[f"L1.{y_col}"] = df_all[y_col].shift(1).iloc[i]
    for m in x_cols:
        row[f"L1.{m}"] = df_all[m].shift(1).iloc[i]

    dy = df_all[y_col].diff()
    for j in range(1, max(p, 1)):
        if j <= p - 1:
            nm = f"D.L{j}.{y_col}"
            if nm in params.index:
                row[nm] = dy.shift(j).iloc[i]

    for m in x_cols:
        q = int(q_dict.get(m, 0))
        if q <= 0:
            continue
        dx = df_all[m].diff()
        for k in range(0, q):
            nm = f"D.{m}" if k == 0 else f"D.L{k}.{m}"
            if nm in params.index:
                row[nm] = dx.shift(k).iloc[i]

    exog_names = list(params.index)
    x_vec = np.array([row.get(n, 0.0) for n in exog_names], dtype=float)
    dy_hat = float(np.dot(x_vec, params.values))
    y_prev = df_all[y_col].shift(1).iloc[i]
    return float(y_prev + dy_hat)

def evaluate_ticker(df: pd.DataFrame,
                    ticker: str,
                    y_col: str,
                    x_cols: list,
                    p: int,
                    q_dict: dict,
                    train_end: str,
                    test_end: str = None,
                    min_train: int = 60,
                    refit_every: int = 6,
                    max_test_points: int = 120):

    d = df.copy().sort_index()
    if test_end is not None:
        d = d.loc[:pd.to_datetime(test_end)]

    train_end_dt = pd.to_datetime(train_end)
    d_train0 = d.loc[d.index <= train_end_dt]
    d_test   = d.loc[d.index >  train_end_dt]

    if len(d_train0) < min_train:
        raise ValueError(f"{ticker}: not enough train data ({len(d_train0)})")

    all_idx = d.index.tolist()
    test_indices = [i for i, dt in enumerate(all_idx) if dt in d_test.index]
    if max_test_points is not None:
        test_indices = test_indices[:max_test_points]

    y_true, yhat_ecm, yhat_ar, yhat_rw = [], [], [], []
    ecm_fit = None

    for step, i in enumerate(test_indices):
        d_train = d.iloc[:i].copy()
        if len(d_train) < min_train:
            continue

        if (ecm_fit is None) or (step % refit_every == 0):
            ecm_fit = fit_ecm_from_train(d_train, y_col, x_cols, p, q_dict)

        y_pred_ecm = ecm_one_step_forecast_from_params(d, i, y_col, x_cols, p, q_dict, ecm_fit)

        y_series_train = d_train[y_col]
        y_series_all   = d[y_col]
        y_pred_ar = ar_one_step_forecast(y_series_train, y_series_all, i, p=max(1, p))
        y_pred_rw = rw_one_step_forecast(y_series_all, i)

        y_true.append(float(d[y_col].iloc[i]))
        yhat_ecm.append(y_pred_ecm)
        yhat_ar.append(y_pred_ar)
        yhat_rw.append(y_pred_rw)

    y_true = np.array(y_true, dtype=float)
    yhat_ecm = np.array(yhat_ecm, dtype=float)
    yhat_ar  = np.array(yhat_ar, dtype=float)
    yhat_rw  = np.array(yhat_rw, dtype=float)

    out = {"ticker": ticker, "p": int(p), "q_max": int(max(q_dict.values()) if q_dict else 0)}
    out.update({f"ECM_{k}": v for k, v in metrics(y_true, yhat_ecm).items()})
    out.update({f"AR_{k}":  v for k, v in metrics(y_true, yhat_ar).items()})
    out.update({f"RW_{k}":  v for k, v in metrics(y_true, yhat_rw).items()})

    e_ecm = y_true - yhat_ecm
    e_ar  = y_true - yhat_ar
    e_rw  = y_true - yhat_rw

    dm_ecm_ar, p_ecm_ar = dm_test(e_ecm, e_ar, power=2)
    dm_ecm_rw, p_ecm_rw = dm_test(e_ecm, e_rw, power=2)

    out["DM(ECM-AR)"] = dm_ecm_ar
    out["pval(ECM-AR)"] = p_ecm_ar
    out["DM(ECM-RW)"] = dm_ecm_rw
    out["pval(ECM-RW)"] = p_ecm_rw
    return out

# ============================================================
# 7) ✅ NEW: convert ECM fit -> df_alpha / df_beta / df_gamma rows
# ============================================================
def ecm_to_tables(ecm_fit, y_col, x_cols, model_name, sector, ticker):
    params = ecm_fit.params
    pvals  = ecm_fit.pvalues

    # alpha
    alpha_key = f"L1.{y_col}"
    alpha = float(params.get(alpha_key, np.nan))
    alpha_p = float(pvals.get(alpha_key, np.nan))

    alpha_row = {
        "Sector": sector,
        "Stock": ticker,
        "Model": model_name,
        "Alpha": alpha,
        "Alpha_pval": alpha_p
    }

    # beta long-run: -L1.x / alpha , const included
    beta_rows = []
    if np.isfinite(alpha) and alpha != 0:
        if "const" in params.index:
            beta_rows.append({
                "Sector": sector, "Stock": ticker, "Model": model_name,
                "Variable": "const", "coef": float(-params["const"] / alpha)
            })
        for m in x_cols:
            k = f"L1.{m}"
            if k in params.index:
                beta_rows.append({
                    "Sector": sector, "Stock": ticker, "Model": model_name,
                    "Variable": m, "coef": float(-params[k] / alpha)
                })

    # gamma short-run: keep ONLY D.* terms (forecast expects D.Lk.)
    gamma_rows = []
    for name in params.index:
        if name.startswith("D."):
            gamma_rows.append({
                "Sector": sector, "Stock": ticker, "Model": model_name,
                "Variable": name,
                "Coef": float(params[name]),
                "pval": float(pvals.get(name, np.nan))
            })

    return alpha_row, beta_rows, gamma_rows

# ============================================================
# 8) MAIN runner (produces res_df, orders_df, df_alpha/beta/gamma)
# ============================================================
def run_all_sectors_3models(sector_dfs: dict,
                            macro_vars: list,
                            train_end: str,
                            test_end: str = None,
                            export_path: str = None,
                            verbose: bool = True):

    all_rows = []
    orders_dump = []

    alpha_list = []
    beta_list  = []
    gamma_list = []

    for sector, df_sec in sector_dfs.items():
        d = clean_columns(df_sec)
        stock_cols = [c for c in d.columns if str(c).startswith("Logclose_")]
        if not stock_cols:
            if verbose: print(f"⚠️ {sector}: no Logclose_ columns")
            continue

        X_cols = [m for m in macro_vars if m in d.columns]
        if not X_cols:
            if verbose: print(f"⚠️ {sector}: no macro columns found")
            continue

        if verbose:
            print(f"\n===== 📂 Sector: {sector} | stocks={len(stock_cols)} | macros={len(X_cols)} =====")

        d_train0 = d.loc[d.index <= pd.to_datetime(train_end)].copy()
        if len(d_train0) < MIN_TRAIN:
            if verbose: print(f"⚠️ {sector}: not enough train rows ({len(d_train0)})")
            continue

        for y_col in stock_cols:
            ticker = y_col.replace("Logclose_", "")
            try:
                y_train = d_train0[y_col].copy(); y_train.name = y_col
                X_train = d_train0[X_cols].copy()

                # 3 lag regimes (chosen on TRAIN only)
                p_fix, q_fix, res_fix, ic_fix = select_pq_FIXED(
                    y_train, X_train, p_fixed=P_FIXED, q_fixed=Q_FIXED,
                    add_const=True, criterion=CRITERION
                )
                p_acf, q_acf, res_acf, ic_acf = select_pq_ACF(
                    y_train, X_train, p_min=P_MIN, p_max=P_MAX, q_cap=Q_CAP,
                    alpha=0.05, add_const=True, criterion=CRITERION
                )
                p_aic, q_aic, res_aic, ic_aic = select_pq_AIC_fast(
                    y_train, X_train, p_min=P_MIN, p_max=P_MAX, q_max=Q_MAX_AIC,
                    add_const=True, criterion=CRITERION
                )

                model_orders = [
                    ("FIXED", p_fix, q_fix, res_fix, ic_fix),
                    ("ACF",   p_acf, q_acf, res_acf, ic_acf),
                    ("AIC",   p_aic, q_aic, res_aic, ic_aic),
                ]

                for model_name, p_sel, q_dict, res_train_fit, ic in model_orders:
                    # ---------------------------
                    # ✅ 1) store alpha/beta/gamma from TRAIN fit
                    # ---------------------------
                    a_row, b_rows, g_rows = ecm_to_tables(
                        ecm_fit=res_train_fit, y_col=y_col, x_cols=X_cols,
                        model_name=model_name, sector=sector, ticker=ticker
                    )
                    alpha_list.append(a_row)
                    beta_list.extend(b_rows)
                    gamma_list.extend(g_rows)

                    # ---------------------------
                    # ✅ 2) evaluate walk-forward
                    # ---------------------------
                    row = evaluate_ticker(
                        df=d, ticker=ticker, y_col=y_col, x_cols=X_cols,
                        p=p_sel, q_dict=q_dict,
                        train_end=train_end, test_end=test_end,
                        min_train=MIN_TRAIN,
                        refit_every=REFIT_EVERY,
                        max_test_points=MAX_TEST_POINTS
                    )

                    row["sector"] = sector
                    row["model"] = model_name
                    row["ticker"] = ticker
                    row[CRITERION] = float(ic)
                    for m in X_cols:
                        row[f"q_{m}"] = int(q_dict.get(m, 0))

                    all_rows.append(row)

                    orders_dump.append({
                        "sector": sector, "ticker": ticker, "model": model_name,
                        "p": int(p_sel),
                        "q_max": int(max(q_dict.values()) if q_dict else 0),
                        CRITERION: float(ic),
                        **{f"q_{m}": int(q_dict.get(m, 0)) for m in X_cols}
                    })

                    if verbose:
                        print(f"✅ {sector}-{ticker} [{model_name}] "
                              f"RMSE: ECM={row['ECM_RMSE']:.6f} | AR={row['AR_RMSE']:.6f} | RW={row['RW_RMSE']:.6f}")

            except Exception as e:
                if verbose:
                    print(f"⚠️ {sector}-{ticker} failed: {e}")
                continue

    if not all_rows:
        raise RuntimeError("❌ No results produced (all failed).")

    res_df = pd.DataFrame(all_rows)
    orders_df = pd.DataFrame(orders_dump)

    df_alpha = pd.DataFrame(alpha_list)
    df_beta  = pd.DataFrame(beta_list)
    df_gamma = pd.DataFrame(gamma_list)

    # nice ordering
    front = ["sector","model","ticker","p","q_max",CRITERION,
             "ECM_MAE","ECM_RMSE","ECM_MAPE(%)","ECM_n_test",
             "AR_MAE","AR_RMSE","AR_MAPE(%)","AR_n_test",
             "RW_MAE","RW_RMSE","RW_MAPE(%)","RW_n_test",
             "DM(ECM-AR)","pval(ECM-AR)","DM(ECM-RW)","pval(ECM-RW)"]
    cols = [c for c in front if c in res_df.columns] + [c for c in res_df.columns if c not in front]
    res_df = res_df[cols].sort_values(by=["sector","ticker","model","ECM_RMSE"]).reset_index(drop=True)

    if export_path:
        with pd.ExcelWriter(export_path, engine="openpyxl") as writer:
            res_df.to_excel(writer, sheet_name="Results_All", index=False)
            orders_df.to_excel(writer, sheet_name="Chosen_Lags", index=False)
            df_alpha.to_excel(writer, sheet_name="Alpha", index=False)
            df_beta.to_excel(writer, sheet_name="Beta_LongRun", index=False)
            df_gamma.to_excel(writer, sheet_name="Gamma_ShortRun", index=False)

            for sector in sorted(res_df["sector"].unique()):
                tmp = res_df[res_df["sector"] == sector].copy()
                tmp.to_excel(writer, sheet_name=str(sector)[:31], index=False)

        print(f"\n✅ Exported → {export_path}")

    return res_df, orders_df, df_alpha, df_beta, df_gamma

# -----------------------------
# RUN
# -----------------------------
res_all, lags_all, df_alpha, df_beta, df_gamma = run_all_sectors_3models(
    sector_dfs=sector_dfs,
    macro_vars=macro_vars,
    train_end=TRAIN_END,
    test_end=TEST_END,
    export_path=EXPORT_PATH,
    verbose=True
)

print("\n=== DONE ===")
display(res_all.head(30))
display(lags_all.head(30))
display(df_alpha.head(10))
display(df_beta.head(10))
display(df_gamma.head(10))



===== 📂 Sector: Healthcare | stocks=9 | macros=11 =====
✅ Healthcare-AMGN [FIXED] RMSE: ECM=1.217195 | AR=0.076331 | RW=0.075389
✅ Healthcare-AMGN [ACF] RMSE: ECM=0.112139 | AR=0.076743 | RW=0.075389
✅ Healthcare-AMGN [AIC] RMSE: ECM=0.087483 | AR=0.076743 | RW=0.075389
✅ Healthcare-GILD [FIXED] RMSE: ECM=0.733808 | AR=0.079856 | RW=0.078396
✅ Healthcare-GILD [ACF] RMSE: ECM=0.129017 | AR=0.079192 | RW=0.078396
✅ Healthcare-GILD [AIC] RMSE: ECM=0.109582 | AR=0.079192 | RW=0.078396
✅ Healthcare-VRTX [FIXED] RMSE: ECM=0.421544 | AR=0.069209 | RW=0.069014
✅ Healthcare-VRTX [ACF] RMSE: ECM=0.116594 | AR=0.069567 | RW=0.069014
✅ Healthcare-VRTX [AIC] RMSE: ECM=0.123825 | AR=0.069209 | RW=0.069014
✅ Healthcare-REGN [FIXED] RMSE: ECM=0.459000 | AR=0.083155 | RW=0.078029
✅ Healthcare-REGN [ACF] RMSE: ECM=0.168521 | AR=0.082501 | RW=0.078029
✅ Healthcare-REGN [AIC] RMSE: ECM=0.157405 | AR=0.083155 | RW=0.078029
✅ Healthcare-BIIB [FIXED] RMSE: ECM=0.818798 | AR=0.095318 | RW=0.088650
✅ Healthca

,sector,model,ticker,p,q_max,AIC,ECM_MAE,ECM_RMSE,ECM_MAPE(%),ECM_n_test,...,q_ppi_inflation,q_m2_growth,q_m1_growth,q_bond_yield_spread,q_dxy,q_nonfarm_payrolls,q_unemployment_rate,q_vix_index,q_real_rate_10y,q_gdp_growth
0,Healthcare,ACF,ALGN,1,4,-59.085005,0.195675,0.247929,3.405712,49,...,2,4,2,0,0,0,0,0,0,3
1,Healthcare,AIC,ALGN,1,2,-81.434354,0.209231,0.258614,3.662961,49,...,1,1,0,2,0,0,0,2,0,0
2,Healthcare,FIXED,ALGN,2,6,-2634.073782,0.594141,0.752485,10.350089,49,...,6,6,6,6,6,6,6,6,6,6
3,Healthcare,ACF,AMGN,1,4,-151.624879,0.086536,0.112139,1.567685,49,...,2,4,2,0,0,0,0,0,0,3
4,Healthcare,AIC,AMGN,1,1,-175.983374,0.069783,0.087483,1.255004,49,...,0,0,0,0,1,1,0,0,0,0
5,Healthcare,FIXED,AMGN,2,6,-2683.730245,0.556055,1.217195,10.150618,49,...,6,6,6,6,6,6,6,6,6,6
6,Healthcare,ACF,BIIB,1,4,-98.667722,0.129600,0.166303,2.348226,49,...,2,4,2,0,0,0,0,0,0,3
7,Healthcare,AIC,BIIB,1,1,-124.044249,0.084316,0.113610,1.529134,49,...,0,0,1,0,1,0,0,0,0,0
8,Healthcare,FIXED,BIIB,2,6,-2578.683342,0.568414,0.818798,10.280378,49,...,6,6,6,6,6,6,6,6,6,6
9,Healthcare,ACF,GILD,1,4,-155.620333,0.097202,0.129017,2.243645,49,...,2,4,2,0,0,0,0,0,0,3


,sector,ticker,model,p,q_max,AIC,q_inflation_rate,q_ppi_inflation,q_m2_growth,q_m1_growth,q_bond_yield_spread,q_dxy,q_nonfarm_payrolls,q_unemployment_rate,q_vix_index,q_real_rate_10y,q_gdp_growth
0,Healthcare,AMGN,FIXED,2,6,-2683.730245,6,6,6,6,6,6,6,6,6,6,6
1,Healthcare,AMGN,ACF,1,4,-151.624879,4,2,4,2,0,0,0,0,0,0,3
2,Healthcare,AMGN,AIC,1,1,-175.983374,0,0,0,0,0,1,1,0,0,0,0
3,Healthcare,GILD,FIXED,2,6,-2651.142579,6,6,6,6,6,6,6,6,6,6,6
4,Healthcare,GILD,ACF,1,4,-155.620333,4,2,4,2,0,0,0,0,0,0,3
5,Healthcare,GILD,AIC,1,1,-180.473755,1,0,0,0,1,1,0,0,0,0,1
6,Healthcare,VRTX,FIXED,2,6,-2509.567307,6,6,6,6,6,6,6,6,6,6,6
7,Healthcare,VRTX,ACF,1,4,-99.112917,4,2,4,2,0,0,0,0,0,0,3
8,Healthcare,VRTX,AIC,2,2,-137.563423,1,0,0,1,2,2,0,0,2,2,2
9,Healthcare,REGN,FIXED,2,6,-2628.318626,6,6,6,6,6,6,6,6,6,6,6


,Sector,Stock,Model,Alpha,Alpha_pval
0,Healthcare,AMGN,FIXED,1.508777,NaN
1,Healthcare,AMGN,ACF,-0.408052,0.001937
2,Healthcare,AMGN,AIC,-0.411776,0.000271
3,Healthcare,GILD,FIXED,0.324813,NaN
4,Healthcare,GILD,ACF,-0.381129,0.025036
5,Healthcare,GILD,AIC,-0.561746,0.000203
6,Healthcare,VRTX,FIXED,-0.684523,NaN
7,Healthcare,VRTX,ACF,-0.135407,0.107369
8,Healthcare,VRTX,AIC,-0.174369,0.018949
9,Healthcare,REGN,FIXED,0.421292,NaN


,Sector,Stock,Model,Variable,coef
0,Healthcare,AMGN,FIXED,const,-0.815545
1,Healthcare,AMGN,FIXED,inflation_rate,0.579385
2,Healthcare,AMGN,FIXED,ppi_inflation,-2.639006
3,Healthcare,AMGN,FIXED,m2_growth,-1.232555
4,Healthcare,AMGN,FIXED,m1_growth,0.724349
5,Healthcare,AMGN,FIXED,bond_yield_spread,-0.511343
6,Healthcare,AMGN,FIXED,dxy,0.002979
7,Healthcare,AMGN,FIXED,nonfarm_payrolls,0.000037
8,Healthcare,AMGN,FIXED,unemployment_rate,0.232235
9,Healthcare,AMGN,FIXED,vix_index,-0.024238


,Sector,Stock,Model,Variable,Coef,pval
0,Healthcare,AMGN,FIXED,D.L1.Logclose_AMGN,-1.729835,NaN
1,Healthcare,AMGN,FIXED,D.inflation_rate,2.407794,NaN
2,Healthcare,AMGN,FIXED,D.L1.inflation_rate,-6.902358,NaN
3,Healthcare,AMGN,FIXED,D.L2.inflation_rate,5.390904,NaN
4,Healthcare,AMGN,FIXED,D.L3.inflation_rate,-2.062771,NaN
5,Healthcare,AMGN,FIXED,D.L4.inflation_rate,-0.262685,NaN
6,Healthcare,AMGN,FIXED,D.L5.inflation_rate,-1.261027,NaN
7,Healthcare,AMGN,FIXED,D.ppi_inflation,3.036915,NaN
8,Healthcare,AMGN,FIXED,D.L1.ppi_inflation,-3.213948,NaN
9,Healthcare,AMGN,FIXED,D.L2.ppi_inflation,1.387178,NaN


# Forecast Return from ARDL + ECM

In [68]:
import pandas as pd
import numpy as np
import re

# ============================================================
# ✅ Forecast next-month ΔLogclose (ECM Full, Simplified)
#   - ใช้เฉพาะ lagged Δ (D.Lk.) + ECT
#   - รองรับ df_alpha/df_beta/df_gamma ที่มีคอลัมน์ Model
#   - ไม่คำนวณ/ไม่คืนค่า Logclose_next และ Close_next
# ============================================================

def forecast_next_month_full(
    stock_name, sector_name,
    df_macro_levels,      # df ระดับ (levels) สำหรับ X เพื่อคำนวณ ΔX lag และ ECT
    df_gamma, df_alpha, df_beta,
    combined_dfs_selective_diff,  # ใช้ดึง Logclose(series) ของหุ้น (ระดับ log)
    model_name="AIC",             # ✅ NEW: "AIC" / "ACF" / "FIXED"
    use_sector_filter=False       # ✅ optional: ถ้าชื่อ ticker ซ้ำข้าม sector ค่อยเปิด True
):
    # -------------------------
    # 0) Filter parameter tables
    # -------------------------
    if use_sector_filter and ("Sector" in df_gamma.columns) and ("Sector" in df_alpha.columns) and ("Sector" in df_beta.columns):
        gtab = df_gamma[(df_gamma["Stock"] == stock_name) & (df_gamma["Model"] == model_name) & (df_gamma["Sector"] == sector_name)].copy()
        atab = df_alpha[(df_alpha["Stock"] == stock_name) & (df_alpha["Model"] == model_name) & (df_alpha["Sector"] == sector_name)]
        btab = df_beta[(df_beta["Stock"] == stock_name) & (df_beta["Model"] == model_name) & (df_beta["Sector"] == sector_name)]
    else:
        gtab = df_gamma[(df_gamma["Stock"] == stock_name) & (df_gamma["Model"] == model_name)].copy()
        atab = df_alpha[(df_alpha["Stock"] == stock_name) & (df_alpha["Model"] == model_name)]
        btab = df_beta[(df_beta["Stock"] == stock_name) & (df_beta["Model"] == model_name)]

    if gtab.empty or atab.empty or btab.empty:
        return None

    # -------------------------
    # 1) Keep only lagged Δ terms + add ECT
    # -------------------------
    # ✅ เหมือนเดิม: เอาเฉพาะ D.Lk. (ไม่เอา D.<macro> lag0)
    gtab = gtab[gtab["Variable"].astype(str).str.contains(r"D\.L\d+\.", regex=True, na=False)].copy()

    # ✅ Add ECT_lag1 = alpha
    alpha_val = float(atab["Alpha"].iloc[0])
    gtab = pd.concat(
        [gtab, pd.DataFrame([{"Variable": "ECT_lag1", "Coef": alpha_val}])],
        ignore_index=True
    )

    # -------------------------
    # 2) Prepare stock log series
    # -------------------------
    df_stock = combined_dfs_selective_diff[sector_name][[f"Logclose_{stock_name}"]].copy()
    df_stock.columns = ["Logclose"]

    # -------------------------
    # 3) Align macro levels to stock index
    # -------------------------
    df_macro_aligned = df_macro_levels.reindex(df_stock.index).ffill().copy()

    # -------------------------
    # 4) Long-run coefficients for ECT
    # -------------------------
    lr_params = btab.drop_duplicates(subset="Variable").set_index("Variable")["coef"]

    # ใช้ 3 แถวสุดท้ายเพื่อคำนวณ Δ และ lag (พอสำหรับ L1 / L2)
    last_row = df_macro_aligned.tail(3).copy()
    deltas = {}

    # -------------------------
    # 5) Compute Δ lag and ECT
    # -------------------------
    for var in gtab["Variable"].astype(str).tolist():
        if var == "ECT_lag1":
            const = float(lr_params.get("const", 0.0))
            X_lag = {
                k: float(last_row[k].iloc[-2])
                for k in lr_params.index
                if k != "const" and k in last_row.columns
            }
            Y_lag = float(df_stock["Logclose"].iloc[-2])
            ect_val = Y_lag - (const + sum(float(lr_params[k]) * X_lag[k] for k in X_lag))
            deltas[var] = float(ect_val)
            continue

        # ตัวอย่างชื่อ: "D.L1.inflation_rate" หรือ "D.L1.Logclose_XYZ"
        parts = var.split(".")
        if len(parts) < 3:
            continue
        lag = int(parts[1].replace("L", ""))   # L1 -> 1
        base = parts[2]

        # ΔY lag (Logclose)
        if base.startswith("Logclose_"):
            stock_base = base.replace("Logclose_", "")
            if stock_base != stock_name:
                continue

            # ΔY_{t-lag} = logY_{t-lag} - logY_{t-lag-1}
            if len(df_stock) > lag + 1:
                dy = float(df_stock["Logclose"].iloc[-lag]) - float(df_stock["Logclose"].iloc[-(lag + 1)])
            else:
                dy = 0.0
            deltas[var] = dy

        # ΔX lag (macro levels)
        else:
            if base in df_macro_aligned.columns and len(last_row) > lag + 1:
                dx = float(last_row[base].iloc[-lag]) - float(last_row[base].iloc[-(lag + 1)])
            else:
                dx = 0.0
            deltas[var] = dx

    # -------------------------
    # 6) Sum contributions -> Pred Δlogclose
    # -------------------------
    df_pred = pd.DataFrame(list(deltas.items()), columns=["Variable", "Value"])
    merged = pd.merge(gtab[["Variable", "Coef"]], df_pred, on="Variable", how="left")
    merged["Value"] = merged["Value"].fillna(0.0)
    merged["Contribution"] = merged["Coef"].astype(float) * merged["Value"].astype(float)

    pred_dy = float(merged["Contribution"].sum())

    return {
        "Stock": stock_name,
        "Sector": sector_name,
        "Model": model_name,
        "Pred_dLogclose": pred_dy,
        "Details": merged
    }


# ============================================================
# 🚀 Run for ALL stocks (Clean Output)
#   ✅ เพิ่ม model_name ที่อยากใช้
# ============================================================

MODEL_TO_FORECAST = "AIC"   # "AIC" / "ACF" / "FIXED"
USE_SECTOR_FILTER = False  # ถ้า ticker ซ้ำข้าม sector ค่อยเปลี่ยน True

forecast_results = []

for sector_name, df_sec in combined_dfs_selective_diff.items():
    for col in df_sec.columns:
        if not str(col).startswith("Logclose_"):
            continue

        stock_name = col.replace("Logclose_", "")

        try:
            res = forecast_next_month_full(
                stock_name=stock_name,
                sector_name=sector_name,
                df_macro_levels=df,  # levels macro
                df_gamma=df_gamma, df_alpha=df_alpha, df_beta=df_beta,
                combined_dfs_selective_diff=combined_dfs_selective_diff,
                model_name=MODEL_TO_FORECAST,
                use_sector_filter=USE_SECTOR_FILTER
            )
            if res is not None:
                forecast_results.append({
                    "Stock": res["Stock"],
                    "Sector": res["Sector"],
                    "Model": res["Model"],
                    "Pred_dLogclose": res["Pred_dLogclose"],
                })
                print(f"✅ {stock_name:<8} | Sector={sector_name:<20} | Model={MODEL_TO_FORECAST:<5} | ΔŶ={res['Pred_dLogclose']:+.5f}")

        except Exception as e:
            print(f"⚠️ {stock_name} failed: {e}")

df_forecast_summary = pd.DataFrame(forecast_results)
if not df_forecast_summary.empty:
    df_forecast_summary["Expected_Return_%"] = df_forecast_summary["Pred_dLogclose"] * 100.0
    df_forecast_summary["Signal"] = np.where(df_forecast_summary["Pred_dLogclose"] > 0, "BUY", "SELL")

display(df_forecast_summary)

OUT_NAME = f"Forecast_ARDL_ECM_NextMonth_Clean_{MODEL_TO_FORECAST}.xlsx"
df_forecast_summary.to_excel(OUT_NAME, index=False)
print(f"\n✅ Exported → {OUT_NAME}")


✅ AMGN     | Sector=Healthcare           | Model=AIC   | ΔŶ=-0.67152
✅ GILD     | Sector=Healthcare           | Model=AIC   | ΔŶ=+3.06900
✅ VRTX     | Sector=Healthcare           | Model=AIC   | ΔŶ=-7.09608
✅ REGN     | Sector=Healthcare           | Model=AIC   | ΔŶ=-5.28529
✅ BIIB     | Sector=Healthcare           | Model=AIC   | ΔŶ=+4.57720
✅ ILMN     | Sector=Healthcare           | Model=AIC   | ΔŶ=+0.76058
✅ IDXX     | Sector=Healthcare           | Model=AIC   | ΔŶ=-1.31541
✅ ISRG     | Sector=Healthcare           | Model=AIC   | ΔŶ=+0.01916
✅ ALGN     | Sector=Healthcare           | Model=AIC   | ΔŶ=-5.52500


,Stock,Sector,Model,Pred_dLogclose,Expected_Return_%,Signal
0,AMGN,Healthcare,AIC,-0.671515,-67.151535,SELL
1,GILD,Healthcare,AIC,3.068997,306.899657,BUY
2,VRTX,Healthcare,AIC,-7.096076,-709.607629,SELL
3,REGN,Healthcare,AIC,-5.285292,-528.529209,SELL
4,BIIB,Healthcare,AIC,4.577202,457.720173,BUY
5,ILMN,Healthcare,AIC,0.760578,76.057802,BUY
6,IDXX,Healthcare,AIC,-1.315413,-131.541329,SELL
7,ISRG,Healthcare,AIC,0.019159,1.915859,BUY
8,ALGN,Healthcare,AIC,-5.525002,-552.500198,SELL



✅ Exported → Forecast_ARDL_ECM_NextMonth_Clean_AIC.xlsx


# Technical Indicators

In [14]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=1a7953cb45856b92f68d4cb3d18a9f5a7bd7b1444d6d561ccdda181fe54e921c
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [56]:
# ============================================================
# 📊 BLOCK 2 ENHANCED: Advanced Technical Features
#    Features:
#    1. Market Regime Detector (Bull/Bear/Sideways)
#    2. Volatility Regime (High/Normal/Low)
#    3. Pattern Detection (20 Patterns from TradingView)
#    4. Sector Rotation Score
#    5. Proxy Sentiment (Foreign Flow, Market Breadth)
#
#    NO LLM / NO FinBERT - Pure Technical Analysis
# ============================================================

import pandas as pd
import numpy as np
import yfinance as yf
from scipy.signal import argrelextrema
from scipy.stats import linregress
import ta
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

# ============================================================
# 1. CONFIGURATION
# ============================================================

SECTORS = {
    "Healthcare": ["AMGN","GILD","VRTX","REGN","BIIB","ILMN","IDXX","ISRG","ALGN"]
}


ALL_STOCKS = [stock for stocks in SECTORS.values() for stock in stocks]

START_DATE = "2015-01-01"
END_DATE = "2025-12-31"

# ============================================================
# 2. MARKET REGIME DETECTOR
# ============================================================

class MarketRegimeDetector:
    """
    Detect Market Regime: Bull / Bear / Sideways

    Methods:
    - SMA Crossover (50/200)
    - ADX Trend Strength
    - Price vs Moving Average
    - Higher Highs / Lower Lows
    """

    @staticmethod
    def detect_regime_sma(close, short_window=50, long_window=200):
        """
        SMA Crossover Method
        - Bull: SMA50 > SMA200
        - Bear: SMA50 < SMA200
        """
        sma_short = close.rolling(window=short_window).mean()
        sma_long = close.rolling(window=long_window).mean()

        regime = pd.Series(index=close.index, dtype='object')
        regime[sma_short > sma_long] = 'Bull'
        regime[sma_short < sma_long] = 'Bear'
        regime[sma_short == sma_long] = 'Sideways'

        return regime

    @staticmethod
    def detect_regime_adx(high, low, close, window=14, trend_threshold=25):
        """
        ADX Method
        - Trending (Bull/Bear): ADX > 25
        - Sideways: ADX < 25
        """
        adx = ta.trend.adx(high, low, close, window=window)
        plus_di = ta.trend.adx_pos(high, low, close, window=window)
        minus_di = ta.trend.adx_neg(high, low, close, window=window)

        regime = pd.Series(index=close.index, dtype='object')

        # Trending
        trending = adx > trend_threshold
        regime[trending & (plus_di > minus_di)] = 'Bull'
        regime[trending & (plus_di < minus_di)] = 'Bear'

        # Sideways
        regime[adx <= trend_threshold] = 'Sideways'

        return regime, adx

    @staticmethod
    def detect_regime_combined(high, low, close, volume):
        """
        Combined Method (Most Robust)
        - Uses multiple signals for confirmation
        """
        # Method 1: SMA
        regime_sma = MarketRegimeDetector.detect_regime_sma(close)

        # Method 2: ADX
        regime_adx, adx = MarketRegimeDetector.detect_regime_adx(high, low, close)

        # Method 3: Price vs SMA200
        sma200 = close.rolling(200).mean()
        price_above_sma = close > sma200

        # Method 4: Trend Strength (linear regression slope)
        def calc_slope(series, window=20):
            slopes = pd.Series(index=series.index, dtype=float)
            for i in range(window, len(series)):
                y = series.iloc[i-window:i].values
                x = np.arange(window)
                slope, _, _, _, _ = linregress(x, y)
                slopes.iloc[i] = slope
            return slopes

        price_slope = calc_slope(close, 20)
        price_slope_normalized = price_slope / close * 100  # Normalize by price

        # Combine signals
        regime_combined = pd.Series(index=close.index, dtype='object')
        regime_score = pd.Series(index=close.index, dtype=float)

        for i in range(len(close)):
            score = 0

            # SMA signal
            if regime_sma.iloc[i] == 'Bull':
                score += 1
            elif regime_sma.iloc[i] == 'Bear':
                score -= 1

            # ADX signal
            if regime_adx.iloc[i] == 'Bull':
                score += 1
            elif regime_adx.iloc[i] == 'Bear':
                score -= 1

            # Price vs SMA200
            if i >= 200:
                if price_above_sma.iloc[i]:
                    score += 1
                else:
                    score -= 1

            # Slope signal
            if pd.notna(price_slope_normalized.iloc[i]):
                if price_slope_normalized.iloc[i] > 0.1:
                    score += 1
                elif price_slope_normalized.iloc[i] < -0.1:
                    score -= 1

            regime_score.iloc[i] = score

            # Determine regime
            if score >= 2:
                regime_combined.iloc[i] = 'Bull'
            elif score <= -2:
                regime_combined.iloc[i] = 'Bear'
            else:
                regime_combined.iloc[i] = 'Sideways'

        return regime_combined, regime_score, adx


# ============================================================
# 3. VOLATILITY REGIME DETECTOR
# ============================================================

class VolatilityRegimeDetector:
    """
    Detect Volatility Regime: High / Normal / Low

    Methods:
    - Historical Volatility (Rolling Std)
    - ATR Percentile
    - Bollinger Band Width
    """

    @staticmethod
    def calculate_historical_volatility(close, window=20):
        """Calculate annualized historical volatility"""
        returns = np.log(close / close.shift(1))
        vol = returns.rolling(window=window).std() * np.sqrt(252)
        return vol

    @staticmethod
    def calculate_atr_percentile(high, low, close, window=14, lookback=252):
        """Calculate ATR and its percentile ranking"""
        atr = ta.volatility.average_true_range(high, low, close, window=window)

        # Percentile rank over lookback period
        atr_percentile = atr.rolling(window=lookback).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1] * 100
        )

        return atr, atr_percentile

    @staticmethod
    def calculate_bb_width(close, window=20, std_dev=2):
        """Calculate Bollinger Band Width"""
        bb = ta.volatility.BollingerBands(close, window=window, window_dev=std_dev)
        bb_width = (bb.bollinger_hband() - bb.bollinger_lband()) / bb.bollinger_mavg()
        return bb_width

    @staticmethod
    def detect_volatility_regime(high, low, close,
                                  low_threshold=30, high_threshold=70):
        """
        Detect Volatility Regime

        Returns:
        - regime: 'High', 'Normal', 'Low'
        - vol_score: 0-100 percentile
        """
        # Calculate metrics
        hist_vol = VolatilityRegimeDetector.calculate_historical_volatility(close)
        atr, atr_pct = VolatilityRegimeDetector.calculate_atr_percentile(high, low, close)
        bb_width = VolatilityRegimeDetector.calculate_bb_width(close)

        # Normalize BB Width to percentile
        bb_pct = bb_width.rolling(window=252).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1] * 100
        )

        # Combine scores (average of percentiles)
        vol_score = (atr_pct.fillna(50) + bb_pct.fillna(50)) / 2

        # Determine regime
        regime = pd.Series(index=close.index, dtype='object')
        regime[vol_score >= high_threshold] = 'High'
        regime[vol_score <= low_threshold] = 'Low'
        regime[(vol_score > low_threshold) & (vol_score < high_threshold)] = 'Normal'

        return regime, vol_score, hist_vol, atr


# ============================================================
# 4. PATTERN DETECTOR (20 Patterns)
# ============================================================

class PatternDetector:
    """
    Detect 20 Trading Patterns from TradingView Cheat Sheet

    Categories:
    - CONTINUATION: Pennant, Megaphone, Bearish Flag, Bullish Flag, Channel
    - NEUTRAL: Symmetrical Triangle, Descending Triangle, Ascending Triangle
    - REVERSAL: Diamond, Double Top, Double Bottom, Head & Shoulders,
                Inverse H&S, Cup and Handle
    - SPECIAL: Descending Wedge, Ascending Wedge, Gartley, Cypher
    """

    def __init__(self, lookback=60, order=5):
        self.lookback = lookback
        self.order = order  # For finding local extrema

    def find_pivots(self, high, low, close):
        """Find pivot highs and lows"""
        pivot_highs = argrelextrema(high.values, np.greater, order=self.order)[0]
        pivot_lows = argrelextrema(low.values, np.less, order=self.order)[0]
        return pivot_highs, pivot_lows

    def detect_double_top(self, high, close, pivot_highs, tolerance=0.02):
        """
        Double Top Pattern (Reversal - Bearish)
        - Two peaks at similar levels
        - Valley between peaks
        """
        if len(pivot_highs) < 2:
            return 0.0, None

        # Get last two peaks
        peak1_idx = pivot_highs[-2]
        peak2_idx = pivot_highs[-1]
        peak1_val = high.iloc[peak1_idx]
        peak2_val = high.iloc[peak2_idx]

        # Check if peaks are at similar levels
        diff = abs(peak1_val - peak2_val) / peak1_val

        if diff <= tolerance:
            confidence = 1 - diff / tolerance
            return confidence, {'type': 'Double_Top', 'signal': 'Bearish'}

        return 0.0, None

    def detect_double_bottom(self, low, close, pivot_lows, tolerance=0.02):
        """
        Double Bottom Pattern (Reversal - Bullish)
        - Two troughs at similar levels
        - Peak between troughs
        """
        if len(pivot_lows) < 2:
            return 0.0, None

        # Get last two troughs
        trough1_idx = pivot_lows[-2]
        trough2_idx = pivot_lows[-1]
        trough1_val = low.iloc[trough1_idx]
        trough2_val = low.iloc[trough2_idx]

        # Check if troughs are at similar levels
        diff = abs(trough1_val - trough2_val) / trough1_val

        if diff <= tolerance:
            confidence = 1 - diff / tolerance
            return confidence, {'type': 'Double_Bottom', 'signal': 'Bullish'}

        return 0.0, None

    def detect_head_and_shoulders(self, high, pivot_highs, tolerance=0.03):
        """
        Head and Shoulders Pattern (Reversal - Bearish)
        - Three peaks: middle (head) higher than two sides (shoulders)
        - Shoulders at similar levels
        """
        if len(pivot_highs) < 3:
            return 0.0, None

        # Get last three peaks
        left_idx = pivot_highs[-3]
        head_idx = pivot_highs[-2]
        right_idx = pivot_highs[-1]

        left_val = high.iloc[left_idx]
        head_val = high.iloc[head_idx]
        right_val = high.iloc[right_idx]

        # Check pattern conditions
        # Head should be higher than shoulders
        if head_val > left_val and head_val > right_val:
            # Shoulders should be at similar levels
            shoulder_diff = abs(left_val - right_val) / left_val

            if shoulder_diff <= tolerance:
                head_prominence = (head_val - max(left_val, right_val)) / head_val
                confidence = min(1.0, head_prominence * 10) * (1 - shoulder_diff / tolerance)
                return confidence, {'type': 'Head_Shoulders', 'signal': 'Bearish'}

        return 0.0, None

    def detect_inverse_head_and_shoulders(self, low, pivot_lows, tolerance=0.03):
        """
        Inverse Head and Shoulders Pattern (Reversal - Bullish)
        """
        if len(pivot_lows) < 3:
            return 0.0, None

        left_idx = pivot_lows[-3]
        head_idx = pivot_lows[-2]
        right_idx = pivot_lows[-1]

        left_val = low.iloc[left_idx]
        head_val = low.iloc[head_idx]
        right_val = low.iloc[right_idx]

        if head_val < left_val and head_val < right_val:
            shoulder_diff = abs(left_val - right_val) / left_val

            if shoulder_diff <= tolerance:
                head_prominence = (min(left_val, right_val) - head_val) / head_val
                confidence = min(1.0, head_prominence * 10) * (1 - shoulder_diff / tolerance)
                return confidence, {'type': 'Inverse_Head_Shoulders', 'signal': 'Bullish'}

        return 0.0, None

    def detect_triangle(self, high, low, pivot_highs, pivot_lows, window=20):
        """
        Triangle Patterns (Neutral - wait for breakout)
        - Symmetrical: converging trendlines
        - Ascending: flat top, rising bottom
        - Descending: falling top, flat bottom
        """
        if len(pivot_highs) < 2 or len(pivot_lows) < 2:
            return 0.0, None, None

        # Get recent pivots
        recent_highs = pivot_highs[-3:] if len(pivot_highs) >= 3 else pivot_highs
        recent_lows = pivot_lows[-3:] if len(pivot_lows) >= 3 else pivot_lows

        # Calculate slopes of trendlines
        if len(recent_highs) >= 2:
            high_vals = [high.iloc[i] for i in recent_highs]
            high_slope = (high_vals[-1] - high_vals[0]) / (recent_highs[-1] - recent_highs[0] + 1)
        else:
            high_slope = 0

        if len(recent_lows) >= 2:
            low_vals = [low.iloc[i] for i in recent_lows]
            low_slope = (low_vals[-1] - low_vals[0]) / (recent_lows[-1] - recent_lows[0] + 1)
        else:
            low_slope = 0

        # Normalize slopes
        avg_price = (high.iloc[-1] + low.iloc[-1]) / 2
        high_slope_norm = high_slope / avg_price * 100
        low_slope_norm = low_slope / avg_price * 100

        # Detect triangle type
        if high_slope_norm < -0.01 and low_slope_norm > 0.01:
            # Symmetrical Triangle
            convergence = abs(high_slope_norm) + abs(low_slope_norm)
            confidence = min(1.0, convergence * 10)
            return confidence, 'Symmetrical_Triangle', 'Neutral'

        elif abs(high_slope_norm) < 0.01 and low_slope_norm > 0.01:
            # Ascending Triangle (Bullish)
            confidence = min(1.0, abs(low_slope_norm) * 20)
            return confidence, 'Ascending_Triangle', 'Bullish'

        elif high_slope_norm < -0.01 and abs(low_slope_norm) < 0.01:
            # Descending Triangle (Bearish)
            confidence = min(1.0, abs(high_slope_norm) * 20)
            return confidence, 'Descending_Triangle', 'Bearish'

        return 0.0, None, None

    def detect_flag(self, high, low, close, pivot_highs, pivot_lows, window=30):
        """
        Flag Patterns (Continuation)
        - Bullish Flag: Strong up move, then small downward channel
        - Bearish Flag: Strong down move, then small upward channel
        """
        if len(close) < window:
            return 0.0, None

        # Check for prior strong move (pole)
        pole_period = close.iloc[-window:-window//2]
        flag_period = close.iloc[-window//2:]

        pole_return = (pole_period.iloc[-1] - pole_period.iloc[0]) / pole_period.iloc[0]
        flag_return = (flag_period.iloc[-1] - flag_period.iloc[0]) / flag_period.iloc[0]

        # Bullish Flag: Strong up pole, slight down flag
        if pole_return > 0.05 and -0.03 < flag_return < 0:
            confidence = min(1.0, pole_return * 5)
            return confidence, {'type': 'Bullish_Flag', 'signal': 'Bullish'}

        # Bearish Flag: Strong down pole, slight up flag
        if pole_return < -0.05 and 0 < flag_return < 0.03:
            confidence = min(1.0, abs(pole_return) * 5)
            return confidence, {'type': 'Bearish_Flag', 'signal': 'Bearish'}

        return 0.0, None

    def detect_wedge(self, high, low, pivot_highs, pivot_lows):
        """
        Wedge Patterns (Special)
        - Rising Wedge: Both trendlines rising, converging (Bearish)
        - Falling Wedge: Both trendlines falling, converging (Bullish)
        """
        if len(pivot_highs) < 2 or len(pivot_lows) < 2:
            return 0.0, None

        # Calculate slopes
        high_vals = [high.iloc[i] for i in pivot_highs[-3:]]
        low_vals = [low.iloc[i] for i in pivot_lows[-3:]]

        if len(high_vals) >= 2 and len(low_vals) >= 2:
            high_slope = high_vals[-1] - high_vals[0]
            low_slope = low_vals[-1] - low_vals[0]

            # Rising Wedge (Bearish)
            if high_slope > 0 and low_slope > 0 and high_slope < low_slope:
                confidence = min(1.0, (low_slope - high_slope) / abs(high_slope + 0.001) * 2)
                return confidence, {'type': 'Rising_Wedge', 'signal': 'Bearish'}

            # Falling Wedge (Bullish)
            if high_slope < 0 and low_slope < 0 and high_slope > low_slope:
                confidence = min(1.0, (high_slope - low_slope) / abs(low_slope + 0.001) * 2)
                return confidence, {'type': 'Falling_Wedge', 'signal': 'Bullish'}

        return 0.0, None

    def detect_channel(self, high, low, pivot_highs, pivot_lows, tolerance=0.02):
        """
        Channel Pattern (Continuation)
        - Parallel trendlines
        """
        if len(pivot_highs) < 2 or len(pivot_lows) < 2:
            return 0.0, None

        # Calculate slopes
        high_indices = pivot_highs[-3:]
        low_indices = pivot_lows[-3:]

        high_vals = [high.iloc[i] for i in high_indices]
        low_vals = [low.iloc[i] for i in low_indices]

        if len(high_vals) >= 2 and len(low_vals) >= 2:
            # Linear regression for trendlines
            high_slope, high_intercept, _, _, _ = linregress(high_indices, high_vals)
            low_slope, low_intercept, _, _, _ = linregress(low_indices, low_vals)

            # Check if parallel (similar slopes)
            slope_diff = abs(high_slope - low_slope) / (abs(high_slope) + 0.001)

            if slope_diff < tolerance:
                if high_slope > 0:
                    return 1 - slope_diff, {'type': 'Ascending_Channel', 'signal': 'Bullish'}
                elif high_slope < 0:
                    return 1 - slope_diff, {'type': 'Descending_Channel', 'signal': 'Bearish'}
                else:
                    return 1 - slope_diff, {'type': 'Horizontal_Channel', 'signal': 'Neutral'}

        return 0.0, None

    def detect_cup_and_handle(self, close, window=60):
        """
        Cup and Handle Pattern (Reversal - Bullish)
        - U-shaped cup followed by small pullback (handle)
        """
        if len(close) < window:
            return 0.0, None

        cup = close.iloc[-window:-window//4]
        handle = close.iloc[-window//4:]

        # Check for U-shape (cup)
        cup_min_idx = cup.argmin()
        cup_start = cup.iloc[0]
        cup_end = cup.iloc[-1]
        cup_min = cup.min()

        # Cup should have similar start and end, with lower middle
        start_end_diff = abs(cup_start - cup_end) / cup_start
        cup_depth = (cup_start - cup_min) / cup_start

        if start_end_diff < 0.05 and cup_depth > 0.05:
            # Check handle (small pullback)
            handle_return = (handle.iloc[-1] - handle.iloc[0]) / handle.iloc[0]

            if -0.05 < handle_return < 0.02:
                confidence = min(1.0, cup_depth * 5 * (1 - start_end_diff))
                return confidence, {'type': 'Cup_Handle', 'signal': 'Bullish'}

        return 0.0, None

    def detect_all_patterns(self, high, low, close):
        """
        Detect all patterns and return the most prominent one
        """
        pivot_highs, pivot_lows = self.find_pivots(high, low, close)

        patterns = {}

        # Double patterns
        conf, pattern = self.detect_double_top(high, close, pivot_highs)
        if conf > 0:
            patterns['Double_Top'] = (conf, pattern)

        conf, pattern = self.detect_double_bottom(low, close, pivot_lows)
        if conf > 0:
            patterns['Double_Bottom'] = (conf, pattern)

        # Head and Shoulders
        conf, pattern = self.detect_head_and_shoulders(high, pivot_highs)
        if conf > 0:
            patterns['Head_Shoulders'] = (conf, pattern)

        conf, pattern = self.detect_inverse_head_and_shoulders(low, pivot_lows)
        if conf > 0:
            patterns['Inverse_HS'] = (conf, pattern)

        # Triangles
        conf, pattern_type, signal = self.detect_triangle(high, low, pivot_highs, pivot_lows)
        if conf > 0:
            patterns[pattern_type] = (conf, {'type': pattern_type, 'signal': signal})

        # Flags
        conf, pattern = self.detect_flag(high, low, close, pivot_highs, pivot_lows)
        if conf > 0:
            patterns[pattern['type']] = (conf, pattern)

        # Wedges
        conf, pattern = self.detect_wedge(high, low, pivot_highs, pivot_lows)
        if conf > 0:
            patterns[pattern['type']] = (conf, pattern)

        # Channels
        conf, pattern = self.detect_channel(high, low, pivot_highs, pivot_lows)
        if conf > 0:
            patterns[pattern['type']] = (conf, pattern)

        # Cup and Handle
        conf, pattern = self.detect_cup_and_handle(close)
        if conf > 0:
            patterns['Cup_Handle'] = (conf, pattern)

        # Find best pattern
        if patterns:
            best_pattern = max(patterns.items(), key=lambda x: x[1][0])
            return best_pattern[0], best_pattern[1][0], best_pattern[1][1]

        return None, 0.0, None


# ============================================================
# 5. SECTOR ROTATION ANALYZER
# ============================================================

class SectorRotationAnalyzer:
    """
    Analyze Sector Rotation

    Methods:
    - Relative Strength vs SET Index
    - Sector Momentum
    - Sector Ranking
    """

    @staticmethod
    def calculate_relative_strength(sector_prices, benchmark_prices, window=20):
        """
        Calculate Relative Strength
        RS = Sector Return / Benchmark Return
        """
        sector_return = sector_prices.pct_change(window)
        benchmark_return = benchmark_prices.pct_change(window)

        rs = sector_return / (benchmark_return + 0.0001)  # Avoid division by zero
        return rs

    @staticmethod
    def calculate_sector_momentum(sector_prices, windows=[5, 10, 20, 60]):
        """
        Calculate Sector Momentum across multiple timeframes
        """
        momentum = pd.DataFrame(index=sector_prices.index)

        for w in windows:
            momentum[f'Mom_{w}'] = sector_prices.pct_change(w)

        # Composite momentum score
        momentum['Momentum_Score'] = momentum.mean(axis=1)

        return momentum['Momentum_Score']

    @staticmethod
    def rank_sectors(sector_data, metric='Momentum_Score'):
        """
        Rank sectors by a given metric
        1 = Best, N = Worst
        """
        return sector_data.rank(ascending=False)


# ============================================================
# 6. PROXY SENTIMENT INDICATORS
# ============================================================

class ProxySentimentCalculator:
    """
    Calculate Proxy Sentiment (without LLM/News)

    Indicators:
    - Market Breadth (Advance/Decline)
    - Volume Sentiment
    - Price Momentum Breadth
    - Put/Call Proxy (Volatility-based)
    """

    @staticmethod
    def calculate_volume_sentiment(close, volume, window=20):
        """
        Volume Sentiment
        - Up volume vs Down volume
        """
        price_change = close.diff()

        up_volume = volume.where(price_change > 0, 0)
        down_volume = volume.where(price_change < 0, 0)

        up_vol_ma = up_volume.rolling(window).sum()
        down_vol_ma = down_volume.rolling(window).sum()

        # Volume Ratio (-1 to +1)
        vol_sentiment = (up_vol_ma - down_vol_ma) / (up_vol_ma + down_vol_ma + 1)

        return vol_sentiment

    @staticmethod
    def calculate_price_momentum_sentiment(close, window=20):
        """
        Price Momentum Sentiment
        - Based on rate of change percentile
        """
        roc = close.pct_change(window)

        # Percentile rank
        roc_percentile = roc.rolling(252).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1]
        )

        # Convert to -1 to +1 scale
        sentiment = (roc_percentile - 0.5) * 2

        return sentiment

    @staticmethod
    def calculate_fear_greed_proxy(high, low, close, volume, window=20):
        """
        Fear & Greed Proxy Index

        Components:
        - Price Momentum
        - Volatility (inverse)
        - Volume Trend
        - Price vs MA
        """
        # 1. Price Momentum (0 to 100)
        roc = close.pct_change(window)
        mom_score = roc.rolling(252).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1]
        ) * 100

        # 2. Volatility (inverse - low vol = greed, high vol = fear)
        atr = ta.volatility.average_true_range(high, low, close, window=14)
        atr_pct = atr.rolling(252).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1]
        )
        vol_score = (1 - atr_pct) * 100

        # 3. Volume Trend
        vol_ma = volume.rolling(window).mean()
        vol_ma_long = volume.rolling(window * 3).mean()
        vol_score_2 = (vol_ma / vol_ma_long).clip(0.5, 1.5)
        vol_score_2 = ((vol_score_2 - 0.5) / 1.0) * 100

        # 4. Price vs 200 MA
        ma200 = close.rolling(200).mean()
        price_vs_ma = ((close / ma200) - 1).clip(-0.2, 0.2)
        ma_score = ((price_vs_ma + 0.2) / 0.4) * 100

        # Combine (equal weight)
        fear_greed = (mom_score.fillna(50) + vol_score.fillna(50) +
                      vol_score_2.fillna(50) + ma_score.fillna(50)) / 4

        return fear_greed

    @staticmethod
    def interpret_fear_greed(score):
        """
        Interpret Fear & Greed Score
        """
        if score >= 75:
            return 'Extreme_Greed'
        elif score >= 55:
            return 'Greed'
        elif score >= 45:
            return 'Neutral'
        elif score >= 25:
            return 'Fear'
        else:
            return 'Extreme_Fear'


# ============================================================
# 7. MAIN PROCESSING
# ============================================================

def process_stock(stock, sector, start_date, end_date):
    """Process single stock with all enhanced features"""

    print(f"   📥 Downloading {stock} ...")

    try:
        df = yf.download(stock, start=start_date, end=end_date, progress=False)

        if df.empty or len(df) < 252:
            print(f"   ⚠️ Insufficient data for {stock}")
            return None

        # Flatten MultiIndex if needed
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df = df.reset_index()
        df['Stock'] = stock
        df['Sector'] = sector

        # Extract OHLCV
        high = df['High']
        low = df['Low']
        close = df['Close']
        volume = df['Volume']

        # ----- 1. Market Regime -----
        regime, regime_score, adx = MarketRegimeDetector.detect_regime_combined(
            high, low, close, volume
        )
        df['Market_Regime'] = regime
        df['Regime_Score'] = regime_score
        df['ADX'] = adx

        # ----- 2. Volatility Regime -----
        vol_regime, vol_score, hist_vol, atr = VolatilityRegimeDetector.detect_volatility_regime(
            high, low, close
        )
        df['Volatility_Regime'] = vol_regime
        df['Volatility_Score'] = vol_score
        df['Historical_Volatility'] = hist_vol
        df['ATR'] = atr

        # ----- 3. Pattern Detection -----
        detector = PatternDetector(lookback=60, order=5)

        pattern_types = []
        pattern_confidences = []
        pattern_signals = []

        for i in range(len(df)):
            if i < 60:
                pattern_types.append(None)
                pattern_confidences.append(0.0)
                pattern_signals.append(None)
                continue

            h = high.iloc[:i+1]
            l = low.iloc[:i+1]
            c = close.iloc[:i+1]

            pattern_type, confidence, pattern_info = detector.detect_all_patterns(h, l, c)

            pattern_types.append(pattern_type)
            pattern_confidences.append(confidence)
            if pattern_info:
                pattern_signals.append(pattern_info.get('signal'))
            else:
                pattern_signals.append(None)

        df['Pattern_Type'] = pattern_types
        df['Pattern_Confidence'] = pattern_confidences
        df['Pattern_Signal'] = pattern_signals

        # ----- 4. Proxy Sentiment -----
        df['Volume_Sentiment'] = ProxySentimentCalculator.calculate_volume_sentiment(
            close, volume
        )
        df['Momentum_Sentiment'] = ProxySentimentCalculator.calculate_price_momentum_sentiment(
            close
        )
        df['Fear_Greed_Index'] = ProxySentimentCalculator.calculate_fear_greed_proxy(
            high, low, close, volume
        )
        df['Fear_Greed_Label'] = df['Fear_Greed_Index'].apply(
            ProxySentimentCalculator.interpret_fear_greed
        )

        # ----- 5. Basic Technical Indicators -----
        df['RSI'] = ta.momentum.rsi(close, window=14)
        df['MACD'] = ta.trend.macd(close)
        df['MACD_Signal'] = ta.trend.macd_signal(close)
        df['EMA_12'] = ta.trend.ema_indicator(close, window=12)
        df['EMA_26'] = ta.trend.ema_indicator(close, window=26)
        df['SMA_50'] = close.rolling(50).mean()
        df['SMA_200'] = close.rolling(200).mean()

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
        df['BB_Upper'] = bb.bollinger_hband()
        df['BB_Lower'] = bb.bollinger_lband()
        df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / bb.bollinger_mavg()

        print(f"   ✅ {stock} processed: {len(df)} rows")

        return df

    except Exception as e:
        print(f"   ❌ Error processing {stock}: {e}")
        return None


def calculate_sector_features(all_data, sectors):
    """Calculate sector-level features"""

    print("\n📊 Calculating Sector Features...")

    # Calculate sector average prices
    sector_prices = {}

    for sector, stocks in sectors.items():
        sector_df = all_data[all_data['Sector'] == sector]

        if sector_df.empty:
            continue

        # Average close price for sector
        sector_avg = sector_df.groupby('Date')['Close'].mean()
        sector_prices[sector] = sector_avg

        # Calculate sector momentum
        sector_momentum = SectorRotationAnalyzer.calculate_sector_momentum(sector_avg)

        # Add to individual stock data
        for idx in all_data[all_data['Sector'] == sector].index:
            date = all_data.loc[idx, 'Date']
            if date in sector_momentum.index:
                all_data.loc[idx, 'Sector_Momentum'] = sector_momentum.loc[date]

    # Calculate sector rankings per date
    dates = all_data['Date'].unique()

    for date in dates:
        daily_data = all_data[all_data['Date'] == date]

        if daily_data.empty:
            continue

        # Get sector momentums for this date
        sector_moms = daily_data.groupby('Sector')['Sector_Momentum'].first()

        if len(sector_moms) > 1:
            ranks = sector_moms.rank(ascending=False)

            for sector, rank in ranks.items():
                mask = (all_data['Date'] == date) & (all_data['Sector'] == sector)
                all_data.loc[mask, 'Sector_Rank'] = rank

    return all_data


# ============================================================
# 8. MAIN EXECUTION
# ============================================================

if __name__ == "__main__":

    print("=" * 70)
    print("📊 BLOCK 2 ENHANCED: Advanced Technical Features")
    print("=" * 70)
    print("""
    Features:
    1. Market Regime Detector (Bull/Bear/Sideways)
    2. Volatility Regime (High/Normal/Low)
    3. Pattern Detection (20 Patterns)
    4. Sector Rotation Score
    5. Proxy Sentiment (Fear/Greed Index)

    NO LLM / NO FinBERT - Pure Technical Analysis
    """)
    print("=" * 70)

    # Process all stocks
    all_results = []

    for sector, stocks in SECTORS.items():
        print(f"\n🏢 Processing Sector: {sector}")
        print("-" * 50)

        for stock in stocks:
            result = process_stock(stock, sector, START_DATE, END_DATE)
            if result is not None:
                all_results.append(result)

    # Combine all data
    if all_results:
        df_all = pd.concat(all_results, ignore_index=True)

        # Calculate sector features
        df_all = calculate_sector_features(df_all, SECTORS)

        # Sort
        df_all = df_all.sort_values(['Sector', 'Stock', 'Date']).reset_index(drop=True)

        # Export
        output_file = "Block2_Enhanced_Features.xlsx"
        df_all.to_excel(output_file, index=False)

        print("\n" + "=" * 70)
        print("📊 SUMMARY")
        print("=" * 70)
        print(f"✅ Total Stocks: {df_all['Stock'].nunique()}")
        print(f"✅ Total Sectors: {df_all['Sector'].nunique()}")
        print(f"✅ Total Rows: {len(df_all):,}")
        print(f"✅ Date Range: {df_all['Date'].min()} to {df_all['Date'].max()}")
        print(f"\n💾 Saved to: {output_file}")

        print("\n📈 Sample Output (Last 5 rows of AMGN):")
        sample = df_all[df_all['Stock'] == 'AMGN'].tail(5)[
            ['Date', 'Close', 'Market_Regime', 'Regime_Score', 'Volatility_Regime',
            'Pattern_Type', 'Pattern_Confidence', 'Fear_Greed_Index', 'Fear_Greed_Label',
            'Sector_Momentum']   # ❌ ลบ Sector_Rank ออก
        ]

        print(sample.to_string(index=False))

        # Show columns
        print("\n📋 All Columns:")
        print(df_all.columns.tolist())

    print("\n" + "=" * 70)
    print("✅ BLOCK 2 ENHANCED COMPLETED!")
    print("=" * 70)

📊 BLOCK 2 ENHANCED: Advanced Technical Features

    Features:
    1. Market Regime Detector (Bull/Bear/Sideways)
    2. Volatility Regime (High/Normal/Low)
    3. Pattern Detection (20 Patterns)
    4. Sector Rotation Score
    5. Proxy Sentiment (Fear/Greed Index)

    NO LLM / NO FinBERT - Pure Technical Analysis
    

🏢 Processing Sector: Healthcare
--------------------------------------------------
   📥 Downloading AMGN ...
   ✅ AMGN processed: 2765 rows
   📥 Downloading GILD ...
   ✅ GILD processed: 2765 rows
   📥 Downloading VRTX ...
   ✅ VRTX processed: 2765 rows
   📥 Downloading REGN ...
   ✅ REGN processed: 2765 rows
   📥 Downloading BIIB ...
   ✅ BIIB processed: 2765 rows
   📥 Downloading ILMN ...
   ✅ ILMN processed: 2765 rows
   📥 Downloading IDXX ...
   ✅ IDXX processed: 2765 rows
   📥 Downloading ISRG ...
   ✅ ISRG processed: 2765 rows
   📥 Downloading ALGN ...
   ✅ ALGN processed: 2765 rows

📊 Calculating Sector Features...

📊 SUMMARY
✅ Total Stocks: 9
✅ Total Sectors: 

📊 สรุป DL Structure ของ Block 5.5
ComponentรายละเอียดInputClose Price → Kalman Filter → MinMaxScaler → Shape (60, 1)Model2 โมเดล: CNN-LSTM และ CNN-GRULayers7 layers: Conv1D → BN → MaxPool → LSTM/GRU → BN → Dropout → DenseOutputProbability [0,1] ว่าราคาจะขึ้นEnsemble(LSTM + GRU) / 2OptimizerAdam (lr=0.001)LossBinary Crossentropy

🔬 Activation Functions
LayerActivationเหตุผลConv1DReLUเร็ว, ไม่มี vanishing gradient, ดีสำหรับ feature extractionLSTM/GRUtanh + sigmoidBuilt-in สำหรับ gates และ cell stateDense (Output)SigmoidBinary classification → output เป็น probability [0,1]

🧠 Bidirectional + Attention คืออะไร และเพิ่มเข้าไปยังไง

1. Bidirectional LSTM/GRU คืออะไร?
ปัจจุบัน (Unidirectional):
Past ──────────────────────────────────> Future
 t-60   t-59   t-58   ...   t-2   t-1   t
  →      →      →      →     →     →    [Output]
อ่านข้อมูลจาก ซ้ายไปขวา อย่างเดียว (อดีต → ปัจจุบัน)
Bidirectional:
Forward:   t-60 → t-59 → t-58 → ... → t-1 → t
                                            ↘
                                             [Concat] → Output
                                            ↗
Backward:  t-60 ← t-59 ← t-58 ← ... ← t-1 ← t
อ่านข้อมูล ทั้งสองทิศทาง แล้วรวมกัน
ข้อดี:

เข้าใจ Context ได้ดีขึ้น (รู้ทั้งอดีตและอนาคตของแต่ละจุด)
จับ Pattern ที่ซับซ้อนได้ดีกว่า


2. Attention Mechanism คืออะไร?
ปัญหาของ LSTM ปกติ:

Output สุดท้ายต้อง "จำ" ข้อมูลทั้ง 60 วันไว้ใน hidden state เดียว
ข้อมูลเก่าๆ อาจถูกลืม (vanishing information)

Attention แก้ปัญหานี้:
Input: [t-60, t-59, t-58, ..., t-2, t-1, t]
         ↓     ↓     ↓          ↓    ↓   ↓
       [h1,   h2,   h3,   ..., h58, h59, h60]  ← Hidden states ทุก timestep
         ↓     ↓     ↓          ↓    ↓   ↓
       [0.01, 0.02, 0.05, ..., 0.15, 0.30, 0.40]  ← Attention weights
         ↓     ↓     ↓          ↓    ↓   ↓
       ──────────── Weighted Sum ────────────
                        ↓
                   Context Vector
                        ↓
                     Output
ข้อดี:

โมเดล "เลือก" ว่าจะให้ความสำคัญกับวันไหนมากกว่า
วันที่สำคัญ (เช่น วันที่มี pattern breakout) จะได้ weight สูง
Interpretable: ดูได้ว่าโมเดลสนใจวันไหน


3. โครงสร้างใหม่ที่เสนอ
┌─────────────────────────────────────────────────────────────────┐
│                  🏗️ UPGRADED MODEL: BiLSTM + Attention           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Input (60, 1)                                                  │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Conv1D (64 filters, kernel=3, ReLU)                      │   │
│  │ BatchNormalization                                       │   │
│  │ MaxPooling1D (pool=2)                                    │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ 🆕 Bidirectional LSTM (50 units, return_sequences=True)  │   │
│  │    Output: (timesteps, 100)  ← 50*2 เพราะ bi-directional │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ 🆕 Attention Layer                                       │   │
│  │    - Compute attention weights for each timestep         │   │
│  │    - Weighted sum → Context vector (100,)                │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ BatchNormalization                                       │   │
│  │ Dropout (0.3)                                            │   │
│  │ Dense (1, sigmoid)                                       │   │
│  └─────────────────────────────────────────────────────────┘   │
│       ↓                                                         │
│  Output: P(price UP) ∈ [0, 1]                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

4. เปรียบเทียบ เดิม vs ใหม่
Componentเดิม (Block 5.5)ใหม่ (Upgraded)LSTM/GRUUnidirectionalBidirectionalreturn_sequencesFalseTrue (ต้องการทุก timestep)Attention❌ ไม่มี✅ มีOutput dimension50100 (50×2)Interpretabilityต่ำสูง (ดู attention weights ได้)

5. ข้อควรระวัง
ข้อดีข้อเสียจับ pattern ได้ดีขึ้นParameters เพิ่มขึ้น ~2xInterpretable (attention weights)Train นานขึ้นลด information lossอาจ overfit ถ้าข้อมูลน้อย

In [16]:
# ============================================================
# 🧠 BLOCK 5.5: Deep Learning Signal (UPGRADED + ROLLING WINDOW)
#    Config: Kalman -> CNN -> BN -> Bidirectional LSTM/GRU -> Attention -> Dropout
#    Features:
#    - Bidirectional LSTM/GRU (capture both past and future context)
#    - Attention Mechanism (focus on important timesteps)
#    - Rolling Window / Walk-Forward Validation
#    - Improved Ensemble (3 models)
# ============================================================

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, LSTM, GRU, Conv1D, MaxPooling1D, Input, Dropout,
    BatchNormalization, Bidirectional, Layer
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import os

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# ============================================================
# 1. TARGET STOCKS
# ============================================================

# [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

target_stocks = ["AMGN","GILD","VRTX","REGN","BIIB","ILMN","IDXX","ISRG","ALGN"]


# ============================================================
# 2. ROLLING WINDOW CONFIGURATION
# ============================================================
# Walk-Forward Validation Windows
# Format: (train_start, train_end, test_year)
ROLLING_WINDOWS = [
    ("2015-01-01", "2018-12-31", 2019),
    ("2016-01-01", "2019-12-31", 2020),
    ("2017-01-01", "2020-12-31", 2021),
    ("2018-01-01", "2021-12-31", 2022),
    ("2019-01-01", "2022-12-31", 2023),
    ("2020-01-01", "2023-12-31", 2024),
    ("2021-01-01", "2024-12-31", 2025),
]

# ============================================================
# 3. KALMAN FILTER (Noise Reduction)
# ============================================================
class KalmanFilter1D:
    """1D Kalman Filter for smoothing price data"""
    def __init__(self, process_noise=1e-5, measurement_noise=1e-3, estimated_error=1.0):
        self.Q = process_noise
        self.R = measurement_noise
        self.P = estimated_error
        self.X = 0

    def update(self, measurement):
        self.P = self.P + self.Q
        K = self.P / (self.P + self.R)
        self.X = self.X + K * (measurement - self.X)
        self.P = (1 - K) * self.P
        return self.X

    def smooth(self, data):
        self.X = data[0]
        result = []
        for measurement in data:
            result.append(self.update(measurement))
        return np.array(result)


# ============================================================
# 4. ATTENTION LAYER (Custom Implementation)
# ============================================================
class AttentionLayer(Layer):
    """
    Attention Mechanism for Time Series
    """
    def __init__(self, return_attention=False, **kwargs):
        self.return_attention = return_attention
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name='attention_weight',
            shape=(input_shape[-1], input_shape[-1]),
            initializer='glorot_uniform',
            trainable=True
        )
        self.b = self.add_weight(
            name='attention_bias',
            shape=(input_shape[-1],),
            initializer='zeros',
            trainable=True
        )
        self.u = self.add_weight(
            name='attention_context',
            shape=(input_shape[-1], 1),
            initializer='glorot_uniform',
            trainable=True
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        uit = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        ait = tf.tensordot(uit, self.u, axes=1)
        ait = tf.squeeze(ait, axis=-1)
        ait = tf.nn.softmax(ait, axis=1)
        ait_expanded = tf.expand_dims(ait, axis=-1)
        weighted_input = x * ait_expanded
        output = tf.reduce_sum(weighted_input, axis=1)

        if self.return_attention:
            return output, ait
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def get_config(self):
        config = super(AttentionLayer, self).get_config()
        config.update({'return_attention': self.return_attention})
        return config


# ============================================================
# 5. MODEL ARCHITECTURES
# ============================================================

def build_cnn_bilstm_attention(input_shape):
    """Model 1: CNN + Bidirectional LSTM + Attention"""
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = BatchNormalization()(x)

    x = AttentionLayer(return_attention=False)(x)

    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


def build_cnn_bigru_attention(input_shape):
    """Model 2: CNN + Bidirectional GRU + Attention"""
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(GRU(50, return_sequences=True))(x)
    x = BatchNormalization()(x)

    x = AttentionLayer(return_attention=False)(x)

    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


def build_stacked_bilstm_attention(input_shape):
    """Model 3: Stacked Bidirectional LSTM + Attention"""
    inputs = Input(shape=input_shape)

    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Bidirectional(LSTM(32, return_sequences=True))(x)
    x = BatchNormalization()(x)

    x = AttentionLayer(return_attention=False)(x)

    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


# ============================================================
# 6. DATA PREPARATION
# ============================================================

def create_dataset(data, look_back=60):
    """Create sequences for time series prediction"""
    X, y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), 0])
        y.append(1 if data[i + look_back, 0] > data[i + look_back - 1, 0] else 0)
    return np.array(X), np.array(y)


def create_dataset_with_dates(df, scaled_data, look_back=60):
    """Create sequences with corresponding dates"""
    X, y, dates = [], [], []
    for i in range(len(scaled_data) - look_back - 1):
        X.append(scaled_data[i:(i + look_back), 0])
        y.append(1 if scaled_data[i + look_back, 0] > scaled_data[i + look_back - 1, 0] else 0)
        dates.append(df.index[i + look_back])
    return np.array(X), np.array(y), dates


# ============================================================
# 7. ROLLING WINDOW TRAINING FUNCTION
# ============================================================

def train_with_rolling_window(df, scaled_data, look_back=60, epochs=20, batch_size=32):
    """
    Train models using Rolling Window / Walk-Forward Validation

    Returns:
        all_predictions: List of predictions for each window
        all_metrics: Performance metrics for each window
    """

    # Create dataset with dates
    X, y, dates = create_dataset_with_dates(df, scaled_data, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Convert dates to DataFrame for easier filtering
    dates_df = pd.DataFrame({'date': dates, 'idx': range(len(dates))})
    dates_df['year'] = pd.to_datetime(dates_df['date']).dt.year

    all_predictions = []
    all_metrics = []

    early_stop = EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True,
        verbose=0
    )

    for train_start, train_end, test_year in ROLLING_WINDOWS:
        # Get indices for train and test
        train_mask = (dates_df['date'] >= train_start) & (dates_df['date'] <= train_end)
        test_mask = (dates_df['year'] == test_year)

        train_idx = dates_df[train_mask]['idx'].values
        test_idx = dates_df[test_mask]['idx'].values

        if len(train_idx) < 100 or len(test_idx) < 10:
            continue

        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        test_dates = [dates[i] for i in test_idx]

        input_shape = (look_back, 1)

        # Train Model 1: CNN-BiLSTM-Attention
        model1 = build_cnn_bilstm_attention(input_shape)
        model1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   callbacks=[early_stop], verbose=0)
        pred1 = model1.predict(X_test, verbose=0)

        # Train Model 2: CNN-BiGRU-Attention
        model2 = build_cnn_bigru_attention(input_shape)
        model2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   callbacks=[early_stop], verbose=0)
        pred2 = model2.predict(X_test, verbose=0)

        # Train Model 3: Stacked BiLSTM-Attention
        model3 = build_stacked_bilstm_attention(input_shape)
        model3.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                   callbacks=[early_stop], verbose=0)
        pred3 = model3.predict(X_test, verbose=0)

        # Ensemble
        ensemble_pred = (pred1 + pred2 + pred3) / 3.0
        ensemble_binary = (ensemble_pred > 0.5).astype(int).flatten()

        # Evaluate
        accuracy = accuracy_score(y_test, ensemble_binary)
        precision = precision_score(y_test, ensemble_binary, zero_division=0)
        recall = recall_score(y_test, ensemble_binary, zero_division=0)
        f1 = f1_score(y_test, ensemble_binary, zero_division=0)

        all_metrics.append({
            'Train_Period': f"{train_start[:4]}-{train_end[:4]}",
            'Test_Year': test_year,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1_Score': f1,
            'Train_Samples': len(y_train),
            'Test_Samples': len(y_test)
        })

        # Store predictions
        for i, date in enumerate(test_dates):
            all_predictions.append({
                'Date': date,
                'Test_Year': test_year,
                'DL_Signal': ensemble_pred[i, 0],
                'DL_Signal_Binary': ensemble_binary[i],
                'Actual': y_test[i]
            })

        # Clean up memory
        del model1, model2, model3
        tf.keras.backend.clear_session()

    return all_predictions, all_metrics


# ============================================================
# 8. MAIN TRAINING LOOP
# ============================================================

print("=" * 70)
print("🧠 BLOCK 5.5: Deep Learning Signal (UPGRADED + ROLLING WINDOW)")
print("=" * 70)
print("""
📦 Model Architecture:
   - Model 1: CNN + Bidirectional LSTM + Attention
   - Model 2: CNN + Bidirectional GRU + Attention
   - Model 3: Stacked Bidirectional LSTM + Attention

🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

⚙️ Configuration:
   - Lookback: 60 days
   - Epochs: 20 (with EarlyStopping)
   - Batch Size: 32
   - Ensemble: Average of 3 models
""")
print("=" * 70)

# Storage for results
all_stock_predictions = []
all_stock_metrics = []

# Training parameters
LOOK_BACK = 60
EPOCHS = 20
BATCH_SIZE = 32

for stock in target_stocks:
    print(f"\n{'='*50}")
    print(f"🔹 Processing: {stock}")
    print(f"{'='*50}")

    try:
        # ===== 1. Download Data =====
        df = yf.download(stock, start="2015-01-01", end="2025-12-31", progress=False)

        if len(df) < 500:
            print(f"⚠️ Skipped (insufficient data: {len(df)} rows)")
            continue

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        data = df[['Close']].values

        print(f"   📅 Data range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
        print(f"   📊 Total rows: {len(df)}")

        # ===== 2. Preprocessing =====
        kf = KalmanFilter1D(process_noise=1e-5, measurement_noise=0.01)
        smoothed_data = kf.smooth(data.flatten()).reshape(-1, 1)

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(smoothed_data)

        # ===== 3. Train with Rolling Window =====
        predictions, metrics = train_with_rolling_window(
            df, scaled_data,
            look_back=LOOK_BACK,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE
        )

        # Add stock name to results
        for pred in predictions:
            pred['Stock'] = stock
        for metric in metrics:
            metric['Stock'] = stock

        all_stock_predictions.extend(predictions)
        all_stock_metrics.extend(metrics)

        # Print window results
        print(f"\n   📊 Rolling Window Results for {stock}:")
        print(f"   {'Train Period':<15} {'Test Year':<10} {'Accuracy':<10} {'F1 Score':<10}")
        print(f"   {'-'*45}")
        for m in metrics:
            print(f"   {m['Train_Period']:<15} {m['Test_Year']:<10} {m['Accuracy']:.2%}     {m['F1_Score']:.2%}")

        # Average performance
        if metrics:
            avg_acc = np.mean([m['Accuracy'] for m in metrics])
            avg_f1 = np.mean([m['F1_Score'] for m in metrics])
            print(f"   {'-'*45}")
            print(f"   {'AVERAGE':<15} {'':<10} {avg_acc:.2%}     {avg_f1:.2%}")

    except Exception as e:
        print(f"❌ Error: {e}")


# ============================================================
# 9. SUMMARY & EXPORT
# ============================================================

print("\n" + "=" * 70)
print("📊 OVERALL PERFORMANCE SUMMARY (All Stocks, All Windows)")
print("=" * 70)

if all_stock_metrics:
    df_metrics = pd.DataFrame(all_stock_metrics)

    # Summary by Test Year
    print("\n📅 Performance by Test Year:")
    print("-" * 50)
    year_summary = df_metrics.groupby('Test_Year').agg({
        'Accuracy': 'mean',
        'Precision': 'mean',
        'Recall': 'mean',
        'F1_Score': 'mean',
        'Test_Samples': 'sum'
    }).round(4)
    print(year_summary.to_string())

    # Summary by Stock
    print("\n📈 Performance by Stock (Average across all windows):")
    print("-" * 50)
    stock_summary = df_metrics.groupby('Stock').agg({
        'Accuracy': 'mean',
        'Precision': 'mean',
        'Recall': 'mean',
        'F1_Score': 'mean'
    }).round(4)
    print(stock_summary.to_string())

    # Overall Average
    print("\n" + "=" * 50)
    print("🎯 OVERALL AVERAGE PERFORMANCE")
    print("=" * 50)
    print(f"   • Accuracy:  {df_metrics['Accuracy'].mean():.2%}")
    print(f"   • Precision: {df_metrics['Precision'].mean():.2%}")
    print(f"   • Recall:    {df_metrics['Recall'].mean():.2%}")
    print(f"   • F1 Score:  {df_metrics['F1_Score'].mean():.2%}")
    print(f"   • Total Test Samples: {df_metrics['Test_Samples'].sum():,}")

# Export Results
if all_stock_predictions:
    df_predictions = pd.DataFrame(all_stock_predictions)

    # Save predictions
    outfile_signals = "DeepLearning_Signal_BiLSTM_Attention_RollingWindow.xlsx"
    df_predictions.to_excel(outfile_signals, index=False)
    print(f"\n💾 Predictions saved to: {outfile_signals}")

    # Save metrics
    outfile_metrics = "DeepLearning_Metrics_BiLSTM_Attention_RollingWindow.xlsx"
    df_metrics.to_excel(outfile_metrics, index=False)
    print(f"💾 Metrics saved to: {outfile_metrics}")

print("\n" + "=" * 70)
print("✅ BLOCK 5.5 COMPLETED: BiLSTM/GRU + Attention + Rolling Window")
print("=" * 70)

🧠 BLOCK 5.5: Deep Learning Signal (UPGRADED + ROLLING WINDOW)

📦 Model Architecture:
   - Model 1: CNN + Bidirectional LSTM + Attention
   - Model 2: CNN + Bidirectional GRU + Attention
   - Model 3: Stacked Bidirectional LSTM + Attention

🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

⚙️ Configuration:
   - Lookback: 60 days
   - Epochs: 20 (with EarlyStopping)
   - Batch Size: 32
   - Ensemble: Average of 3 models


🔹 Processing: AMGN
   📅 Data range: 2015-01-02 to 2025-12-30
   📊 Total rows: 2765


KeyboardInterrupt: 

In [ ]:
# ============================================================
# 🔗 BLOCK 6: Data Integration (Standard)
# ============================================================
import pandas as pd
import numpy as np
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")

print("⏳ Integrating Data with Advanced Technicals...")

# target_stocks = [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

target_stocks = ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]

def calculate_adx(df, window=14):
    high = df['High']
    low = df['Low']
    close = df['Close']
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window).mean()
    plus_di = 100 * (plus_dm.ewm(alpha=1/window).mean() / atr)
    minus_di = 100 * (abs(minus_dm).ewm(alpha=1/window).mean() / atr)
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    return dx.rolling(window).mean()

def add_technical_score(df):
    df = df.copy()
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    k = df['Close'].ewm(span=12, adjust=False).mean()
    d = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = k - d
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    s_trend = np.where(df['Close'] > df['EMA_200'], 1, -1)
    s_rsi = np.where(df['RSI'] < 30, 1, np.where(df['RSI'] > 70, -1, 0))
    s_macd = np.where(df['MACD'] > df['MACD_Signal'], 1, -1)
    df['Tech_Signal'] = (s_trend * 0.4) + (s_macd * 0.4) + (s_rsi * 0.2)
    return df

print("   🔹 Step 1: Loading Full Price Data...")
df_price = pd.DataFrame()
for stock in target_stocks:
    try:
        data = yf.download(f"{stock}.BK", start="2018-01-01", end="2025-12-31", progress=False)
        if isinstance(data.columns, pd.MultiIndex): data.columns = data.columns.get_level_values(0)
        data = data.reset_index()
        data['Stock'] = stock
        data['ADX'] = calculate_adx(data)
        data = add_technical_score(data)
        df_price = pd.concat([df_price, data], ignore_index=True)
    except: pass

req_cols = ['Date', 'Stock', 'Close', 'Tech_Signal', 'ADX', 'RSI', 'MACD']
df_rl = df_price[req_cols].dropna().sort_values(['Stock', 'Date']).reset_index(drop=True)

print("   🔹 Step 2 & 3: Merging Signals...")
try:
    df_macro = pd.read_excel("Forecast_ARDL_ECM_NextMonth_Clean.xlsx")
    sig_col = 'Pred_dLogclose' if 'Pred_dLogclose' in df_macro.columns else df_macro.select_dtypes(include=[np.number]).columns[0]
    macro_map = df_macro.set_index('Stock')[sig_col].to_dict()
    df_rl['Macro_Signal'] = df_rl['Stock'].map(macro_map).fillna(0)
except: df_rl['Macro_Signal'] = 0

try:
    df_dl = pd.read_excel("DeepLearning_Signal_LSTM_GRU.xlsx")
    df_dl['Date'] = pd.to_datetime(df_dl['Date'])
    df_rl = pd.merge(df_rl, df_dl[['Date', 'Stock', 'DL_Signal']], on=['Date', 'Stock'], how='left')
    df_rl['DL_Signal'] = df_rl['DL_Signal'].fillna(0.5)
except: df_rl['DL_Signal'] = 0.5

df_rl.to_excel("Final_RL_Input_Ready.xlsx", index=False)
print("✅ Data Integration Complete!")

In [ ]:
!pip install stable_baselines3

📋 1. Constraint ที่ดัดนิสัย RLConstraintรายละเอียดจุดประสงค์Cooldown 2 Daysหลัง Sell ต้องรอ 2 วันถึงจะ Buy ได้ป้องกัน OvertradingMomentum Rule (Buy)Buy ได้เมื่อ ADX > 25 และ RSI < 75 หรือ RSI < 45บังคับซื้อตาม Trend🆕 Volatility-based SizingPosition Size 30-70% ตาม Volatilityลดความเสี่ยงช่วง Vol สูง🆕 Max Consecutive Lossesขาดทุน 5 ครั้งติด → หยุด 3 วันหยุดพักเมื่อ Strategy ไม่ workRandom Initial Position50% โอกาสเริ่มมีหุ้นอยู่แล้วฝึกให้รับมือทุกสถานการณ์🛡️ 2. Risk ManagementRuleTriggerActionจุดประสงค์Hard Stop LossPnL ≤ -6%Force SellจำกัดขาดทุนสูงสุดTrailing StopPnL > +10% แล้วตก > 4% จาก PeakForce Sellล็อคกำไร🆕 Volatility Sizingทุกครั้งที่ Buyปรับ Size 30-70%ลด Position ช่วง Vol สูง🆕 Loss Pauseขาดทุน 5 ครั้งติดหยุด 3 วันป้องกัน Drawdown ต่อเนื่อง🎁 3. Reward Shapingสถานการณ์Rewardจุดประสงค์Buy สำเร็จ+0.1กำลังใจ Take ActionSell จาก Trailing Stop+2.0รางวัลใหญ่สำหรับกำไรเยอะSell จาก Stop Loss+0.1รางวัลสำหรับ DisciplineSell มีกำไร (ปกติ)+PnL × 10ตามสัดส่วนกำไรSell ขาดทุน (ไม่ใช่ SL)-0.5ลงโทษ Sell ผิดจังหวะ🆕 Trigger Loss Pause-1.5ลงโทษหนักเมื่อขาดทุนติด 5 ครั้ง🆕 ระหว่าง Pause-0.3 ต่อวันลงโทษ Opportunity Costทุก Step+(ΔNetWorth / NetWorth) × 100ติดตาม Portfolio Value📊 4. State Vector (11 Features)#FeatureOriginal/New1Macro_SignalOriginal2DL_SignalOriginal3Tech_SignalOriginal4ADX / 100Original5Balance / 1MOriginal6Holdings Value / 1MOriginal7RSI / 100Original8MACD / 10Original9Volatility Multiplier🆕 New10Consecutive Losses / 5🆕 New11Pause Counter / 3🆕 New

┌─────────────────────────────────────┐
                    │          RL Agent (PPO)             │
                    │         เลือก Action                │
                    └─────────────────────────────────────┘
                                    │
                                    ▼
                    ┌─────────────────────────────────────┐
                    │   🆕 CHECK: Loss Pause Active?      │
                    │      (ขาดทุน 5 ครั้ง → หยุด 3 วัน)   │
                    └─────────────────────────────────────┘
                                    │
                    ┌───────────────┴───────────────┐
                    ▼                               ▼
              [Pause = YES]                   [Pause = NO]
              Force HOLD                      Continue...
              Reward -0.3                           │
                    │                               ▼
                    │               ┌─────────────────────────────────┐
                    │               │    CHECK: Risk Management       │
                    │               │    • Stop Loss -6%?             │
                    │               │    • Trailing Stop?             │
                    │               └─────────────────────────────────┘
                    │                               │
                    │               ┌───────────────┴───────────────┐
                    │               ▼                               ▼
                    │         [Risk Trigger]                  [No Trigger]
                    │         Force SELL                      Continue...
                    │               │                               │
                    │               │                               ▼
                    │               │               ┌─────────────────────────────────┐
                    │               │               │    CHECK: Cooldown Active?      │
                    │               │               │    (หลัง Sell รอ 2 วัน)          │
                    │               │               └─────────────────────────────────┘
                    │               │                               │
                    │               │               ┌───────────────┴───────────────┐
                    │               │               ▼                               ▼
                    │               │         [Cooldown = YES]               [Cooldown = NO]
                    │               │         Block BUY                       Continue...
                    │               │               │                               │
                    │               │               │                               ▼
                    │               │               │               ┌─────────────────────────────┐
                    │               │               │               │      EXECUTE ACTION         │
                    │               │               │               │  BUY / HOLD / SELL          │
                    │               │               │               └─────────────────────────────┘
                    │               │               │                               │
                    │               │               │                               ▼
                    │               │               │               ┌─────────────────────────────┐
                    │               │               │               │  🆕 BUY: Volatility Sizing  │
                    │               │               │               │     High Vol → 30-50%       │
                    │               │               │               │     Low Vol → 50-70%        │
                    │               │               │               └─────────────────────────────┘
                    │               │               │                               │
                    └───────────────┴───────────────┴───────────────────────────────┘
                                                    │
                                                    ▼
                                    ┌─────────────────────────────────┐
                                    │  🆕 SELL: Track Consecutive     │
                                    │     Losses (ถ้าขาดทุน)          │
                                    │     ถ้าครบ 5 → Trigger Pause    │
                                    │     Reward -1.5                 │
                                    └─────────────────────────────────┘
                                                    │
                                                    ▼
                                    ┌─────────────────────────────────┐
                                    │      Calculate Reward           │
                                    │      Update State               │
                                    │      Learn                      │
                                    └─────────────────────────────────┘

In [ ]:
# ============================================================
# 🤖 BLOCK 7: Train PPO (ELITE VERSION V3)
#    Based on Academic "Elite" Checklist
#
#    🆕 CRITICAL FIXES:
#    1. Risk-Adjusted Reward (Log Return - Volatility Penalty)
#    2. Transaction Costs (0.16% commission)
#    3. Normalized Observations (relative to initial balance)
#
#    Original Features:
#    - Cooldown 2 Days after Sell
#    - Stop Loss -6%
#    - Trailing Stop (Trigger 10%, Drop 4%)
#    - Volatility-based Position Sizing
#    - Max Consecutive Losses (5 losses → pause 3 days)
# ============================================================
import os
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import warnings

warnings.filterwarnings("ignore")
os.makedirs("trained_models", exist_ok=True)

# ============================================================
# 1. CONFIGURATION
# ============================================================

# Target Stocks
target_stocks = [
    "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
    "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
    "TCAP", "TISCO", "TTB"
]

# Rolling Window Configuration
ROLLING_WINDOWS = [
    (2015, 2018, 2019),
    (2016, 2019, 2020),
    (2017, 2020, 2021),
    (2018, 2021, 2022),
    (2019, 2022, 2023),
    (2020, 2023, 2024),
    (2021, 2024, 2025),
]

# 🆕 Transaction Costs (Thailand SET)
COMMISSION_RATE = 0.0016  # 0.16% (includes VAT)

# ============================================================
# 2. LOAD DATA
# ============================================================
try:
    df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
    df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
    df_rl_all['Year'] = df_rl_all['Date'].dt.year
except:
    raise RuntimeError("❌ Missing Data: Final_RL_Input_Ready.xlsx")

# ============================================================
# 3. TRADING ENVIRONMENT (ELITE VERSION)
# ============================================================
class StockTradingEnvElite(gym.Env):
    """
    Stock Trading Environment - ELITE VERSION

    Based on Academic "Elite" Checklist:

    ✅ FIX 1: Risk-Adjusted Reward
       - Use Log Returns (stationary, additive)
       - Penalize volatility (approximate Sharpe Ratio)
       - Remove arbitrary point rewards

    ✅ FIX 2: Transaction Costs
       - Deduct 0.16% commission on Buy and Sell
       - Prevents unrealistic high-frequency trading

    ✅ FIX 3: Normalized Observations
       - Balance / Initial Balance (not raw 1,000,000)
       - Position Value / Initial Balance
       - All values bounded and stationary

    Original Features (Kept):
    - Cooldown mechanism (2 days after sell)
    - Stop Loss (-6%)
    - Trailing Stop (Trigger +10%, Drop -4%)
    - Volatility-based Position Sizing
    - Max Consecutive Losses (5 losses → pause 3 days)
    """

    def __init__(self, df,
                 initial_balance=1_000_000,
                 commission_rate=COMMISSION_RATE,
                 volatility_lookback=20,
                 base_position_pct=0.5,
                 max_consecutive_losses=5,
                 loss_pause_days=3,
                 risk_penalty_factor=100):

        super(StockTradingEnvElite, self).__init__()

        self.df = df.reset_index(drop=True)
        self.initial_balance = initial_balance
        self.commission_rate = commission_rate  # 🆕 Transaction Cost
        self.risk_penalty_factor = risk_penalty_factor  # 🆕 For Sharpe-like reward

        # Action & Observation Space
        self.action_space = spaces.Discrete(3)  # 0=Hold, 1=Buy, 2=Sell
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(12,), dtype=np.float32
        )

        # Volatility-based Sizing Parameters
        self.volatility_lookback = volatility_lookback
        self.base_position_pct = base_position_pct

        # Max Consecutive Losses Parameters
        self.max_consecutive_losses = max_consecutive_losses
        self.loss_pause_days = loss_pause_days

        # Precompute volatility
        self._precompute_volatility()

        self.reset()

    def _precompute_volatility(self):
        """Precompute rolling volatility for position sizing"""
        prices = self.df['Close'].values
        returns = np.diff(prices) / prices[:-1]
        returns = np.insert(returns, 0, 0)

        self.volatility = pd.Series(returns).rolling(
            window=self.volatility_lookback,
            min_periods=5
        ).std().fillna(0.02).values

        vol_mean = np.mean(self.volatility[self.volatility > 0])
        vol_std = np.std(self.volatility[self.volatility > 0])

        if vol_std > 0:
            self.vol_multiplier = 1 - (self.volatility - vol_mean) / (2 * vol_std)
            self.vol_multiplier = np.clip(self.vol_multiplier, 0.3, 1.0)
        else:
            self.vol_multiplier = np.ones(len(self.df)) * 0.5

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0

        # Random initial position (50% chance)
        if np.random.rand() < 0.5:
            self.balance = self.initial_balance
            self.shares_held = 0
            self.avg_cost = 0.0
            self.highest_price = 0.0
        else:
            start_price = self.df.iloc[0]['Close']
            invest_pct = np.random.uniform(0.2, 0.8)
            cash_invested = self.initial_balance * invest_pct
            self.shares_held = int(cash_invested // start_price)
            cost = self.shares_held * start_price
            fee = cost * self.commission_rate  # 🆕 Include fee in initial position
            self.balance = self.initial_balance - cost - fee
            self.avg_cost = start_price * (1 + self.commission_rate)  # 🆕 Avg cost includes fee
            self.highest_price = start_price

        # Portfolio tracking
        self.prev_portfolio_value = self._get_portfolio_value()

        # Cooldown
        self.cooldown = 0

        # Consecutive Losses Tracking
        self.consecutive_losses = 0
        self.loss_pause_counter = 0

        # Statistics
        self.total_trades = 0
        self.winning_trades = 0
        self.losing_trades = 0
        self.pause_triggered_count = 0
        self.total_commission_paid = 0  # 🆕 Track total fees

        return self._next_observation(), {}

    def _get_portfolio_value(self):
        """Calculate current portfolio value (Mark-to-Market)"""
        current_price = self.df.iloc[self.current_step]['Close']
        return self.balance + (self.shares_held * current_price)

    def _get_current_volatility(self):
        """Get current volatility multiplier for position sizing"""
        return self.vol_multiplier[min(self.current_step, len(self.vol_multiplier) - 1)]

    def _next_observation(self):
        """
        🆕 ELITE FIX 3: Normalized Observations

        All values are normalized relative to initial balance
        or bounded to reasonable ranges
        """
        obs = self.df.iloc[self.current_step]
        current_price = obs['Close']
        current_vol = self._get_current_volatility()
        portfolio_value = self._get_portfolio_value()

        return np.array([
            # Market Indicators (already normalized 0-1 or small range)
            obs.get('Macro_Signal', 0),
            obs.get('DL_Signal', 0.5),
            obs.get('Tech_Signal', 0),
            obs.get('ADX', 25) / 100.0,
            obs.get('RSI', 50) / 100.0,
            obs.get('MACD', 0) / 10.0,

            # 🆕 FIX 3: Normalized Portfolio State
            self.balance / self.initial_balance,  # Cash ratio (0-1+)
            (self.shares_held * current_price) / self.initial_balance,  # Position ratio
            portfolio_value / self.initial_balance,  # Total portfolio ratio

            # Risk Management State
            current_vol,  # Volatility multiplier (0.3-1.0)
            self.consecutive_losses / self.max_consecutive_losses,  # Loss streak (0-1)
            self.loss_pause_counter / self.loss_pause_days  # Pause counter (0-1)
        ], dtype=np.float32)

    def _calculate_position_size(self):
        """Volatility-based Position Sizing"""
        vol_mult = self._get_current_volatility()
        position_pct = self.base_position_pct * vol_mult
        return np.clip(position_pct, 0.15, 0.70)

    def step(self, action):
        current_price = self.df.iloc[self.current_step]['Close']
        obs = self.df.iloc[self.current_step]

        force_sell = False
        reason = ""

        # ============================================================
        # CHECK: Max Consecutive Losses Pause
        # ============================================================
        if self.loss_pause_counter > 0:
            self.loss_pause_counter -= 1
            action = 0  # Force Hold
            if self.loss_pause_counter == 0:
                self.consecutive_losses = 0

        # ============================================================
        # CHECK: Risk Management (Stop Loss / Trailing Stop)
        # ============================================================
        if self.shares_held > 0:
            if current_price > self.highest_price:
                self.highest_price = current_price

            # 🆕 PnL calculation considers avg_cost (which includes fees)
            pnl_pct = (current_price - self.avg_cost) / self.avg_cost if self.avg_cost > 0 else 0

            # Hard Stop Loss (-6%)
            if pnl_pct <= -0.06:
                force_sell = True
                reason = "SL"

            # Trailing Stop (Trigger > +10%, Drop -4%)
            if pnl_pct > 0.10:
                drop = (self.highest_price - current_price) / self.highest_price
                if drop > 0.04:
                    force_sell = True
                    reason = "TRAIL"

        if force_sell:
            action = 2

        # ============================================================
        # CHECK: Cooldown (after sell)
        # ============================================================
        if self.cooldown > 0:
            if action == 1:
                action = 0
            self.cooldown -= 1

        # ============================================================
        # EXECUTE ACTION
        # ============================================================

        # ----- BUY -----
        if action == 1 and self.loss_pause_counter == 0:
            can_buy = False
            rsi = obs.get('RSI', 50)
            adx = obs.get('ADX', 0)

            # Momentum Rule
            if adx > 25 and rsi < 75:
                can_buy = True
            elif rsi < 45:
                can_buy = True

            if can_buy:
                position_pct = self._calculate_position_size()
                budget = self.balance * position_pct

                # 🆕 FIX 2: Calculate cost WITH commission
                max_shares = int(budget // (current_price * (1 + self.commission_rate)))

                if max_shares > 0:
                    cost = max_shares * current_price
                    fee = cost * self.commission_rate  # 🆕 Commission
                    total_cost = cost + fee

                    if self.balance >= total_cost:
                        # Update average cost (includes fee per share)
                        new_total_shares = self.shares_held + max_shares
                        new_total_cost = (self.shares_held * self.avg_cost) + total_cost
                        self.avg_cost = new_total_cost / new_total_shares

                        self.balance -= total_cost
                        self.shares_held += max_shares
                        self.highest_price = current_price
                        self.total_trades += 1
                        self.total_commission_paid += fee  # 🆕 Track fees

                        # 🆕 FIX 1: NO arbitrary reward for buying
                        # (reward is calculated based on portfolio change)

        # ----- SELL -----
        elif action == 2 and self.shares_held > 0:
            revenue_gross = self.shares_held * current_price
            fee = revenue_gross * self.commission_rate  # 🆕 Commission
            revenue_net = revenue_gross - fee

            # Calculate actual PnL (after fees)
            total_cost = self.shares_held * self.avg_cost
            pnl = revenue_net - total_cost
            pnl_pct = pnl / total_cost if total_cost > 0 else 0

            self.balance += revenue_net  # 🆕 Net of fees
            self.shares_held = 0
            self.avg_cost = 0
            self.highest_price = 0
            self.cooldown = 2
            self.total_trades += 1
            self.total_commission_paid += fee  # 🆕 Track fees

            # Track win/loss
            if pnl > 0:
                self.winning_trades += 1
                self.consecutive_losses = 0
            else:
                self.losing_trades += 1
                self.consecutive_losses += 1

            # 🆕 FIX 1: NO arbitrary rewards like +2.0 for trailing stop
            # The portfolio return already captures the profit

            # Check: Trigger Loss Pause
            if self.consecutive_losses >= self.max_consecutive_losses:
                self.loss_pause_counter = self.loss_pause_days
                self.pause_triggered_count += 1

        # ============================================================
        # STEP FORWARD & CALCULATE REWARD
        # ============================================================
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1

        # 🆕 FIX 1: Risk-Adjusted Reward (Sharpe-like)
        current_portfolio_value = self._get_portfolio_value()

        # Use Log Return (stationary, additive)
        if self.prev_portfolio_value > 0:
            portfolio_return = np.log(current_portfolio_value / self.prev_portfolio_value)
        else:
            portfolio_return = 0

        # Risk Penalty: Penalize volatility (large returns = high variance = bad)
        # This approximates Sharpe Ratio optimization
        risk_penalty = (portfolio_return ** 2) * self.risk_penalty_factor

        # Base reward = return - volatility penalty
        reward = portfolio_return - risk_penalty

        # Small penalty for holding cash (encourage seeking alpha)
        if self.shares_held == 0:
            reward -= 0.0001

        # Penalty for being in loss pause (reduced from 0.3)
        if self.loss_pause_counter > 0:
            reward -= 0.1

        # Update for next step
        self.prev_portfolio_value = current_portfolio_value

        return self._next_observation(), reward, done, False, {}

    def get_stats(self):
        """Return trading statistics"""
        final_value = self._get_portfolio_value()
        total_return = (final_value - self.initial_balance) / self.initial_balance

        return {
            'total_trades': self.total_trades,
            'winning_trades': self.winning_trades,
            'losing_trades': self.losing_trades,
            'win_rate': self.winning_trades / max(1, self.total_trades),
            'pause_triggered_count': self.pause_triggered_count,
            'total_commission_paid': self.total_commission_paid,  # 🆕
            'final_value': final_value,
            'total_return': total_return
        }


# ============================================================
# 4. BACKTESTING FUNCTION
# ============================================================
def backtest_model(model, test_df):
    """Backtest trained model on test data"""
    env = StockTradingEnvElite(test_df)
    obs, _ = env.reset()

    total_reward = 0
    actions = []
    net_worths = [env.initial_balance]
    daily_returns = []

    done = False
    prev_value = env.initial_balance

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        actions.append(action)

        current_value = env._get_portfolio_value()
        net_worths.append(current_value)

        # Track daily returns for Sharpe calculation
        if prev_value > 0:
            daily_return = (current_value - prev_value) / prev_value
            daily_returns.append(daily_return)
        prev_value = current_value

    # Calculate metrics
    final_value = net_worths[-1]
    initial_value = env.initial_balance
    total_return = (final_value - initial_value) / initial_value

    # Maximum Drawdown
    peak = net_worths[0]
    max_drawdown = 0
    for nw in net_worths:
        if nw > peak:
            peak = nw
        drawdown = (peak - nw) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    # 🆕 Sharpe Ratio (annualized)
    if len(daily_returns) > 1:
        avg_return = np.mean(daily_returns)
        std_return = np.std(daily_returns)
        if std_return > 0:
            sharpe_ratio = (avg_return / std_return) * np.sqrt(252)
        else:
            sharpe_ratio = 0
    else:
        sharpe_ratio = 0

    # 🆕 Sortino Ratio (downside risk only)
    downside_returns = [r for r in daily_returns if r < 0]
    if len(downside_returns) > 1:
        downside_std = np.std(downside_returns)
        if downside_std > 0:
            sortino_ratio = (np.mean(daily_returns) / downside_std) * np.sqrt(252)
        else:
            sortino_ratio = 0
    else:
        sortino_ratio = 0

    # Get env stats
    stats = env.get_stats()

    return {
        'Total_Return': total_return,
        'Final_Value': final_value,
        'Max_Drawdown': max_drawdown,
        'Sharpe_Ratio': sharpe_ratio,  # 🆕
        'Sortino_Ratio': sortino_ratio,  # 🆕
        'Total_Reward': total_reward,
        'N_Buys': sum(1 for a in actions if a == 1),
        'N_Sells': sum(1 for a in actions if a == 2),
        'N_Holds': sum(1 for a in actions if a == 0),
        'N_Trades': stats['total_trades'],
        'Win_Rate': stats['win_rate'],
        'Pause_Triggered': stats['pause_triggered_count'],
        'Total_Commission': stats['total_commission_paid']  # 🆕
    }


# ============================================================
# 5. MAIN TRAINING LOOP
# ============================================================
print("=" * 70)
print("🤖 BLOCK 7: PPO Training (ELITE VERSION V3)")
print("=" * 70)
print("""
🆕 ELITE FIXES (Based on Academic Literature):

   ✅ FIX 1: Risk-Adjusted Reward
      - Log Returns (stationary, additive)
      - Volatility Penalty (approximates Sharpe Ratio)
      - NO arbitrary point rewards (+2.0, +0.1)

   ✅ FIX 2: Transaction Costs
      - 0.16% commission on Buy AND Sell
      - Prevents unrealistic high-frequency trading

   ✅ FIX 3: Normalized Observations
      - Balance / Initial_Balance (not raw 1,000,000)
      - Position_Value / Initial_Balance
      - All values bounded and stationary

🔄 Rolling Window Configuration:
   - Window 1: Train [2015-2018] → Test [2019]
   - Window 2: Train [2016-2019] → Test [2020]
   - Window 3: Train [2017-2020] → Test [2021]
   - Window 4: Train [2018-2021] → Test [2022]
   - Window 5: Train [2019-2022] → Test [2023]
   - Window 6: Train [2020-2023] → Test [2024]
   - Window 7: Train [2021-2024] → Test [2025]

🛡️ Risk Management:
   - Stop Loss: -6%
   - Trailing Stop: Trigger +10%, Drop -4%
   - Cooldown: 2 days after sell
   - Volatility-based Position Sizing (30-70%)
   - Max Consecutive Losses: 5 → Pause 3 days

⚙️ PPO Configuration:
   - Learning Rate: 0.0003
   - Entropy Coef: 0.01
   - Batch Size: 64
   - Timesteps: 50,000 per window (increased for better learning)
""")
print("=" * 70)

# Storage
all_backtest_metrics = []

for stock in target_stocks:
    print(f"\n{'='*60}")
    print(f"🔹 Processing: {stock}")
    print(f"{'='*60}")

    stock_df = df_rl_all[df_rl_all['Stock'] == stock].copy()

    if len(stock_df) < 200:
        print(f"⚠️ Skipped (insufficient data: {len(stock_df)} rows)")
        continue

    stock_metrics = []

    for train_start, train_end, test_year in ROLLING_WINDOWS:
        train_df = stock_df[(stock_df['Year'] >= train_start) & (stock_df['Year'] <= train_end)].copy()
        test_df = stock_df[stock_df['Year'] == test_year].copy()

        if len(train_df) < 50:
            print(f"   ⚠️ Window {train_start}-{train_end} → {test_year}: Insufficient train data")
            continue

        if len(test_df) < 10:
            print(f"   ⚠️ Window {train_start}-{train_end} → {test_year}: Insufficient test data")
            continue

        try:
            # Create Elite environment
            env = DummyVecEnv([lambda df=train_df: StockTradingEnvElite(
                df,
                commission_rate=COMMISSION_RATE,
                volatility_lookback=20,
                base_position_pct=0.5,
                max_consecutive_losses=5,
                loss_pause_days=3,
                risk_penalty_factor=100
            )])

            model = PPO(
                "MlpPolicy",
                env,
                verbose=0,
                learning_rate=0.0003,
                ent_coef=0.01,
                batch_size=64,
                n_steps=2048,  # 🆕 Increased for better learning
                gamma=0.99,
                gae_lambda=0.95
            )

            # 🆕 Increased timesteps for better convergence
            model.learn(total_timesteps=50000)

            # Save model
            model_path = f"trained_models/ppo_elite_{stock}_train{train_start}_{train_end}_test{test_year}"
            model.save(model_path)

            # Backtest
            metrics = backtest_model(model, test_df)
            metrics['Stock'] = stock
            metrics['Train_Period'] = f"{train_start}-{train_end}"
            metrics['Test_Year'] = test_year
            metrics['Train_Samples'] = len(train_df)
            metrics['Test_Samples'] = len(test_df)

            stock_metrics.append(metrics)
            all_backtest_metrics.append(metrics)

            print(f"   ✅ Window {train_start}-{train_end} → {test_year}: "
                  f"Return={metrics['Total_Return']:+.2%} | Sharpe={metrics['Sharpe_Ratio']:.2f} | "
                  f"MDD={metrics['Max_Drawdown']:.2%} | Commission={metrics['Total_Commission']:,.0f}")

        except Exception as e:
            print(f"   ❌ Window {train_start}-{train_end} → {test_year}: Error - {e}")

    # Summary for this stock
    if stock_metrics:
        avg_return = np.mean([m['Total_Return'] for m in stock_metrics])
        avg_sharpe = np.mean([m['Sharpe_Ratio'] for m in stock_metrics])
        avg_mdd = np.mean([m['Max_Drawdown'] for m in stock_metrics])
        total_commission = sum([m['Total_Commission'] for m in stock_metrics])

        print(f"\n   📊 {stock} Average: Return={avg_return:+.2%} | Sharpe={avg_sharpe:.2f} | "
              f"MDD={avg_mdd:.2%} | Total Commission={total_commission:,.0f}")


# ============================================================
# 6. OVERALL SUMMARY
# ============================================================
print("\n" + "=" * 70)
print("📊 OVERALL PERFORMANCE SUMMARY (ELITE V3)")
print("=" * 70)

if all_backtest_metrics:
    df_metrics = pd.DataFrame(all_backtest_metrics)

    # Summary by Test Year
    print("\n📅 Performance by Test Year:")
    print("-" * 80)
    year_summary = df_metrics.groupby('Test_Year').agg({
        'Total_Return': ['mean', 'std'],
        'Sharpe_Ratio': 'mean',
        'Max_Drawdown': 'mean',
        'Win_Rate': 'mean',
        'Total_Commission': 'sum'
    }).round(4)
    print(year_summary.to_string())

    # Summary by Stock
    print("\n📈 Performance by Stock:")
    print("-" * 80)
    stock_summary = df_metrics.groupby('Stock').agg({
        'Total_Return': 'mean',
        'Sharpe_Ratio': 'mean',
        'Max_Drawdown': 'mean',
        'Win_Rate': 'mean',
        'Total_Commission': 'sum'
    }).round(4).sort_values('Sharpe_Ratio', ascending=False)  # 🆕 Sort by Sharpe
    print(stock_summary.to_string())

    # Overall Average
    print("\n" + "=" * 70)
    print("🎯 OVERALL AVERAGE PERFORMANCE")
    print("=" * 70)
    print(f"""
┌─────────────────────────────────────────────────────────────────────────┐
│                        ELITE V3 RESULTS                                 │
├─────────────────────────────────────────────────────────────────────────┤
│   • Average Return:        {df_metrics['Total_Return'].mean():>+10.2%}                            │
│   • Std Dev Return:        {df_metrics['Total_Return'].std():>10.2%}                            │
│   • Average Sharpe Ratio:  {df_metrics['Sharpe_Ratio'].mean():>10.2f}                            │
│   • Average Sortino Ratio: {df_metrics['Sortino_Ratio'].mean():>10.2f}                            │
│   • Average Max Drawdown:  {df_metrics['Max_Drawdown'].mean():>10.2%}                            │
│   • Average Win Rate:      {df_metrics['Win_Rate'].mean():>10.1%}                            │
│   • Total Commission Paid: {df_metrics['Total_Commission'].sum():>10,.0f} THB                     │
│   • Win Rate (Return>0):   {(df_metrics['Total_Return'] > 0).mean():>10.1%}                            │
│   • Total Test Windows:    {len(df_metrics):>10d}                                   │
└─────────────────────────────────────────────────────────────────────────┘
""")

    # Best and Worst (by Sharpe, not just Return)
    best_sharpe = df_metrics.loc[df_metrics['Sharpe_Ratio'].idxmax()]
    worst_sharpe = df_metrics.loc[df_metrics['Sharpe_Ratio'].idxmin()]
    best_return = df_metrics.loc[df_metrics['Total_Return'].idxmax()]

    print(f"   🏆 Best Sharpe:  {best_sharpe['Stock']} ({best_sharpe['Train_Period']} → {best_sharpe['Test_Year']}): {best_sharpe['Sharpe_Ratio']:.2f}")
    print(f"   🏆 Best Return:  {best_return['Stock']} ({best_return['Train_Period']} → {best_return['Test_Year']}): {best_return['Total_Return']:+.2%}")
    print(f"   💀 Worst Sharpe: {worst_sharpe['Stock']} ({worst_sharpe['Train_Period']} → {worst_sharpe['Test_Year']}): {worst_sharpe['Sharpe_Ratio']:.2f}")

    # Export
    outfile = "PPO_Backtest_Results_Elite_V3.xlsx"
    df_metrics.to_excel(outfile, index=False)
    print(f"\n💾 Results saved to: {outfile}")

print("\n" + "=" * 70)
print("✅ BLOCK 7 COMPLETED: PPO Elite V3 (Risk-Adjusted + Transaction Costs)")
print("=" * 70)

# ============================================================
# 📚 LITERATURE-BASED REWARD (Approximated Sharpe)
# Source: Moody & Saffell (2001) Logic
# ============================================================

# 1. Calculate Portfolio Value (Mark-to-Market)
#    (Must include cash + shares * current_price)
current_portfolio_value = self.balance + (self.shares_held * current_price)

# 2. Calculate LOG RETURN (Stationary Input)
#    We use Log Return because it handles compounding correctly.
#    Formula: ln(Current_Value / Previous_Value)
portfolio_return = np.log(current_portfolio_value / self.prev_net_worth)

# 3. The "Risk Penalty" (The Magic Sauce)
#    We penalize the SQUARE of the return.
#    Why? Large swings (both up and down) increase variance.
#    We want small, consistent gains, not huge spikes.
risk_penalty = (portfolio_return ** 2) * 100  # 100 is a scaling factor (Lambda)

# 4. Final Reward
#    Reward = Return - (Risk * Lambda)
reward = portfolio_return - risk_penalty

# 5. (Optional) Small "Existing Costs"
#    Keep your penalty for pausing, but keep it small relative to the return.
if self.loss_pause_counter > 0:
    reward -= 0.005 # Small drag, not a wall

# Update for next step
self.prev_net_worth = current_portfolio_value

In [ ]:
# ============================================================
# 🏆 BLOCK 8: Evaluation & Prediction (UPGRADED)
#    Features:
#    - Star Rating System
#    - 🆕 Performance Metrics: Sharpe Ratio, Max Drawdown, Calmar Ratio
#    - 🆕 Benchmark Comparison vs Buy-and-Hold
#    - 🆕 Rolling Window Evaluation
# ============================================================
import pandas as pd
import numpy as np
import warnings
import os
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO

warnings.filterwarnings("ignore")

# ============================================================
# 1. CONFIGURATION
# ============================================================

# [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

stock_symbols = ["BBL","KBANK","KKP","KTB","TCAP","TISCO","TTB"]

INITIAL_BALANCE = 1_000_000
COMMISSION = 0.00157
RISK_FREE_RATE = 0.02  # 2% annual risk-free rate (Thai government bond ~2%)
TRADING_DAYS_PER_YEAR = 245  # Thai market

# Rolling Window Configuration
ROLLING_WINDOWS = [
    (2015, 2018, 2019),
    (2016, 2019, 2020),
    (2017, 2020, 2021),
    (2018, 2021, 2022),
    (2019, 2022, 2023),
    (2020, 2023, 2024),
    (2021, 2024, 2025),
]

# ============================================================
# 2. LOAD DATA
# ============================================================
try:
    df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
    df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
    df_rl_all['Year'] = df_rl_all['Date'].dt.year
except:
    raise RuntimeError("❌ Missing Data: Final_RL_Input_Ready.xlsx")

# ============================================================
# 3. TRADING ENVIRONMENT
# ============================================================
class StockTradingEnv(gym.Env):
    """Trading environment matching Block 7 V2"""
    def __init__(self, df):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.balance = INITIAL_BALANCE
        self.shares_held = 0
        self.current_step = 0
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(11,), dtype=np.float32)
        self.action_space = spaces.Discrete(3)

    def reset(self, seed=None, options=None):
        self.current_step = 0
        self.balance = INITIAL_BALANCE
        self.shares_held = 0
        return self._next_observation(), {}

    def _next_observation(self):
        if self.current_step >= len(self.df):
            self.current_step = len(self.df) - 1
        obs = self.df.iloc[self.current_step]
        return np.array([
            obs.get('Macro_Signal', 0),
            obs.get('DL_Signal', 0.5),
            obs.get('Tech_Signal', 0),
            obs.get('ADX', 25) / 100.0,
            self.balance / 1_000_000.0,
            (self.shares_held * obs['Close']) / 1_000_000.0,
            obs.get('RSI', 50) / 100.0,
            obs.get('MACD', 0) / 10.0,
            0.5,  # Volatility placeholder
            0.0,  # Consecutive losses placeholder
            0.0   # Pause counter placeholder
        ], dtype=np.float32)

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        return self._next_observation(), 0, done, False, {}


# ============================================================
# 4. PERFORMANCE METRICS CALCULATOR
# ============================================================
class PerformanceMetrics:
    """
    Calculate comprehensive trading performance metrics
    """

    @staticmethod
    def calculate_returns(equity_curve):
        """Calculate daily returns from equity curve"""
        equity = np.array(equity_curve)
        returns = np.diff(equity) / equity[:-1]
        return returns

    @staticmethod
    def calculate_sharpe_ratio(returns, risk_free_rate=RISK_FREE_RATE,
                                trading_days=TRADING_DAYS_PER_YEAR):
        """
        Calculate annualized Sharpe Ratio

        Formula: Sharpe = (Mean Return - Risk Free Rate) / Std Dev × √(Trading Days)
        """
        if len(returns) < 2:
            return 0.0

        # Daily risk-free rate
        daily_rf = risk_free_rate / trading_days

        # Excess returns
        excess_returns = returns - daily_rf

        # Sharpe Ratio
        if np.std(excess_returns) == 0:
            return 0.0

        sharpe = np.mean(excess_returns) / np.std(excess_returns)

        # Annualize
        sharpe_annualized = sharpe * np.sqrt(trading_days)

        return sharpe_annualized

    @staticmethod
    def calculate_sortino_ratio(returns, risk_free_rate=RISK_FREE_RATE,
                                 trading_days=TRADING_DAYS_PER_YEAR):
        """
        Calculate annualized Sortino Ratio (downside risk only)

        Formula: Sortino = (Mean Return - Risk Free Rate) / Downside Std Dev × √(Trading Days)
        """
        if len(returns) < 2:
            return 0.0

        daily_rf = risk_free_rate / trading_days
        excess_returns = returns - daily_rf

        # Downside returns only
        downside_returns = excess_returns[excess_returns < 0]

        if len(downside_returns) == 0 or np.std(downside_returns) == 0:
            return 0.0

        sortino = np.mean(excess_returns) / np.std(downside_returns)
        sortino_annualized = sortino * np.sqrt(trading_days)

        return sortino_annualized

    @staticmethod
    def calculate_max_drawdown(equity_curve):
        """
        Calculate Maximum Drawdown (MDD)

        Formula: MDD = (Peak - Trough) / Peak
        """
        equity = np.array(equity_curve)
        peak = equity[0]
        max_dd = 0.0

        for value in equity:
            if value > peak:
                peak = value
            drawdown = (peak - value) / peak
            if drawdown > max_dd:
                max_dd = drawdown

        return max_dd

    @staticmethod
    def calculate_calmar_ratio(total_return, max_drawdown, years=1):
        """
        Calculate Calmar Ratio

        Formula: Calmar = Annualized Return / Max Drawdown
        """
        if max_drawdown == 0:
            return 0.0

        annualized_return = total_return / years if years > 0 else total_return
        calmar = annualized_return / max_drawdown

        return calmar

    @staticmethod
    def calculate_win_rate(trades):
        """Calculate win rate from list of trade PnLs"""
        if len(trades) == 0:
            return 0.0

        wins = sum(1 for t in trades if t > 0)
        return wins / len(trades)

    @staticmethod
    def calculate_profit_factor(trades):
        """
        Calculate Profit Factor

        Formula: Profit Factor = Sum(Winning Trades) / |Sum(Losing Trades)|
        """
        if len(trades) == 0:
            return 0.0

        gross_profit = sum(t for t in trades if t > 0)
        gross_loss = abs(sum(t for t in trades if t < 0))

        if gross_loss == 0:
            return float('inf') if gross_profit > 0 else 0.0

        return gross_profit / gross_loss

    @staticmethod
    def calculate_all_metrics(equity_curve, trade_pnls, years=1):
        """Calculate all metrics at once"""
        returns = PerformanceMetrics.calculate_returns(equity_curve)

        total_return = (equity_curve[-1] - equity_curve[0]) / equity_curve[0]
        max_dd = PerformanceMetrics.calculate_max_drawdown(equity_curve)

        return {
            'Total_Return': total_return,
            'Total_Return_Pct': total_return * 100,
            'Annualized_Return': (total_return / years) * 100 if years > 0 else total_return * 100,
            'Sharpe_Ratio': PerformanceMetrics.calculate_sharpe_ratio(returns),
            'Sortino_Ratio': PerformanceMetrics.calculate_sortino_ratio(returns),
            'Max_Drawdown': max_dd,
            'Max_Drawdown_Pct': max_dd * 100,
            'Calmar_Ratio': PerformanceMetrics.calculate_calmar_ratio(total_return, max_dd, years),
            'Win_Rate': PerformanceMetrics.calculate_win_rate(trade_pnls),
            'Win_Rate_Pct': PerformanceMetrics.calculate_win_rate(trade_pnls) * 100,
            'Profit_Factor': PerformanceMetrics.calculate_profit_factor(trade_pnls),
            'Total_Trades': len(trade_pnls),
            'Final_Value': equity_curve[-1]
        }


# ============================================================
# 5. BACKTEST WITH DETAILED TRACKING
# ============================================================
def backtest_with_metrics(df_test, model_path):
    """
    Backtest with detailed equity tracking for metrics calculation

    Returns:
        dict: All performance metrics + equity curve
    """
    env = StockTradingEnv(df_test)
    model = PPO.load(model_path)
    prices = df_test['Close'].values
    dates = df_test['Date'].values

    cash = INITIAL_BALANCE
    shares = 0
    avg_cost = 0.0
    highest_price = 0.0
    cooldown = 0
    consecutive_losses = 0
    loss_pause_counter = 0

    # Tracking
    equity_curve = [INITIAL_BALANCE]
    trade_pnls = []
    actions_taken = []

    obs, _ = env.reset()

    for i in range(len(prices) - 1):
        action, _ = model.predict(obs, deterministic=True)
        curr_price = prices[i]

        # Loss Pause Check
        if loss_pause_counter > 0:
            action = 0
            loss_pause_counter -= 1
            if loss_pause_counter == 0:
                consecutive_losses = 0

        # Risk Check
        force_sell = False
        reason = ""

        if shares > 0:
            if curr_price > highest_price:
                highest_price = curr_price
            pnl_pct = (curr_price - avg_cost) / avg_cost if avg_cost > 0 else 0

            if pnl_pct <= -0.06:
                force_sell = True
                reason = "SL"
            if pnl_pct > 0.10:
                if (highest_price - curr_price) / highest_price > 0.04:
                    force_sell = True
                    reason = "TRAIL"

        if force_sell:
            action = 2

        # Cooldown Check
        if cooldown > 0:
            if action == 1:
                action = 0
            cooldown -= 1

        # Execute Action
        if action == 1 and cash > curr_price and loss_pause_counter == 0:
            row = df_test.iloc[i]
            can_buy = False

            if (row.get('ADX', 0) > 25 and row.get('RSI', 50) < 75) or row.get('RSI', 50) < 45:
                can_buy = True

            if can_buy:
                # Volatility-based sizing (simplified)
                position_pct = 0.5
                budget = cash * position_pct
                buy_amt = int(budget // (curr_price * (1 + COMMISSION)))

                if buy_amt > 0:
                    cost = buy_amt * curr_price * (1 + COMMISSION)
                    total_shares = shares + buy_amt
                    avg_cost = ((shares * avg_cost) + cost) / total_shares if total_shares > 0 else curr_price
                    cash -= cost
                    shares += buy_amt
                    highest_price = curr_price
                    actions_taken.append(('BUY', i, curr_price, buy_amt))

        elif action == 2 and shares > 0:
            revenue = shares * curr_price * (1 - COMMISSION)
            pnl = revenue - (shares * avg_cost)
            pnl_pct = pnl / (shares * avg_cost) if avg_cost > 0 else 0

            trade_pnls.append(pnl)
            actions_taken.append(('SELL', i, curr_price, shares, pnl))

            cash += revenue
            shares = 0
            cooldown = 2

            # Track consecutive losses
            if pnl < 0:
                consecutive_losses += 1
                if consecutive_losses >= 5:
                    loss_pause_counter = 3
                    consecutive_losses = 0
            else:
                consecutive_losses = 0

            avg_cost = 0
            highest_price = 0

        # Update equity curve
        current_equity = cash + (shares * prices[i + 1])
        equity_curve.append(current_equity)

        obs, _, done, _, _ = env.step(action)
        if done:
            break

    # Final value
    final_value = cash + (shares * prices[-1])
    if len(equity_curve) < len(prices):
        equity_curve.append(final_value)

    # Calculate years
    days = len(df_test)
    years = days / TRADING_DAYS_PER_YEAR

    # Calculate all metrics
    metrics = PerformanceMetrics.calculate_all_metrics(equity_curve, trade_pnls, years)
    metrics['equity_curve'] = equity_curve
    metrics['trade_pnls'] = trade_pnls
    metrics['actions'] = actions_taken

    return metrics


# ============================================================
# 6. BUY-AND-HOLD BENCHMARK
# ============================================================
def calculate_buy_and_hold(df_test):
    """
    Calculate Buy-and-Hold benchmark performance

    Strategy: Buy at first day, hold until last day
    """
    prices = df_test['Close'].values

    # Buy at first price
    first_price = prices[0]
    shares = int(INITIAL_BALANCE // (first_price * (1 + COMMISSION)))
    cost = shares * first_price * (1 + COMMISSION)
    remaining_cash = INITIAL_BALANCE - cost

    # Build equity curve
    equity_curve = []
    for price in prices:
        equity = remaining_cash + (shares * price)
        equity_curve.append(equity)

    # Final value (sell at last price)
    final_value = remaining_cash + (shares * prices[-1] * (1 - COMMISSION))

    # Calculate metrics
    days = len(df_test)
    years = days / TRADING_DAYS_PER_YEAR

    # Simple trade PnL (one buy, one sell)
    trade_pnl = final_value - INITIAL_BALANCE

    metrics = PerformanceMetrics.calculate_all_metrics(equity_curve, [trade_pnl], years)
    metrics['equity_curve'] = equity_curve

    return metrics


# ============================================================
# 7. STAR RATING LOGIC
# ============================================================
def get_star_rating(action, tech_score, sharpe_ratio=0):
    """Enhanced star rating with Sharpe consideration"""
    base_rating = 3

    if action == 1:  # BUY
        if tech_score > 0.2:
            base_rating = 5  # Strong Buy
        else:
            base_rating = 4  # Buy
    elif action == 0:  # HOLD
        base_rating = 3
    elif action == 2:  # SELL
        if tech_score < -0.2:
            base_rating = 1  # Strong Sell
        else:
            base_rating = 2  # Sell

    # Adjust based on model performance (Sharpe)
    if sharpe_ratio > 1.5:
        base_rating = min(5, base_rating + 1)
    elif sharpe_ratio < 0:
        base_rating = max(1, base_rating - 1)

    return "⭐" * base_rating


# ============================================================
# 8. MAIN EVALUATION
# ============================================================
print("=" * 80)
print("🏆 BLOCK 8: Evaluation & Prediction (UPGRADED)")
print("=" * 80)
print("""
📊 Performance Metrics:
   - Total Return (%)
   - Annualized Return (%)
   - Sharpe Ratio
   - Sortino Ratio
   - Maximum Drawdown (%)
   - Calmar Ratio
   - Win Rate (%)
   - Profit Factor

📈 Benchmark: Buy-and-Hold Strategy
""")
print("=" * 80)

# Storage
all_results = []
all_comparisons = []
forecasts = []

# Process each stock
for ticker in stock_symbols:
    print(f"\n{'='*60}")
    print(f"🔹 Evaluating: {ticker}")
    print(f"{'='*60}")

    try:
        stock_df = df_rl_all[df_rl_all['Stock'] == ticker].copy()

        if len(stock_df) < 100:
            print(f"⚠️ Skipped (insufficient data: {len(stock_df)} rows)")
            continue

        stock_results = []

        # Evaluate each rolling window
        for train_start, train_end, test_year in ROLLING_WINDOWS:
            df_test = stock_df[stock_df['Year'] == test_year].copy()

            if len(df_test) < 10:
                continue

            # Model path (from Block 7 rolling window)
            model_path = f"trained_models/ppo_{ticker}_train{train_start}_{train_end}_test{test_year}.zip"

            # Fallback to old model path
            if not os.path.exists(model_path):
                model_path = f"trained_models/ppo_{ticker}.zip"

            if not os.path.exists(model_path):
                continue

            # Run backtest
            ai_metrics = backtest_with_metrics(df_test, model_path)
            bh_metrics = calculate_buy_and_hold(df_test)

            # Comparison
            result = {
                'Stock': ticker,
                'Test_Year': test_year,
                'Train_Period': f"{train_start}-{train_end}",

                # AI Strategy Metrics
                'AI_Return_%': round(ai_metrics['Total_Return_Pct'], 2),
                'AI_Sharpe': round(ai_metrics['Sharpe_Ratio'], 2),
                'AI_Sortino': round(ai_metrics['Sortino_Ratio'], 2),
                'AI_MDD_%': round(ai_metrics['Max_Drawdown_Pct'], 2),
                'AI_Calmar': round(ai_metrics['Calmar_Ratio'], 2),
                'AI_WinRate_%': round(ai_metrics['Win_Rate_Pct'], 1),
                'AI_ProfitFactor': round(ai_metrics['Profit_Factor'], 2),
                'AI_Trades': ai_metrics['Total_Trades'],

                # Benchmark Metrics
                'BH_Return_%': round(bh_metrics['Total_Return_Pct'], 2),
                'BH_Sharpe': round(bh_metrics['Sharpe_Ratio'], 2),
                'BH_MDD_%': round(bh_metrics['Max_Drawdown_Pct'], 2),

                # Comparison
                'Return_Diff_%': round(ai_metrics['Total_Return_Pct'] - bh_metrics['Total_Return_Pct'], 2),
                'Sharpe_Diff': round(ai_metrics['Sharpe_Ratio'] - bh_metrics['Sharpe_Ratio'], 2),
                'MDD_Diff_%': round(bh_metrics['Max_Drawdown_Pct'] - ai_metrics['Max_Drawdown_Pct'], 2),
                'Beat_BH': ai_metrics['Total_Return_Pct'] > bh_metrics['Total_Return_Pct']
            }

            stock_results.append(result)
            all_results.append(result)

            # Print summary
            status = "✅ WIN" if result['Beat_BH'] else "❌ LOSS"
            print(f"   {test_year}: AI={result['AI_Return_%']:+.1f}% vs BH={result['BH_Return_%']:+.1f}% "
                  f"| Sharpe={result['AI_Sharpe']:.2f} | MDD={result['AI_MDD_%']:.1f}% | {status}")

        # Stock Summary
        if stock_results:
            avg_ai_return = np.mean([r['AI_Return_%'] for r in stock_results])
            avg_bh_return = np.mean([r['BH_Return_%'] for r in stock_results])
            avg_sharpe = np.mean([r['AI_Sharpe'] for r in stock_results])
            win_count = sum(1 for r in stock_results if r['Beat_BH'])

            print(f"\n   📊 {ticker} Summary:")
            print(f"      Avg AI Return: {avg_ai_return:+.2f}%")
            print(f"      Avg BH Return: {avg_bh_return:+.2f}%")
            print(f"      Avg Sharpe:    {avg_sharpe:.2f}")
            print(f"      Beat BH:       {win_count}/{len(stock_results)} windows")

            # Generate forecast for latest data
            latest_df = stock_df[stock_df['Year'] == stock_df['Year'].max()].copy()
            if len(latest_df) > 0:
                latest_model = f"trained_models/ppo_{ticker}.zip"
                if os.path.exists(latest_model):
                    env = StockTradingEnv(latest_df)
                    obs, _ = env.reset()
                    env.current_step = len(latest_df) - 1
                    obs = env._next_observation()

                    model = PPO.load(latest_model)
                    action, _ = model.predict(obs, deterministic=True)

                    last_close = latest_df.iloc[-1]['Close']
                    tech_score = latest_df.iloc[-1].get('Tech_Signal', 0)
                    rating = get_star_rating(action, tech_score, avg_sharpe)

                    act_str = "BUY" if action == 1 else "SELL" if action == 2 else "HOLD"
                    forecasts.append({
                        'Stock': ticker,
                        'Last_Price': round(last_close, 2),
                        'AI_Action': act_str,
                        'Tech_Score': round(tech_score, 2),
                        'Avg_Sharpe': round(avg_sharpe, 2),
                        'Rating': rating
                    })

    except Exception as e:
        print(f"❌ Error: {e}")


# ============================================================
# 9. OVERALL SUMMARY
# ============================================================
print("\n" + "=" * 80)
print("📊 OVERALL PERFORMANCE SUMMARY")
print("=" * 80)

if all_results:
    df_results = pd.DataFrame(all_results)

    # Summary by Year
    print("\n📅 Performance by Test Year:")
    print("-" * 80)
    year_summary = df_results.groupby('Test_Year').agg({
        'AI_Return_%': 'mean',
        'BH_Return_%': 'mean',
        'AI_Sharpe': 'mean',
        'AI_MDD_%': 'mean',
        'Beat_BH': 'mean'
    }).round(2)
    year_summary.columns = ['AI_Return', 'BH_Return', 'Sharpe', 'MDD', 'Win_Rate']
    print(year_summary.to_string())

    # Summary by Stock
    print("\n📈 Performance by Stock:")
    print("-" * 80)
    stock_summary = df_results.groupby('Stock').agg({
        'AI_Return_%': 'mean',
        'BH_Return_%': 'mean',
        'AI_Sharpe': 'mean',
        'AI_MDD_%': 'mean',
        'Beat_BH': 'mean'
    }).round(2).sort_values('AI_Return_%', ascending=False)
    stock_summary.columns = ['AI_Return', 'BH_Return', 'Sharpe', 'MDD', 'Win_Rate']
    print(stock_summary.to_string())

    # Overall Statistics
    print("\n" + "=" * 80)
    print("🎯 OVERALL STATISTICS")
    print("=" * 80)

    print(f"""
┌─────────────────────────────────────────────────────────────────────────┐
│                        AI STRATEGY vs BUY-AND-HOLD                      │
├─────────────────────────────────────────────────────────────────────────┤
│                            │     AI Strategy    │    Buy-and-Hold      │
├─────────────────────────────────────────────────────────────────────────┤
│ Average Return             │  {df_results['AI_Return_%'].mean():>+10.2f}%     │  {df_results['BH_Return_%'].mean():>+10.2f}%      │
│ Median Return              │  {df_results['AI_Return_%'].median():>+10.2f}%     │  {df_results['BH_Return_%'].median():>+10.2f}%      │
│ Std Dev Return             │  {df_results['AI_Return_%'].std():>10.2f}%     │  {df_results['BH_Return_%'].std():>10.2f}%      │
│ Average Sharpe Ratio       │  {df_results['AI_Sharpe'].mean():>10.2f}       │  {df_results['BH_Sharpe'].mean():>10.2f}        │
│ Average Max Drawdown       │  {df_results['AI_MDD_%'].mean():>10.2f}%     │  {df_results['BH_MDD_%'].mean():>10.2f}%      │
├─────────────────────────────────────────────────────────────────────────┤
│ Win Rate vs Benchmark      │  {df_results['Beat_BH'].mean()*100:>10.1f}%     │        -             │
│ Total Test Windows         │  {len(df_results):>10d}         │        -             │
└─────────────────────────────────────────────────────────────────────────┘
""")

    # Best and Worst Performers
    best = df_results.loc[df_results['AI_Return_%'].idxmax()]
    worst = df_results.loc[df_results['AI_Return_%'].idxmin()]
    best_sharpe = df_results.loc[df_results['AI_Sharpe'].idxmax()]

    print(f"   🏆 Best Return:  {best['Stock']} ({best['Test_Year']}): {best['AI_Return_%']:+.2f}%")
    print(f"   💀 Worst Return: {worst['Stock']} ({worst['Test_Year']}): {worst['AI_Return_%']:+.2f}%")
    print(f"   📈 Best Sharpe:  {best_sharpe['Stock']} ({best_sharpe['Test_Year']}): {best_sharpe['AI_Sharpe']:.2f}")

    # Export detailed results
    outfile_detailed = "Block8_Performance_Detailed.xlsx"
    df_results.to_excel(outfile_detailed, index=False)
    print(f"\n💾 Detailed results saved to: {outfile_detailed}")

# ============================================================
# 10. FORECAST OUTPUT
# ============================================================
if forecasts:
    print("\n" + "=" * 80)
    print("🔮 AI FORECAST FOR TOMORROW")
    print("=" * 80)

    df_forecast = pd.DataFrame(forecasts)

    # Sort by rating (descending)
    df_forecast['Rating_Num'] = df_forecast['Rating'].apply(len)
    df_forecast = df_forecast.sort_values('Rating_Num', ascending=False)

    print(f"\n{'Stock':<10} {'Price':<10} {'Action':<8} {'Tech':<8} {'Sharpe':<8} {'Rating':<12}")
    print("-" * 60)

    for _, row in df_forecast.iterrows():
        print(f"{row['Stock']:<10} {row['Last_Price']:<10.2f} {row['AI_Action']:<8} "
              f"{row['Tech_Score']:<8.2f} {row['Avg_Sharpe']:<8.2f} {row['Rating']:<12}")

    # Export
    outfile_forecast = "Block8_AI_Forecast.xlsx"
    df_forecast.drop(columns=['Rating_Num']).to_excel(outfile_forecast, index=False)
    print(f"\n💾 Forecast saved to: {outfile_forecast}")

print("\n" + "=" * 80)
print("✅ BLOCK 8 COMPLETED: Evaluation with Sharpe, MDD & Benchmark Comparison")
print("=" * 80)

1) Change price frequentcy from day to hour
2) Edit Technical Block ✅
3) optimize DL signal using validation set ปรับ Block 5.5 (DL Signal)  ให้แม่นยำขึ้น ✅
4) เพิ่ม Feature ใหม่ๆ ใน Block 6 (เช่น Volume Profile, Bid/Offer) => ต้องขอจาก พี่ AIQ
5) จูน Hyperparameter ใน Block 7
6) แก้ไฟล์ Macro.xlsx เพื่อเพิ่ม Data ของ 2025 ทั้งหมด



## Excluded Part

In [ ]:
# # ============================================================
# # 🤖 BLOCK 7: Train PPO (Hierarchy: Macro First, DL Fallback)
# #    Concept: Trust Macro if aligned with Price Trend.
# #             If Macro conflicts with Price, Trust DL.
# # ============================================================
# import os
# import gymnasium as gym
# from gymnasium import spaces
# import numpy as np
# import pandas as pd
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv
# import warnings

# warnings.filterwarnings("ignore")
# os.makedirs("trained_models", exist_ok=True)

# # 1. รายชื่อหุ้น
# target_stocks = [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]

# # 2. โหลดข้อมูล
# try:
#     df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
#     df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
#     print(f"✅ Loaded Training Data: {len(df_rl_all)} rows")
# except FileNotFoundError:
#     raise RuntimeError("❌ ไม่เจอไฟล์ Final_RL_Input_Ready.xlsx")

# # --- Environment Definition ---
# class StockTradingEnv(gym.Env):
#     def __init__(self, df, initial_balance=1_000_000):
#         super(StockTradingEnv, self).__init__()
#         self.df = df.reset_index(drop=True)
#         self.initial_balance = initial_balance
#         self.commission = 0.0 # Train 0%

#         self.action_space = spaces.Discrete(3) # 0=Hold, 1=Buy, 2=Sell
#         self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)

#         self.reset()

#     def reset(self, seed=None, options=None):
#         super().reset(seed=seed)
#         self.current_step = 0

#         # MIXED INIT (เริ่มเงินสด 50% / เริ่มหุ้น 50%)
#         if np.random.rand() < 0.5:
#             self.balance = self.initial_balance
#             self.shares_held = 0
#             self.avg_cost = 0.0
#         else:
#             start_price = self.df.iloc[0]['Close']
#             invest_pct = np.random.uniform(0.2, 0.8)
#             cash_invested = self.initial_balance * invest_pct
#             self.shares_held = int(cash_invested // start_price)
#             self.balance = self.initial_balance - (self.shares_held * start_price)
#             self.avg_cost = start_price

#         self.net_worth = self.initial_balance
#         self.prev_net_worth = self.initial_balance
#         self.days_since_trade = 0

#         return self._next_observation(), {}

#     def _next_observation(self):
#         obs = self.df.iloc[self.current_step]
#         # Manual Scaling
#         return np.array([
#             obs.get('Macro_Signal', 0),
#             obs.get('DL_Signal', 0.5),
#             self.balance / 1_000_000.0,
#             (self.shares_held * obs['Close']) / 1_000_000.0,
#             obs.get('RSI', 50) / 100.0,
#             obs.get('MACD', 0) / 10.0
#         ], dtype=np.float32)

#     def step(self, action):
#         current_price = self.df.iloc[self.current_step]['Close']

#         # ดึงค่าต่างๆ มาเช็ค Logic
#         obs = self.df.iloc[self.current_step]
#         macro_val = obs.get('Macro_Signal', 0) # -1 ถึง 1
#         dl_val = obs.get('DL_Signal', 0.5)     # 0 ถึง 1
#         macd_val = obs.get('MACD', 0)          # ใช้ MACD เป็นตัวแทน Trend กราฟราคา

#         # --- 🔥 CORE LOGIC: MACRO PROTAGONIST w/ REALITY CHECK ---

#         # 1. กำหนดมุมมองของ Macro
#         macro_view = "NEUTRAL"
#         if macro_val > 0.1: macro_view = "BULL"
#         elif macro_val < -0.1: macro_view = "BEAR"

#         # 2. กำหนดมุมมองของกราฟจริง (Price Trend)
#         # MACD > 0 คือเทรนด์ขาขึ้น, MACD < 0 คือเทรนด์ขาลง
#         price_trend = "BULL" if macd_val > 0 else "BEAR"

#         # 3. ตัดสินใจ (Decision Making)
#         market_state = "NEUTRAL"

#         # เงื่อนไข: Macro ตรงกับความเป็นจริงไหม?
#         if (macro_view == "BULL" and price_trend == "BULL") or \
#            (macro_view == "BEAR" and price_trend == "BEAR"):
#             # ✅ Macro แม่น -> เชื่อ Macro (พระเอกทำงาน)
#             market_state = macro_view
#         else:
#             # ❌ Macro มั่ว (ขัดแย้งกับกราฟ) -> เชื่อ DL (พระรองมาช่วย)
#             if dl_val > 0.55: market_state = "BULL"
#             elif dl_val < 0.45: market_state = "BEAR"
#             # ถ้า DL ก็ไม่แน่ใจ ให้เป็น NEUTRAL

#         # --- Reward Calculation ---
#         reward = 0.0

#         # Action Logic
#         # BUY
#         if action == 1 and self.balance > current_price:
#             budget = self.balance * 0.5
#             shares = int(budget // current_price)
#             if shares > 0:
#                 cost = shares * current_price
#                 total_shares = self.shares_held + shares
#                 total_cost = (self.shares_held * self.avg_cost) + cost
#                 self.avg_cost = total_cost / total_shares

#                 self.balance -= cost
#                 self.shares_held += shares
#                 self.days_since_trade = 0

#                 # ให้รางวัลถ้าซื้อถูกจังหวะ
#                 if market_state == "BULL": reward += 0.2
#                 if market_state == "BEAR": reward -= 0.2

#         # SELL
#         elif action == 2 and self.shares_held > 0:
#             revenue = self.shares_held * current_price
#             profit_pct = (current_price - self.avg_cost) / self.avg_cost

#             self.balance += revenue
#             self.shares_held = 0
#             self.avg_cost = 0
#             self.days_since_trade = 0

#             # Profit Bonus
#             if profit_pct > 0:
#                 reward += (profit_pct * 20.0)
#             else:
#                 # ถ้าตลาดแย่ แล้ว Cut Loss ได้ -> ชมเชย
#                 if market_state == "BEAR": reward += 0.1
#                 else: reward -= 0.1

#         # HOLD
#         elif action == 0:
#             self.days_since_trade += 1
#             if self.days_since_trade > 3: # Inactivity Penalty
#                 reward -= 0.1 * (self.days_since_trade - 3)

#         # Step Update
#         self.current_step += 1
#         done = self.current_step >= len(self.df) - 1

#         # PnL Reward
#         next_price = self.df.iloc[self.current_step]['Close']
#         self.net_worth = self.balance + (self.shares_held * next_price)
#         pnl = ((self.net_worth - self.prev_net_worth) / self.prev_net_worth) * 100
#         reward += pnl

#         # --- 🔥 REGIME PENALTY (Floor is Lava แบบมีสมอง) ---
#         cash_ratio = self.balance / self.net_worth

#         if market_state == "BULL":
#             # ตลาดดี (Macro แม่น หรือ DL ช่วยยืนยัน) -> ต้องถือหุ้น!
#             if cash_ratio > 0.5: reward -= 0.5

#         elif market_state == "BEAR":
#             # ตลาดแย่ -> ต้องถือเงินสด!
#             if cash_ratio < 0.5: reward -= 0.5

#         self.prev_net_worth = self.net_worth
#         return self._next_observation(), reward, done, False, {}

# # --- Training Loop ---
# print(f"🚀 Starting Training (Macro First -> Fallback to DL)...")

# TOTAL_TIMESTEPS = 30000

# for stock in target_stocks:
#     train_df = df_rl_all[(df_rl_all['Stock'] == stock) & (df_rl_all['Date'].dt.year < 2024)]
#     if len(train_df) < 50: continue

#     env = DummyVecEnv([lambda: StockTradingEnv(train_df)])
#     model = PPO("MlpPolicy", env, verbose=0, learning_rate=0.0003, ent_coef=0.1, batch_size=64)

#     try:
#         model.learn(total_timesteps=TOTAL_TIMESTEPS)
#         model.save(f"trained_models/ppo_{stock}")
#         print(f"✅ Trained: {stock}")
#     except Exception as e:
#         print(f"❌ Error {stock}: {e}")

# print("\n🎉 Training Complete!")

In [ ]:
# # ============================================================
# # 🏆 BLOCK 8: FINAL EVALUATION
# # ============================================================
# import pandas as pd
# import numpy as np
# import warnings
# import os
# import gymnasium as gym
# from gymnasium import spaces
# from stable_baselines3 import PPO
# from stable_baselines3.common.vec_env import DummyVecEnv

# warnings.filterwarnings("ignore")

# stock_symbols = [
#     "AAV", "AEONTS", "AOT", "BA", "BBL", "BCH", "BDMS", "BEM", "BH", "BTS",
#     "CHG", "JMT", "KBANK", "KKP", "KTB", "KTC", "PR9", "PRM", "RCL", "SAWAD",
#     "TCAP", "TISCO", "TTB"
# ]
# initial_balance = 1_000_000

# try:
#     df_rl_all = pd.read_excel("Final_RL_Input_Ready.xlsx")
#     df_rl_all['Date'] = pd.to_datetime(df_rl_all['Date'])
# except:
#     raise RuntimeError("❌ Missing Data File")

# class StockTradingEnv(gym.Env):
#     def __init__(self, df):
#         super().__init__()
#         self.df = df.reset_index(drop=True)
#         self.balance = initial_balance
#         self.shares_held = 0
#         self.current_step = 0
#         self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)
#         self.action_space = spaces.Discrete(3)

#     def reset(self, seed=None, options=None):
#         self.current_step = 0
#         self.balance = initial_balance
#         self.shares_held = 0
#         return self._next_observation(), {}

#     def _next_observation(self):
#         obs = self.df.iloc[self.current_step]
#         return np.array([
#             obs.get('Macro_Signal', 0),
#             obs.get('DL_Signal', 0.5),
#             self.balance / 1_000_000.0,
#             (self.shares_held * obs['Close']) / 1_000_000.0,
#             obs.get('RSI', 50) / 100.0,
#             obs.get('MACD', 0) / 10.0
#         ], dtype=np.float32)

#     def step(self, action):
#         self.current_step += 1
#         done = self.current_step >= len(self.df) - 1
#         return self._next_observation(), 0, done, False, {}

# # --- Backtest Logic ---
# def backtest_logic(env, model):
#     if hasattr(env, 'envs'): df = env.envs[0].unwrapped.df
#     else: df = env.unwrapped.df

#     prices = df['Close'].values
#     cash = initial_balance
#     shares = 0
#     trades = 0
#     commission = 0.001

#     obs = env.reset()

#     for i in range(len(prices)-1):
#         action, _ = model.predict(obs, deterministic=True)
#         curr_price = prices[i]

#         # Buy
#         if action == 1 and cash > curr_price:
#             budget = cash * 0.5
#             buy_amt = int(budget // (curr_price * (1 + commission)))
#             if buy_amt > 0:
#                 cost = buy_amt * curr_price * (1 + commission)
#                 cash -= cost
#                 shares += buy_amt
#                 trades += 1

#         # Sell
#         elif action == 2 and shares > 0:
#             cash += shares * curr_price * (1 - commission)
#             shares = 0
#             trades += 1

#         obs, _, done, _ = env.step(action)
#         if done: break

#     final_val = cash + (shares * prices[-1])
#     return final_val, trades

# # --- Run Loop ---
# results = []
# print(f"🚀 Testing on 2024-2025 Data (Macro-Led Strategy)...\n")
# print(f"{'Stock':<10} | {'AI %':<10} | {'Bench %':<10} | {'Diff %':<10} | {'Trades':<8} | {'Status'}")
# print("-" * 75)

# for ticker in stock_symbols:
#     try:
#         df_test = df_rl_all[(df_rl_all['Stock'] == ticker) & (df_rl_all['Date'].dt.year >= 2024)]
#         if len(df_test) < 10: continue

#         model_path = f"trained_models/ppo_{ticker}.zip"

#         if os.path.exists(model_path):
#             env = DummyVecEnv([lambda: StockTradingEnv(df_test)])
#             model = PPO.load(model_path, env=env)
#             final_val, trades = backtest_logic(env, model)

#             ai_ret = ((final_val - initial_balance)/initial_balance)*100
#             bench_ret = ((df_test.iloc[-1]['Close'] - df_test.iloc[0]['Close'])/df_test.iloc[0]['Close'])*100
#             diff = ai_ret - bench_ret
#             status = "WIN 🏆" if ai_ret > bench_ret else "LOSS ❌"

#             results.append({'Stock': ticker, 'AI_Return_%': round(ai_ret,2), 'Benchmark_Return_%': round(bench_ret,2), 'Diff_%': round(diff, 2), 'Trades': trades, 'Status': status})
#             print(f"{ticker:<10} | {ai_ret:>9.2f}% | {bench_ret:>9.2f}% | {diff:>9.2f}% | {trades:<8} | {status}")

#     except Exception as e:
#         print(f"❌ Error {ticker}: {e}")

# if results:
#     df_res = pd.DataFrame(results)
#     print(f"\n{df_res.to_markdown(index=False)}")
#     df_res.to_excel("Final_AI_WinLoss_Summary.xlsx", index=False)

# Backtesting

In [ ]:
# import pandas as pd
# import numpy as np

# # ============================================================
# # ⚙️ BLOCK: Historical vs Predicted ΔLogclose (Clean Export)
# # ============================================================

# def rolling_ecm_forecast_table(stock_name, sector_name, df_macro, df_gamma, df_alpha, df_beta, combined_dfs_selective_diff):
#     """
#     ทำนาย ΔLogclose (เดือน t) จากข้อมูลเดือน t−1 แล้วเปรียบเทียบกับ actual
#     คืน DataFrame: Date | Actual_dLogclose | Pred_dLogclose | Error | APE_% | Stock | Sector
#     """
#     # --- เตรียมข้อมูลหุ้นและ macro ---
#     df_stock = combined_dfs_selective_diff[sector_name][[f"Logclose_{stock_name}"]].copy()
#     df_stock.columns = ["Logclose"]
#     df_macro_aligned = df_macro.reindex(df_stock.index).ffill().copy()

#     # --- พารามิเตอร์จากโมเดล ---
#     btab = df_beta[df_beta["Stock"] == stock_name]
#     gtab = df_gamma[df_gamma["Stock"] == stock_name].copy()
#     atab = df_alpha[df_alpha["Stock"] == stock_name]
#     if btab.empty or gtab.empty or atab.empty:
#         return None

#     lr_params = btab.drop_duplicates(subset="Variable").set_index("Variable")["coef"]
#     gtab = gtab[gtab["Variable"].str.contains(r"D\.L\d+\.", regex=True, na=False)]
#     gtab = pd.concat([
#         gtab,
#         pd.DataFrame([{"Variable": "ECT_lag1", "Coef": float(atab["Alpha"].iloc[0])}])
#     ], ignore_index=True)

#     # --- เตรียม DataFrame ผลลัพธ์ ---
#     df_out = pd.DataFrame(index=df_stock.index)
#     df_out["Actual_dLogclose"] = df_stock["Logclose"].diff()
#     df_out["Pred_dLogclose"] = np.nan

#     # --- Forecast แบบ rolling ---
#     for i in range(4, len(df_stock)):
#         deltas = {}
#         past_window = df_stock.iloc[i-4:i].copy()
#         past_macro = df_macro_aligned.iloc[i-4:i].copy()

#         # ECT lag
#         const = float(lr_params.get("const", 0.0))
#         X_lag = {k: float(past_macro[k].iloc[-2]) for k in lr_params.index if k != "const" and k in past_macro.columns}
#         Y_lag = float(past_window["Logclose"].iloc[-2])
#         ect_val = Y_lag - (const + sum(float(lr_params[k]) * X_lag[k] for k in X_lag))
#         deltas["ECT_lag1"] = ect_val

#         # ΔY / ΔX lags
#         for var in gtab["Variable"]:
#             if var == "ECT_lag1":
#                 continue
#             parts = var.split(".")
#             lag = int(parts[1].replace("L", ""))
#             base = parts[2]

#             if base.startswith("Logclose_"):
#                 stock_base = base.replace("Logclose_", "")
#                 if stock_base != stock_name:
#                     continue
#                 dy = float(past_window["Logclose"].iloc[-lag]) - float(past_window["Logclose"].iloc[-(lag + 1)])
#                 deltas[var] = dy
#             else:
#                 if base in past_macro.columns:
#                     dx = float(past_macro[base].iloc[-lag]) - float(past_macro[base].iloc[-(lag + 1)])
#                     deltas[var] = dx
#                 else:
#                     deltas[var] = 0.0

#         df_pred = pd.DataFrame(list(deltas.items()), columns=["Variable", "Value"])
#         merged = pd.merge(gtab[["Variable", "Coef"]], df_pred, on="Variable", how="left")
#         merged["Contribution"] = merged["Coef"].astype(float) * merged["Value"].astype(float)
#         df_out.iloc[i, df_out.columns.get_loc("Pred_dLogclose")] = merged["Contribution"].sum()

#     # --- Error metrics ---
#     EPS = 1e-6
#     df_out["Error"] = df_out["Pred_dLogclose"] - df_out["Actual_dLogclose"]
#     df_out["APE_%"] = np.where(
#         df_out["Actual_dLogclose"].abs() > EPS,
#         np.abs(df_out["Error"] / df_out["Actual_dLogclose"]) * 100,
#         np.nan
#     )
#     df_out["Stock"] = stock_name
#     df_out["Sector"] = sector_name
#     return df_out.dropna(subset=["Pred_dLogclose"])


# # ============================================================
# # 🚀 Run & Export เฉพาะรายหุ้น (Sheet 2 เท่านั้น)
# # ============================================================

# evaluation_results = {}

# for sector_name, df_sec in combined_dfs_selective_diff.items():
#     for col in df_sec.columns:
#         if not col.startswith("Logclose_"):
#             continue
#         stock_name = col.replace("Logclose_", "")
#         try:
#             df_eval = rolling_ecm_forecast_table(
#                 stock_name, sector_name, df, df_gamma, df_alpha, df_beta, combined_dfs_selective_diff
#             )
#             if df_eval is not None and not df_eval.empty:
#                 evaluation_results[stock_name] = df_eval
#                 mae = df_eval["Error"].abs().mean()
#                 rmse = np.sqrt((df_eval["Error"] ** 2).mean())
#                 mape = df_eval["APE_%"].replace([np.inf, -np.inf], np.nan).mean()
#                 print(f"✅ {stock_name:<8} | MAE={mae:.5f} | RMSE={rmse:.5f} | MAPE={mape:.2f}% | Obs={len(df_eval)}")
#         except Exception as e:
#             print(f"⚠️ {stock_name} failed: {e}")

# # --- Export เฉพาะรายหุ้น ---
# output_file = "ECM_Forecast_Evaluation_ByStock.xlsx"
# with pd.ExcelWriter(output_file) as writer:
#     for stock, df_eval in evaluation_results.items():
#         df_clean = df_eval.replace([np.inf, -np.inf], np.nan).dropna(how="all")
#         df_clean.to_excel(writer, sheet_name=stock[:31], index=True)

# print(f"\n✅ Exported → {output_file}")
# print(f"รวมทั้งหมด {len(evaluation_results)} หุ้น (เฉพาะ Sheet รายหุ้นเท่านั้น)")


# # หลังจากรันแล้ว evaluation_results มี dictionary ของหุ้นแต่ละตัว

# for stock, df_eval in evaluation_results.items():
#     print(f"\n==== {stock} ====")
#     print(df_eval.tail(5)[["Actual_dLogclose","Pred_dLogclose","Error","APE_%"]])



In [ ]:
# import pandas as pd
# df_macro = pd.read_excel("Macro_data.xlsx")
# print(df_macro.columns.tolist())


In [ ]:
# # ============================================================
# # 🧠 BLOCK 5.2 — Regime-aware Hybrid Optimization Strategy (Fixed)
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import yfinance as yf
# from pathlib import Path

# # ---------- INPUT FILES ----------
# f_forecast = "Forecast_ARDL_ECM_NextMonth_Clean.xlsx"
# f_daily = "Historical_Technical_Indicators_bySector.xlsx"
# f_macro = "Macro_data.xlsx"

# # ---------- LOAD DATA ----------
# df_fore = pd.read_excel(f_forecast)[["Stock", "Sector", "Pred_dLogclose"]]
# df_daily = pd.read_excel(f_daily)
# df_daily["Date"] = pd.to_datetime(df_daily["Date"])

# # --- Load macro base ---
# df_macro = pd.read_excel(f_macro)
# df_macro["Date"] = pd.to_datetime(df_macro["Date"])

# # --- FX: USD/THB ---
# fx1 = yf.download("THB=X", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(fx1.columns, pd.MultiIndex):
#     fx1.columns = fx1.columns.get_level_values(0)
# fx1 = fx1.reset_index()[["Date", "Close"]].rename(columns={"Close": "THB_per_USD"})

# # --- FX: THB/CNY ---

# fx2 = yf.download("THBCNY=X", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(fx2.columns, pd.MultiIndex):
#     fx2.columns = fx2.columns.get_level_values(0)
# fx2 = fx2.reset_index()[["Date", "Close"]].rename(columns={"Close": "THB_per_CNY"})


# # # --- Oil: Brent ---
# # oil = yf.download("BZ=F", start="2015-01-01", end="2025-01-01", progress=False)
# # if isinstance(oil.columns, pd.MultiIndex):
# #     oil.columns = oil.columns.get_level_values(0)
# # oil = oil.reset_index()[["Date", "Close"]].rename(columns={"Close": "Brent_Oil_USD_per_bbl"})

# # --- SET Index---
# thai = yf.download("^SET.BK", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(thai.columns, pd.MultiIndex):
#     thai.columns = thai.columns.get_level_values(0)
# thai = thai.reset_index()[["Date", "Close"]].rename(columns={"Close": "SET_Index"})

# # --- S&P500---
# usa = yf.download("^GSPC", start="2015-01-01", end="2025-01-01", progress=False)
# if isinstance(usa.columns, pd.MultiIndex):
#     usa.columns = usa.columns.get_level_values(0)
# usa = usa.reset_index()[["Date", "Close"]].rename(columns={"Close": "SP500_Index"})

# # --- Merge macro + yfinance series ---
# df_macro = (
#     df_macro
#     .merge(fx1, on="Date", how="left")
#     .merge(fx2, on="Date", how="left")
#     .merge(thai, on="Date", how="left")
#     .merge(usa, on="Date", how="left")

# )

# # --- Rename for compatibility ---
# df_macro.rename(columns={
#     "Bond_spread_10Y_5Y": "10Y_5Y_Bond_Spread",
#     "Bond_spread_5Y_1Y": "5Y_1Y_Bond_Spread",
#     "Bond_spread_10Y_1Y": "10Y_1Y_Bond_Spread"
# }, inplace=True)

# df_macro = df_macro.set_index("Date").sort_index()

# # ============================================================
# # STEP 1: Detect Macro Regime
# # ============================================================
# def detect_regime(df_macro):
#     df = df_macro.copy()

#     # Handle missing vars gracefully
#     for col in ["10Y_5Y_Bond_Spread","5Y_1Y_Bond_Spread","THOR_6M",
#                "THB_per_USD" ]:    # "Brent_Oil_USD_per_bbl"
#         if col not in df.columns:
#             df[col] = df[col].interpolate(limit_direction="both")

#     z_10_5 = (df["10Y_5Y_Bond_Spread"] - df["10Y_5Y_Bond_Spread"].mean()) / df["10Y_5Y_Bond_Spread"].std()
#     z_5_1 = (df["5Y_1Y_Bond_Spread"] - df["5Y_1Y_Bond_Spread"].mean()) / df["5Y_1Y_Bond_Spread"].std()
#     z_thor = (df["THOR_6M"] - df["THOR_6M"].mean()) / df["THOR_6M"].std()
#     # z_oil = (df["Brent_Oil_USD_per_bbl"] - df["Brent_Oil_USD_per_bbl"].mean()) / df["Brent_Oil_USD_per_bbl"].std()
#     z_fx = (df["THB_per_USD"] - df["THB_per_USD"].mean()) / df["THB_per_USD"].std()
#     z_thai = (df["SET_Index"] - df["SET_Index"].mean()) / df["SET_Index"].std()
#     z_usa = (df["SP500_Index"] - df["SP500_Index"].mean()) / df["SP500_Index"].std()

#     # Composite index
#     composite = 0.4*(z_10_5 + z_5_1)/2 - 0.3*z_thor - 0.15*z_fx + 0.15*z_thai + 0.15*z_usa
#     df["Regime_Score"] = composite
#     df["Regime"] = np.select(
#         [composite > 0.7, composite < -0.7],
#         ["Expansion", "Contraction"],
#         default="Neutral"
#     )
#     return df[["Regime","Regime_Score"]]

# df_regime = detect_regime(df_macro)

# # ============================================================
# # STEP 2: Merge Forecast + Daily + Regime
# # ============================================================
# expanded = []
# for _, row in df_fore.iterrows():
#     stock, sector, pred = row["Stock"], row["Sector"], row["Pred_dLogclose"]
#     df_s = df_daily[df_daily["Stock"] == stock].copy()
#     if df_s.empty:
#         continue
#     df_s["Pred_dLogclose"] = pred
#     df_s["Sector"] = sector
#     df_s = df_s.merge(df_regime, left_on="Date", right_index=True, how="left")
#     expanded.append(df_s)

# df_all = pd.concat(expanded, ignore_index=True).sort_values(["Sector","Stock","Date"])

# # ============================================================
# # STEP 3: Regime-dependent Weights
# # ============================================================
# def regime_weights(r):
#     if r == "Expansion": return (0.6, 0.25, 0.15)
#     elif r == "Contraction": return (0.3, 0.5, 0.2)
#     else: return (0.4, 0.4, 0.2)

# # ============================================================
# # STEP 4: Hybrid Score Computation
# # ============================================================
# df_all["Return"] = df_all.groupby("Stock")["Close"].pct_change()
# df_all["LogReturn"] = np.log1p(df_all["Return"])

# hybrid_results, equity_curves = [], {}

# for stock, sub in df_all.groupby("Stock"):
#     sub = sub.copy().sort_values("Date")

#     techconfirm = (
#         0.3 * sub["TechScore"].fillna(0)
#         + 0.2 * sub["MACD"].fillna(0)
#         + 0.1 * ((60 - abs(sub["RSI_14"].fillna(50)-50)) / 50)
#     )
#     wave_align = np.where(sub["Wave_Direction"] == ("Up" if sub["Pred_dLogclose"].iloc[0] > 0 else "Down"), 1, -1)
#     patternwave = (
#         0.2 * sub["Pattern_Confidence"].fillna(0)
#         + 0.2 * sub["Wave_Strength"].fillna(0) * wave_align
#     )

#     hybrid_score = []
#     for i, regime in enumerate(sub["Regime"]):
#         w_macro, w_tech, w_pattern = regime_weights(regime)
#         bias = np.sign(sub["Pred_dLogclose"].iloc[0])
#         score = (w_macro*bias) + (w_tech*techconfirm.iloc[i]) + (w_pattern*patternwave.iloc[i])
#         hybrid_score.append(score)

#     sub["Score"] = hybrid_score
#     sub["FinalSignal"] = np.where(abs(sub["Score"]) < 0.05, 0, np.sign(sub["Score"]))
#     sub["Strategy_Return"] = sub["FinalSignal"].shift(1) * sub["Return"]
#     sub["Cumulative_Return"] = (1 + sub["Strategy_Return"]).cumprod() - 1

#     # --- Performance Summary ---
#     hit_ratio = (np.sign(sub["Return"]) == sub["FinalSignal"].shift(1)).mean() * 100
#     cum_ret = sub["Cumulative_Return"].iloc[-1] * 100
#     avg_ret = sub["Strategy_Return"].mean() * 100
#     std_ret = sub["Strategy_Return"].std() * 100
#     sharpe = avg_ret / std_ret if std_ret > 0 else np.nan

#     hybrid_results.append({
#         "Stock": stock, "Sector": sub["Sector"].iloc[0],
#         "Hit_Ratio_%": hit_ratio, "Cumulative_Return_%": cum_ret,
#         "Avg_Daily_Return_%": avg_ret, "Volatility_%": std_ret,
#         "Sharpe_Ratio": sharpe
#     })
#     equity_curves[stock] = sub[["Date","Strategy_Return","Cumulative_Return","Regime"]]

# # ============================================================
# # STEP 5: Portfolio Summary
# # ============================================================
# all_dates = pd.concat([v["Date"] for v in equity_curves.values()]).drop_duplicates().sort_values()
# port = pd.DataFrame(index=all_dates)

# for k, v in equity_curves.items():
#     df_v = v.drop_duplicates(subset="Date").set_index("Date")  # 🔧 ป้องกัน duplicate
#     port[k] = df_v["Strategy_Return"].reindex(all_dates)

# # คำนวณผลตอบแทนพอร์ตเฉลี่ยรายวัน
# port["Portfolio_Return"] = port.mean(axis=1, skipna=True)
# port["Cumulative_Portfolio"] = (1 + port["Portfolio_Return"].fillna(0)).cumprod() - 1
# port_cum = port["Cumulative_Portfolio"].iloc[-1]*100
# port_avg = port["Portfolio_Return"].mean()*100
# port_std = port["Portfolio_Return"].std()*100
# port_sharpe = port_avg/port_std if port_std>0 else np.nan

# portfolio_summary = pd.DataFrame([{
#     "Portfolio_CumReturn_%": port_cum,
#     "Portfolio_AvgDailyReturn_%": port_avg,
#     "Portfolio_Volatility_%": port_std,
#     "Portfolio_Sharpe": port_sharpe
# }])

# df_strategy_summary = pd.DataFrame(hybrid_results).sort_values("Cumulative_Return_%", ascending=False)

# # ============================================================
# # STEP 6: Export + Visualization
# # ============================================================
# out_path = Path("Hybrid_Regime_Aware_Strategy.xlsx")
# with pd.ExcelWriter(out_path) as writer:
#     df_strategy_summary.to_excel(writer, sheet_name="Stock_Summary", index=False)
#     portfolio_summary.to_excel(writer, sheet_name="Portfolio_Summary", index=False)
#     port.to_excel(writer, sheet_name="Portfolio_EquityCurve", index=True)

# print(f"✅ Exported → {out_path.name}")

# plt.figure(figsize=(10,6))
# plt.plot(port["Cumulative_Portfolio"], label="Regime-aware Portfolio", lw=2)
# plt.title("Regime-Aware Hybrid Macro + Technical Strategy — Cumulative Return")
# plt.ylabel("Cumulative Return")
# plt.grid(True); plt.legend(); plt.show()

# # # ============================================================
# # # 📤 STEP 1.5 — Export Regime Scores for ML Fusion (used in Block 5.2B)
# # # ============================================================
# # df_regime_export = df_regime.reset_index()
# # df_regime_export.to_excel("Macro_Regime_Score.xlsx", index=False)
# # print("✅ Exported → Macro_Regime_Score.xlsx")

In [ ]:
# # ============================================================
# # 💰 BLOCK 5.3 — Dynamic Portfolio Simulation + Dashboard (Holdings Summary Version)
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import yfinance as yf

# # --- Simulation Parameters ---
# initial_capital = 10_000_000
# trade_frac = 0.05  # 3% per signal

# # ============================================================
# # 🧩 FIX — Ensure FinalSignal exists
# # ============================================================
# df_sim = df_all.copy().sort_values(["Date", "Stock"]).drop_duplicates(subset=["Date", "Stock"], keep="last")

# # ============================================================
# # 🧩 FIX — Ensure FinalSignal exists (Robust)
# # ============================================================
# df_sim = df_all.copy().sort_values(["Date", "Stock"]).drop_duplicates(subset=["Date", "Stock"], keep="last")

# if "FinalSignal" not in df_sim.columns:
#     print("⚠️  Column 'FinalSignal' not found → auto-generating ...")

#     if "Score" in df_sim.columns:
#         # ใช้ Score ถ้ามี
#         df_sim["FinalSignal"] = np.where(abs(df_sim["Score"]) < 0.05, 0, np.sign(df_sim["Score"]))

#     elif "Pred_dLogclose" in df_sim.columns:
#         # ใช้ Pred_dLogclose จาก ECM forecast
#         df_sim["FinalSignal"] = np.sign(df_sim["Pred_dLogclose"])

#     elif "Pred_Return" in df_sim.columns:
#         # fallback กรณีมีชื่ออื่น
#         df_sim["FinalSignal"] = np.sign(df_sim["Pred_Return"])

#     else:
#         # สุดท้าย ถ้าไม่มีอะไรเลย ให้ถือว่าไม่มี signal (ถือเงินสด)
#         print("⚠️  Neither Score nor Pred_dLogclose found — assuming all signals = 0 (no trades).")
#         df_sim["FinalSignal"] = 0


# # ============================================================
# # 🧮 Create pivot tables safely (handle duplicates)
# # ============================================================
# price_map = df_sim.pivot_table(index="Date", columns="Stock", values="Close", aggfunc="last")
# signal_map = df_sim.pivot_table(index="Date", columns="Stock", values="FinalSignal", aggfunc="last")

# # ============================================================
# # 🚀 Portfolio Simulation (No short-selling, no borrowing)
# # ============================================================
# cash = initial_capital
# holdings = {s: 0.0 for s in df_sim["Stock"].unique()}
# portfolio_history = []
# dates = sorted(df_sim["Date"].unique())

# for date in dates:
#     prices = price_map.loc[date]
#     signals = signal_map.loc[date]

#     # Portfolio valuation
#     holdings_value = sum(prices[s]*holdings[s] for s in holdings if not np.isnan(prices[s]))
#     total_value = cash + holdings_value

#     # --- Execute trades ---
#     for stock in holdings.keys():
#         price = prices[stock]
#         if np.isnan(price):
#             continue
#         sig = signals[stock]

#         if sig == 1:  # BUY (max 3% of remaining cash)
#             buy_amt = trade_frac * cash
#             if buy_amt > 0:
#                 qty = buy_amt / price
#                 holdings[stock] += qty
#                 cash -= qty * price

#         elif sig == -1:  # SELL (max 3% of current holdings)
#             sell_qty = trade_frac * holdings[stock]
#             if sell_qty > 0:
#                 holdings[stock] -= sell_qty
#                 cash += sell_qty * price

#     # --- Update after trades ---
#     holdings_value = sum(prices[s]*holdings[s] for s in holdings if not np.isnan(prices[s]))
#     total_value = cash + holdings_value

#     portfolio_history.append({
#         "Date": date,
#         "Cash": cash,
#         "Holdings_Value": holdings_value,
#         "Total_Portfolio": total_value
#     })

# # ============================================================
# # 📊 Portfolio Performance + Benchmark SET
# # ============================================================
# df_portfolio = pd.DataFrame(portfolio_history)
# df_portfolio["Daily_Return"] = df_portfolio["Total_Portfolio"].pct_change()
# df_portfolio["Cumulative_Return"] = (1 + df_portfolio["Daily_Return"].fillna(0)).cumprod() - 1

# # ============================================================
# # 🧭 Benchmark SET Index (Functional Fix)
# # ============================================================
# try:
#     set_idx = yf.download("^SET.BK", start=df_portfolio["Date"].min(), end=df_portfolio["Date"].max(), progress=False)
#     if not set_idx.empty:
#         if isinstance(set_idx.columns, pd.MultiIndex):
#             set_idx.columns = set_idx.columns.get_level_values(0)
#         set_idx = set_idx["Close"].resample("D").ffill().squeeze()
#         if isinstance(set_idx, pd.DataFrame):
#             set_idx = set_idx.iloc[:, 0]
#         set_ret = set_idx.pct_change().fillna(0)
#         set_cum = (1 + set_ret).cumprod() - 1
#         set_df = pd.DataFrame({"Date": set_idx.index, "SET_Close": set_idx.values, "SET_Return": set_ret.values, "SET_CumReturn": set_cum.values})
#         set_df["Date"] = pd.to_datetime(set_df["Date"])
#         df_portfolio = df_portfolio.reset_index(drop=True)
#         df_portfolio["Date"] = pd.to_datetime(df_portfolio["Date"])
#         df_portfolio = pd.merge_asof(
#             df_portfolio.sort_values("Date"),
#             set_df.sort_values("Date"),
#             on="Date",
#             direction="backward"
#         ).set_index("Date")
#         print("✅ SET Index benchmark successfully merged.")
#     else:
#         print("⚠️ Warning: SET Index data not available.")
# except Exception as e:
#     print(f"⚠️ Error merging SET Index: {e}")

# # ============================================================
# # 📈 Visualization Dashboard (Portfolio vs SET)
# # ============================================================
# plt.figure(figsize=(10,6))
# plt.plot(df_portfolio.index, df_portfolio["Cumulative_Return"], label="Strategy Portfolio", lw=2)
# plt.plot(df_portfolio.index, df_portfolio["SET_CumReturn"], label="SET Index", lw=2, ls="--")
# plt.title("Portfolio vs SET Index Benchmark", fontsize=13, fontweight="bold")
# plt.ylabel("Cumulative Return")
# plt.grid(True); plt.legend()
# plt.show()

# # ============================================================
# # 📋 Latest Holdings Summary
# # ============================================================
# latest_date = dates[-1]
# latest_prices = price_map.loc[latest_date]
# hold_df = pd.DataFrame([
#     {"Stock": s,
#      "Quantity": holdings[s],
#      "Last_Price": latest_prices[s],
#      "Market_Value": holdings[s] * latest_prices[s]}
#     for s in holdings.keys() if holdings[s] > 0
# ])
# hold_df["Weight_%"] = (hold_df["Market_Value"] / hold_df["Market_Value"].sum()) * 100
# hold_df = hold_df.sort_values("Market_Value", ascending=False).reset_index(drop=True)

# print("\n📊 Latest Portfolio Holdings (as of {})".format(latest_date.date()))
# display(hold_df.round(2))

# # ============================================================
# # 💾 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Portfolio_Dashboard.xlsx") as writer:
#     df_portfolio.to_excel(writer, sheet_name="Portfolio_TimeSeries")
#     hold_df.to_excel(writer, sheet_name="Latest_Holdings", index=False)
# print("✅ Exported → Hybrid_Portfolio_Dashboard.xlsx")

# # ============================================================
# # 📋 Latest Holdings Summary
# # ============================================================
# latest_date = dates[-1]
# latest_prices = price_map.loc[latest_date]
# hold_df = pd.DataFrame([
#     {"Stock": s,
#      "Quantity": holdings[s],
#      "Last_Price": latest_prices[s],
#      "Market_Value": holdings[s] * latest_prices[s]}
#     for s in holdings.keys() if holdings[s] > 0
# ])
# hold_df["Weight_%"] = (hold_df["Market_Value"] / hold_df["Market_Value"].sum()) * 100
# hold_df = hold_df.sort_values("Market_Value", ascending=False).reset_index(drop=True)

# # --- 🧾 Portfolio Summary (Cash vs Equity) ---
# latest_cash = cash
# latest_equity_value = hold_df["Market_Value"].sum()
# total_portfolio_value = latest_cash + latest_equity_value
# cash_ratio = (latest_cash / total_portfolio_value) * 100
# equity_ratio = (latest_equity_value / total_portfolio_value) * 100

# print("\n📊 Latest Portfolio Holdings (as of {})".format(latest_date.date()))
# display(hold_df.round(2))

# print("\n💰 Portfolio Allocation Summary:")
# print(f"   • Total Portfolio Value : {total_portfolio_value:,.2f} THB")
# print(f"   •   ├─ Cash             : {latest_cash:,.2f} THB ({cash_ratio:.2f}%)")
# print(f"   •   └─ Equity Holdings  : {latest_equity_value:,.2f} THB ({equity_ratio:.2f}%)")

# # ============================================================
# # 💾 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Portfolio_Dashboard.xlsx") as writer:
#     df_portfolio.to_excel(writer, sheet_name="Portfolio_TimeSeries")
#     hold_df.to_excel(writer, sheet_name="Latest_Holdings", index=False)

# summary_df = pd.DataFrame({
#     "Total_Value": [total_portfolio_value],
#     "Cash": [latest_cash],
#     "Equity_Value": [latest_equity_value],
#     "Cash_%": [cash_ratio],
#     "Equity_%": [equity_ratio]
# })
# summary_df.to_excel(writer, sheet_name="Allocation_Summary", index=False)

# print("\n✅ Exported → Hybrid_Portfolio_Dashboard.xlsx")



In [ ]:
# # ============================================================
# # 📊 BLOCK 5.4 — Regime-wise PyFolio Performance Summary (Refined)
# # ============================================================

# import numpy as np
# import pandas as pd

# # --- Ensure Regime column exists ---
# if "Regime" not in df_portfolio.columns:
#     df_reg = df_all[["Date", "Regime"]].drop_duplicates(subset="Date", keep="last")
#     df_reg["Date"] = pd.to_datetime(df_reg["Date"])
#     df_portfolio = df_portfolio.reset_index().merge(df_reg, on="Date", how="left").set_index("Date")

# # ============================================================
# # 🧮 Helper Functions
# # ============================================================
# def CAGR(series, freq=252):
#     if len(series) < 2:
#         return np.nan
#     if series.iloc[0] != 1:  # normalize
#         series = series / series.iloc[0]
#     total_ret = series.iloc[-1] - 1
#     years = len(series) / freq
#     return (1 + total_ret) ** (1 / years) - 1

# def max_drawdown(cumret):
#     roll_max = np.maximum.accumulate(cumret)
#     drawdown = (cumret - roll_max) / roll_max
#     return abs(drawdown.min())

# def win_rate(returns):
#     return (returns > 0).mean()

# def sortino_ratio(returns, rf=0.0):
#     downside = returns[returns < 0].std()
#     if downside == 0 or np.isnan(downside):
#         return np.nan
#     return (returns.mean() - rf) / downside

# def calmar_ratio(cagr, mdd):
#     if mdd == 0 or np.isnan(mdd):
#         return np.nan
#     return cagr / mdd

# # ============================================================
# # 🧾 Compute Metrics per Regime
# # ============================================================
# regime_stats = []

# for regime, df_reg in df_portfolio.groupby("Regime"):
#     if df_reg.empty:
#         continue

#     df_reg = df_reg.copy().sort_index()
#     cum = (1 + df_reg["Daily_Return"].fillna(0)).cumprod()

#     # --- Monthly compounded returns ---
#     monthly_ret = (1 + df_reg["Daily_Return"]).resample("M").prod() - 1
#     monthly_vol = df_reg["Daily_Return"].resample("M").std() * np.sqrt(21)

#     cagr = CAGR(cum)
#     mdd = max_drawdown(cum)
#     wr = win_rate(df_reg["Daily_Return"])
#     sortino = sortino_ratio(df_reg["Daily_Return"])
#     calmar = calmar_ratio(cagr, mdd)

#     regime_stats.append({
#         "Regime": regime,
#         "CAGR_%": cagr * 100,
#         "Max_Drawdown_%": mdd * 100,
#         "Win_Rate_%": wr * 100,
#         "Monthly_Return_%": np.nanmean(monthly_ret) * 100,
#         # "Monthly_Volatility_%": np.nanmean(monthly_vol) * 100,
#         "Sortino": sortino,
#         "Calmar": calmar
#     })

# # ============================================================
# # 📋 Summary Table
# # ============================================================
# df_regime_stats = pd.DataFrame(regime_stats).round(3)
# cols = ["Regime", "CAGR_%", "Max_Drawdown_%", "Win_Rate_%",
#         "Monthly_Return_%", "Sortino", "Calmar"]
# df_regime_stats = df_regime_stats[cols]

# print("\n📈 Regime-wise Performance Summary (PyFolio Style)")
# display(df_regime_stats)

# # ============================================================
# # 💾 Export
# # ============================================================
# df_regime_stats.to_excel("Hybrid_PyFolio_Regime_Summary.xlsx", index=False)
# print("✅ Exported → Hybrid_PyFolio_Regime_Summary.xlsx")


In [ ]:
# # ============================================================
# # 📈 BLOCK 5.5 — Regime-wise Equity Curve & Performance Visualization
# # ============================================================

# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import pandas as pd

# # --- Ensure regime alignment with portfolio ---
# if "Regime" not in df_portfolio.columns:
#     df_reg = df_all[["Date", "Regime"]].drop_duplicates(subset="Date", keep="last")
#     df_reg["Date"] = pd.to_datetime(df_reg["Date"])
#     df_portfolio = df_portfolio.reset_index().merge(df_reg, on="Date", how="left").set_index("Date")

# # --- สร้าง cumulative return ในแต่ละ regime ---
# regime_curves = {}
# for regime, df_reg in df_portfolio.groupby("Regime"):
#     if df_reg.empty:
#         continue
#     cum = (1 + df_reg["Daily_Return"].fillna(0)).cumprod() - 1
#     regime_curves[regime] = cum

# # ============================================================
# # 🧾 Summary Table (รวมทุก Regime)
# # ============================================================
# regime_summary = []
# for regime, cum in regime_curves.items():
#     ann_ret = (1 + cum.iloc[-1]) ** (252 / len(cum)) - 1
#     mdd = abs(((1 + cum).cummax() - (1 + cum)) / (1 + cum).cummax()).max()
#     sharpe = df_portfolio[df_portfolio["Regime"] == regime]["Daily_Return"].mean() / df_portfolio[df_portfolio["Regime"] == regime]["Daily_Return"].std()
#     regime_summary.append({
#         "Regime": regime,
#         "Final_Return_%": cum.iloc[-1] * 100,
#         "Annualized_Return_%": ann_ret * 100,
#         "Max_Drawdown_%": mdd * 100,
#         "Sharpe": sharpe
#     })

# df_regime_equity = pd.DataFrame(regime_summary).round(3)
# print("\n📊 Regime-wise Equity Curve Summary")
# display(df_regime_equity)

# # ============================================================
# # 🎨 Visualization — Overlayed Regime Equity Curves
# # ============================================================
# palette = {"Expansion": "#2ecc71", "Neutral": "#f1c40f", "Contraction": "#e74c3c"}

# plt.figure(figsize=(12, 6))
# for regime, cum in regime_curves.items():
#     plt.plot(cum.index, cum, label=regime, lw=2, color=palette.get(regime, "gray"))

# plt.axhline(0, color="black", lw=1)
# plt.title("Regime-wise Equity Curves (Cumulative Return)", fontsize=14, fontweight="bold")
# plt.ylabel("Cumulative Return")
# plt.legend(title="Regime")
# plt.grid(True, linestyle="--", alpha=0.7)
# plt.tight_layout()
# plt.show()

# # ============================================================
# # 📦 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Regime_Equity_Summary.xlsx") as writer:
#     df_regime_equity.to_excel(writer, sheet_name="Regime_Equity_Summary", index=False)
#     for regime, cum in regime_curves.items():
#         cum_df = cum.reset_index()
#         cum_df.columns = ["Date", f"{regime}_Cumulative_Return"]
#         cum_df.to_excel(writer, sheet_name=f"{regime}_Curve", index=False)

# print("✅ Exported → Hybrid_Regime_Equity_Summary.xlsx")


In [ ]:
# # ============================================================
# # 📊 BLOCK 5.6 — Stock-wise Profit/Loss per Regime (Linked + Ranking Table)
# # ============================================================

# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# import pandas as pd

# # --- ตรวจสอบสัญญาณ FinalSignal ---
# df_price_sig = df_all.copy()
# if "FinalSignal" not in df_price_sig.columns:
#     print("⚠️ Auto-generating FinalSignal from Score / Pred_dLogclose ...")
#     if "Score" in df_price_sig.columns:
#         df_price_sig["FinalSignal"] = np.where(abs(df_price_sig["Score"]) < 0.05, 0, np.sign(df_price_sig["Score"]))
#     elif "Pred_dLogclose" in df_price_sig.columns:
#         df_price_sig["FinalSignal"] = np.sign(df_price_sig["Pred_dLogclose"])
#     else:
#         raise KeyError("No Score or Pred_dLogclose found to create FinalSignal.")

# # --- ตรวจสอบ Regime ---
# if "Regime" not in df_price_sig.columns:
#     df_reg = df_portfolio.reset_index()[["Date", "Regime"]].drop_duplicates()
#     df_price_sig = df_price_sig.merge(df_reg, on="Date", how="left")

# # --- เพิ่ม Weight จากพอร์ตจริง (ถือหุ้นมากน้อยแค่ไหน) ---
# df_portfolio_reset = df_portfolio.reset_index()[["Date", "Total_Portfolio", "Holdings_Value", "Cash"]].copy()
# df_price_sig = df_price_sig.merge(df_portfolio_reset, on="Date", how="left")

# # สมมติให้ Weight ต่อหุ้น (position fraction) สัดส่วนจากกลยุทธ์จริง (3%)
# df_price_sig["Weight"] = 0.03 * (df_price_sig["FinalSignal"].shift(1) != 0).astype(int)

# # --- คำนวณ Return ของแต่ละหุ้นตามน้ำหนักจริง ---
# df_price_sig["Return"] = df_price_sig.groupby("Stock")["Close"].pct_change()
# df_price_sig["Strategy_Return"] = df_price_sig["Return"] * df_price_sig["Weight"] * df_price_sig["FinalSignal"].shift(1)

# # ============================================================
# # 🧮 คำนวณผลตอบแทนเฉลี่ยต่อหุ้นแยกตาม Regime
# # ============================================================
# regime_profit = (
#     df_price_sig.groupby(["Regime", "Stock"])["Strategy_Return"]
#     .mean()
#     .reset_index()
# )

# # Annualize (ประมาณ 252 วัน)
# regime_profit["Avg_Ann_Return_%"] = regime_profit["Strategy_Return"] * 252 * 100

# # ============================================================
# # 🎨 Visualization + Ranking
# # ============================================================
# regime_order = ["Expansion", "Neutral", "Contraction"]
# palette = {"Expansion": "#2ecc71", "Neutral": "#f1c40f", "Contraction": "#e74c3c"}

# ranking_tables = []

# for regime in regime_order:
#     sub = regime_profit[regime_profit["Regime"] == regime]
#     if sub.empty:
#         continue

#     sub_sorted = sub.sort_values("Avg_Ann_Return_%", ascending=False)

#     # --- Plot Bar Chart ---
#     plt.figure(figsize=(10, 5))
#     sns.barplot(
#         data=sub_sorted,
#         x="Stock", y="Avg_Ann_Return_%", color=palette[regime]
#     )
#     plt.axhline(0, color="black", linewidth=1)
#     plt.title(f"Strategy Profit/Loss by Stock — {regime} Regime", fontsize=14, fontweight='bold')
#     plt.ylabel("Average Annualized Return (%)")
#     plt.xlabel("Stock")
#     plt.xticks(rotation=45, ha="right")
#     plt.tight_layout()
#     plt.show()

#     # --- Top & Bottom 3 Ranking ---
#     top3 = sub_sorted.head(3).assign(Rank="Top 3")
#     worst3 = sub_sorted.tail(3).assign(Rank="Bottom 3")
#     ranking_tables.append(pd.concat([top3, worst3], ignore_index=True))

# # ============================================================
# # 📋 Combine Ranking Table
# # ============================================================
# df_ranking = pd.concat(ranking_tables, ignore_index=True)[
#     ["Regime", "Rank", "Stock", "Avg_Ann_Return_%"]
# ]
# df_ranking = df_ranking.sort_values(["Regime", "Rank", "Avg_Ann_Return_%"], ascending=[True, True, False])

# print("\n🏆 Top 3 / Bottom 3 Stocks by Regime (based on Avg Annualized Return %)")
# display(df_ranking.round(2))

# # ============================================================
# # 💾 Export
# # ============================================================
# with pd.ExcelWriter("Hybrid_Regime_StockPerformance_Linked.xlsx") as writer:
#     regime_profit.to_excel(writer, sheet_name="Stock_Returns", index=False)
#     df_ranking.to_excel(writer, sheet_name="Ranking_TopBottom", index=False)

# print("✅ Exported → Hybrid_Regime_StockPerformance_Linked.xlsx")


# Next period signal

In [ ]:
!pip install minisom

In [ ]:
# # ============================================================
# # 🧭 BLOCK 5.7 — Next-Month Signal + Technical + SOM Suggestion
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # ============================================================
# # 1️⃣ ตรวจสอบหรือสร้าง Technical Summary
# # ============================================================

# try:
#     df_technical_latest
#     print("✅ Using existing df_technical_latest.")
# except NameError:
#     print("⚠️ df_technical_latest not found — auto-generating mock version...")
#     df_technical_latest = pd.DataFrame({
#         "Stock": df_forecast_summary["Stock"].unique(),
#         "TechScore": np.random.uniform(-1, 1, len(df_forecast_summary["Stock"].unique())),
#         "MACD": np.random.uniform(-1, 1, len(df_forecast_summary["Stock"].unique())),
#         "RSI_14": np.random.uniform(30, 70, len(df_forecast_summary["Stock"].unique())),
#         "Pattern_Confidence": np.random.uniform(0, 1, len(df_forecast_summary["Stock"].unique())),
#         "Wave_Direction": np.random.choice(["Up", "Down"], len(df_forecast_summary["Stock"].unique())),
#         "Wave_Strength": np.random.uniform(0, 1, len(df_forecast_summary["Stock"].unique()))
#     })
#     print("✅ df_technical_latest mock created.")

# # ============================================================
# # 2️⃣ รวมข้อมูล Macro + Technical → Hybrid Signal
# # ============================================================

# df_signal = df_forecast_summary.copy()
# df_signal = df_signal.merge(df_technical_latest, on="Stock", how="left")

# # --- Macro bias ---
# bias = np.sign(df_signal["Pred_dLogclose"])

# # --- Technical confirmation ---
# techconfirm = (
#     0.3 * df_signal["TechScore"].fillna(0)
#     + 0.2 * df_signal["MACD"].fillna(0)
#     + 0.1 * ((60 - abs(df_signal["RSI_14"].fillna(50) - 50)) / 50)
# )

# # --- Pattern + Wave alignment ---
# wave_align = np.where(df_signal["Wave_Direction"] == np.where(bias>0, "Up", "Down"), 1, -1)
# patternwave = (
#     0.2 * df_signal["Pattern_Confidence"].fillna(0)
#     + 0.2 * df_signal["Wave_Strength"].fillna(0) * wave_align
# )

# # --- Hybrid score ---
# df_signal["Hybrid_Score"] = 0.5*bias + 0.3*techconfirm + 0.2*patternwave

# # --- Expected Return ---
# df_signal["Expected_Return_%"] = df_signal["Pred_dLogclose"] * 100

# # ============================================================
# # 3️⃣ Self-Organizing Map (SOM-style) Decision Zone
# # ============================================================

# def som_zone(row):
#     if row["Hybrid_Score"] > 0.1 and row["Pred_dLogclose"] > 0:
#         return "BUY"
#     elif -0.1 <= row["Hybrid_Score"] <= 0.1:
#         return "HOLD"
#     elif row["Hybrid_Score"] < -0.1 and row["Pred_dLogclose"] < 0:
#         return "SELL"
#     else:
#         return "HOLD"

# df_signal["SOM_Action"] = df_signal.apply(som_zone, axis=1)

# # ============================================================
# # 4️⃣ Sector Summary Table
# # ============================================================

# sector_summary = (
#     df_signal.groupby("Sector")
#     .agg(
#         N_Stocks=("Stock", "count"),
#         Avg_Hybrid_Score=("Hybrid_Score", "mean"),
#         Avg_Exp_Return=("Expected_Return_%", "mean"),
#         N_BUY=("SOM_Action", lambda x: (x == "BUY").sum()),
#         N_HOLD=("SOM_Action", lambda x: (x == "HOLD").sum()),
#         N_SELL=("SOM_Action", lambda x: (x == "SELL").sum())
#     )
#     .reset_index()
# )

# sector_summary["%BUY"] = sector_summary["N_BUY"] / sector_summary["N_Stocks"] * 100
# sector_summary["%SELL"] = sector_summary["N_SELL"] / sector_summary["N_Stocks"] * 100

# def suggest_position(row):
#     if row["%BUY"] > 60 and row["Avg_Hybrid_Score"] > 0.1:
#         return "Overweight"
#     elif row["%SELL"] > 60 and row["Avg_Hybrid_Score"] < -0.1:
#         return "Underweight"
#     else:
#         return "Neutral"

# sector_summary["Position_Suggestion"] = sector_summary.apply(suggest_position, axis=1)
# sector_summary = sector_summary.sort_values("Avg_Hybrid_Score", ascending=False).reset_index(drop=True)

# # ============================================================
# # 5️⃣ แสดงผลลัพธ์รวม
# # ============================================================

# print("\n=== 📈 Next-Month Hybrid Trading Signals (Stock Level) ===")
# display(
#     df_signal[[
#         "Stock","Sector","Pred_dLogclose","Expected_Return_%",
#         "TechScore","Hybrid_Score","SOM_Action"
#     ]].sort_values("Hybrid_Score", ascending=False).round(3)
# )

# print("\n=== 💼 Sector Position Suggestion Table ===")
# display(
#     sector_summary[[
#         "Sector","N_Stocks","N_BUY","N_HOLD","N_SELL",
#         "%BUY","%SELL","Avg_Hybrid_Score","Avg_Exp_Return","Position_Suggestion"
#     ]].round(3)
# )

# # ============================================================
# # 🎨 SOM Visualization — Stock Labels + Hybrid Score
# # ============================================================

# from minisom import MiniSom
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # --- เตรียม features สำหรับ SOM ---
# features = ["Pred_dLogclose", "TechScore", "MACD", "RSI_14", "Pattern_Confidence", "Wave_Strength"]
# X = StandardScaler().fit_transform(df_signal[features].fillna(0))

# # --- Train SOM ---
# som_x, som_y = 8, 8
# som = MiniSom(x=som_x, y=som_y, input_len=X.shape[1],
#               sigma=1.0, learning_rate=0.5,
#               neighborhood_function='gaussian', random_seed=42)
# som.random_weights_init(X)
# som.train_random(X, 1000, verbose=False)

# # --- คำนวณตำแหน่ง node ของแต่ละหุ้น ---
# win_map = np.array([som.winner(x) for x in X])
# df_signal["SOM_X"] = win_map[:,0]
# df_signal["SOM_Y"] = win_map[:,1]

# # --- สีของ Action ---
# color_map = {"BUY": "#2ecc71", "HOLD": "#f1c40f", "SELL": "#e74c3c"}

# # --- สร้างค่าเฉลี่ย Hybrid Score ต่อ node ---
# node_mean = (
#     df_signal.groupby(["SOM_X","SOM_Y"])["Hybrid_Score"]
#     .mean().unstack().fillna(0)
# )

# # ============================================================
# # 🧠 BLOCK 5.7 — Enhanced SOM Cluster Map (Hybrid + Technical)
# # ============================================================

# from minisom import MiniSom
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # --- ⚙️ เตรียม features สำหรับ SOM ---
# df_signal["RSI_14_scaled"] = (df_signal["RSI_14"] - 50) / 25           # RSI 25–75 → -1 ถึง +1
# df_signal["Hybrid_Score_Scaled"] = df_signal["Hybrid_Score"] * 2.5     # ขยาย dynamic range

# features = [
#     "Pred_dLogclose", "TechScore", "MACD",
#     "RSI_14_scaled", "Pattern_Confidence", "Wave_Strength"
# ]
# X = StandardScaler().fit_transform(df_signal[features].fillna(0))

# # --- 🔄 Train SOM ---
# som_x, som_y = 8, 8
# som = MiniSom(
#     x=som_x, y=som_y, input_len=X.shape[1],
#     sigma=1.0, learning_rate=0.5,
#     neighborhood_function='gaussian', random_seed=42
# )
# som.random_weights_init(X)
# som.train_random(X, 1000, verbose=False)

# # --- 📍 หา node ของแต่ละหุ้น ---
# win_map = np.array([som.winner(x) for x in X])
# df_signal["SOM_X"] = win_map[:, 0]
# df_signal["SOM_Y"] = win_map[:, 1]

# # --- 🎨 สี Action ---
# color_map = {"BUY": "#2ecc71", "HOLD": "#f1c40f", "SELL": "#e74c3c"}

# # --- 🧮 คำนวณค่าเฉลี่ย Hybrid Score ต่อ node ---
# node_mean = (
#     df_signal.groupby(["SOM_X", "SOM_Y"])["Hybrid_Score_Scaled"]
#     .mean().unstack().fillna(0)
# )

# # ============================================================
# # 🎨 Visualization — SOM Heatmap + Stock Labels + Score
# # ============================================================
# plt.figure(figsize=(9, 8))
# plt.title("🧠 SOM Cluster Map — Hybrid (Macro + Technical)", fontsize=14, fontweight="bold")

# # --- Heatmap พื้นหลัง (Dynamic scale) ---
# vmin, vmax = node_mean.min().min(), node_mean.max().max()
# sns.heatmap(
#     node_mean,
#     cmap="RdYlGn", vmin=vmin, vmax=vmax,
#     cbar_kws={"label": "Mean Hybrid Score"},
#     square=True, linewidths=0.5, alpha=0.25
# )

# # --- Plot หุ้นแต่ละตัว (ชื่อ + Score + สี Action) ---
# for _, row in df_signal.iterrows():
#     x, y = row["SOM_X"] + 0.5, row["SOM_Y"] + 0.5
#     color = color_map.get(row["SOM_Action"], "gray")
#     plt.scatter(x, y, s=200, color=color, edgecolor="black", alpha=0.9, zorder=3)
#     plt.text(
#         x, y,
#         f"{row['Stock']} ({row['Hybrid_Score']:+.2f})",
#         fontsize=8, ha="center", va="center",
#         color="black", fontweight="bold"
#     )

# plt.xlabel("SOM X (Cluster Column)")
# plt.ylabel("SOM Y (Cluster Row)")
# plt.grid(False)
# plt.tight_layout()
# plt.show()




# # ============================================================
# # 7️⃣ Export Excel
# # ============================================================

# with pd.ExcelWriter("Hybrid_SOM_Position_Suggestion.xlsx") as writer:
#     df_signal.to_excel(writer, sheet_name="Stock_Signals", index=False)
#     sector_summary.to_excel(writer, sheet_name="Sector_Suggestion", index=False)

# print("✅ Exported → Hybrid_SOM_Position_Suggestion.xlsx")


# Dynamic portfolio = Quantitative deterministic (Sharpe-weighted + Signal flip)

=> เร็วกว่า, ใช้แนว Bellman update เบื้องต้นโดยไม่ต้อง train RL model

In [ ]:
# import pandas as pd
# import numpy as np

# # ============================================================
# # ⚙️ BLOCK 6 (Revised): Long-Only RL Deterministic Allocation (All outputs in %)
# # ============================================================

# def adaptive_portfolio_rl_longonly(evaluation_results, window_corr=12, window_sharpe=6):
#     all_dates = sorted(list(set().union(*[df.index for df in evaluation_results.values()])))
#     portfolio_returns = pd.DataFrame(index=all_dates)

#     adj_signals = {}
#     sh_ratio = {}

#     # --- Step 1: Compute adjusted signal and Sharpe ---
#     for stock, df_eval in evaluation_results.items():
#         df = df_eval.copy()
#         df["Signal_raw"] = np.sign(df["Pred_dLogclose"])
#         df["Corr_Pred_Actual"] = df["Pred_dLogclose"].rolling(window_corr).corr(df["Actual_dLogclose"])
#         df["Signal_adj"] = np.where(df["Corr_Pred_Actual"] < 0, -df["Signal_raw"], df["Signal_raw"])
#         df["Return"] = df["Actual_dLogclose"]
#         df["Sharpe"] = (
#             df["Return"].rolling(window_sharpe).mean() / df["Return"].rolling(window_sharpe).std()
#         )
#         adj_signals[stock] = df["Signal_adj"]
#         sh_ratio[stock] = df["Sharpe"]
#         portfolio_returns[stock] = df["Return"]

#     weights_df = pd.DataFrame(index=all_dates, columns=list(evaluation_results.keys()))
#     port_ret_series = []

#     # --- Step 2: Compute weights (non-negative long-only) ---
#     for t, date in enumerate(all_dates):
#         w_list, r_list = [], []

#         for stock in evaluation_results.keys():
#             sig = adj_signals[stock].reindex(all_dates).iloc[t]
#             shrp = sh_ratio[stock].reindex(all_dates).iloc[t]
#             ret = portfolio_returns[stock].reindex(all_dates).iloc[t]

#             if np.isnan(sig) or np.isnan(shrp) or np.isnan(ret):
#                 continue

#             w_val = sig * abs(shrp)
#             w_list.append((stock, w_val))
#             r_list.append((stock, ret))

#         if not w_list:
#             port_ret_series.append(0.0)
#             continue

#         w_df = pd.DataFrame(w_list, columns=["Stock", "w"])
#         w_df["w"] = np.clip(w_df["w"], 0, None)

#         if w_df["w"].sum() > 0:
#             w_df["w"] /= w_df["w"].sum()
#         else:
#             w_df["w"] = 1.0 / len(w_df)

#         for s in w_df["Stock"]:
#             weights_df.loc[date, s] = w_df.loc[w_df["Stock"] == s, "w"].iloc[0]

#         r_df = pd.DataFrame(r_list, columns=["Stock", "Return"])
#         merged = pd.merge(w_df, r_df, on="Stock")
#         port_ret = np.sum(merged["w"] * merged["Return"])
#         port_ret_series.append(port_ret)

#     # --- Step 3: Portfolio performance summary ---
#     portfolio_returns["Portfolio_Return"] = port_ret_series
#     portfolio_returns["Cumulative_Portfolio"] = np.exp(portfolio_returns["Portfolio_Return"].cumsum()) - 1

#     # ✅ Convert ALL numeric columns to percentage
#     portfolio_returns_pct = portfolio_returns.copy() * 100
#     portfolio_returns_pct.columns = [col + "(%)" for col in portfolio_returns.columns]

#     # Compute summary statistics in %
#     avg_r = portfolio_returns["Portfolio_Return"].mean() * 100
#     std_r = portfolio_returns["Portfolio_Return"].std() * 100
#     sharpe_r = avg_r / std_r if std_r > 0 else np.nan
#     cum_r = portfolio_returns["Cumulative_Portfolio"].iloc[-1] * 100

#     summary = pd.DataFrame([{
#         "CumReturn_%": cum_r,
#         "AvgMonthlyReturn_%": avg_r,
#         "Volatility_%": std_r,
#         "Sharpe": sharpe_r
#     }])

#     weights_df = weights_df * 100  # export weights in %
#     return portfolio_returns_pct, weights_df, summary


# # ============================================================
# # 🚀 Run Adaptive Allocation (Long-only, % export)
# # ============================================================

# portfolio_rl, weights_rl, summary_rl = adaptive_portfolio_rl_longonly(evaluation_results)

# with pd.ExcelWriter("ECM_RL_AdaptivePortfolio_LongOnly_Percent.xlsx") as writer:
#     summary_rl.to_excel(writer, sheet_name="Portfolio_Summary", index=False)
#     weights_rl.to_excel(writer, sheet_name="Dynamic_Weights(%)")
#     portfolio_rl.to_excel(writer, sheet_name="Portfolio_Returns(%)")

# print("\n✅ Exported → ECM_RL_AdaptivePortfolio_LongOnly_Percent.xlsx")

# print("\n=== 💼 Portfolio Summary ===")
# display(summary_rl)

# print("\n=== 📊 Dynamic Weights (last 5 dates, %) ===")
# display(weights_rl.tail(5))

# print("\n=== 📈 Portfolio Return Curve (last 10, in %) ===")
# display(portfolio_rl.tail(10))


In [ ]:
# # ============================================================
# # 🎨 BLOCK 7: Visualization of Dynamic Portfolio Allocation (By Sector)
# # ============================================================
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.style.use("seaborn-v0_8-whitegrid")
# sns.set(font_scale=0.9)

# # ============================================================
# # 🔹 STEP 1: Prepare sector mapping
# # ============================================================

# # สร้าง DataFrame mapping จาก evaluation_results (หรือ summary เดิม)
# sector_map = {}
# for stock, df_eval in evaluation_results.items():
#     sector = df_eval["Sector"].iloc[0]
#     sector_map[stock] = sector

# # ตรวจสอบว่ามี mapping ครบไหม
# print("📊 Sector mapping:", sector_map)

# # ============================================================
# # 🔹 STEP 2: Convert weights_rl เป็นราย Sector
# # ============================================================

# weights_sector = pd.DataFrame(index=weights_rl.index)

# for stock in weights_rl.columns:
#     sector = sector_map.get(stock, "Unknown")
#     if sector not in weights_sector.columns:
#         weights_sector[sector] = 0.0
#     weights_sector[sector] += pd.to_numeric(weights_rl[stock], errors="coerce").fillna(0)

# # Normalize ให้ sum ต่อแถว = 100%
# weights_sector = weights_sector.div(weights_sector.sum(axis=1), axis=0) * 100

# print("\n✅ Sector-level weights created:")
# display(weights_sector.tail(3))

# # ============================================================
# # 🔹 STEP 3: Heatmap by Sector
# # ============================================================

# fig, ax = plt.subplots(figsize=(10, 5))
# sns.heatmap(weights_sector.T, cmap="YlOrRd", cbar_kws={'label': 'Weight (%)'}, ax=ax)
# ax.set_title("Dynamic Portfolio Allocation by Sector (%)", fontsize=14, weight="bold")
# ax.set_xlabel("Date")
# ax.set_ylabel("Sector")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

# # ============================================================
# # 🔹 STEP 4: Area Chart by Sector
# # ============================================================

# fig, ax = plt.subplots(figsize=(12, 6))
# weights_sector.plot.area(ax=ax, stacked=True, alpha=0.8)
# ax.set_title("Portfolio Allocation Composition by Sector (%)", fontsize=14, weight="bold")
# ax.set_ylabel("Weight (%)")
# ax.set_xlabel("Date")
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Sector")
# plt.tight_layout()
# plt.show()

# # ============================================================
# # 🔹 STEP 5: Pie Chart (ล่าสุด)
# # ============================================================

# last_date = weights_sector.index[-1]
# last_weights = weights_sector.loc[last_date].dropna()
# fig, ax = plt.subplots(figsize=(6, 6))
# ax.pie(last_weights, labels=last_weights.index, autopct='%1.1f%%', startangle=90)
# ax.set_title(f"Sector Allocation on {last_date.strftime('%Y-%m')}", fontsize=13, weight="bold")
# plt.tight_layout()
# plt.show()


# Next-Month Trading Plan after Bellman theory  => Exclude it

In [ ]:
# # ============================================================
# # 📈 BLOCK: Sharpe-weighted + Signal Flip (with Flip Info + Comments)
# # ============================================================

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # ===== 1️⃣ Sharpe-weighted Portfolio Allocation =====
# df_perf = df_strategy_summary.copy()
# df_perf["Sharpe_Positive"] = np.where(df_perf["Sharpe_Ratio"] > 0, df_perf["Sharpe_Ratio"], 0)
# total_sharpe = df_perf["Sharpe_Positive"].sum()
# df_perf["Weight_Sharpe"] = np.where(total_sharpe > 0, df_perf["Sharpe_Positive"] / total_sharpe, 0)

# # ===== 2️⃣ สัญญาณพยากรณ์ล่าสุด =====
# df_signal_next = df_forecast_summary.copy()
# df_signal_next = df_signal_next.merge(df_perf[["Stock", "Sharpe_Ratio", "Weight_Sharpe"]], on="Stock", how="left")

# # ===== 3️⃣ สร้างสัญญาณและตรวจจับการ Flip =====
# df_signal_next["Raw_Signal"] = np.where(df_signal_next["Pred_dLogclose"] > 0, "BUY", "SELL")
# df_signal_next["Flip_Flag"] = np.where(df_signal_next["Sharpe_Ratio"] < 0, True, False)
# df_signal_next["Final_Signal"] = np.where(
#     df_signal_next["Flip_Flag"],
#     np.where(df_signal_next["Raw_Signal"] == "BUY", "SELL", "BUY"),
#     df_signal_next["Raw_Signal"]
# )

# # ===== 4️⃣ เพิ่ม Comment / Interpretation =====
# df_signal_next["Flip_Status"] = np.where(df_signal_next["Flip_Flag"], "🔁 Flipped", "✅ Normal")
# df_signal_next["Comment"] = np.where(
#     df_signal_next["Flip_Flag"],
#     "Model weak (Sharpe < 0) → Contrarian signal",
#     "Model consistent (Sharpe > 0)"
# )

# df_signal_next["Expected_Return_%"] = df_signal_next["Pred_dLogclose"] * 100
# df_signal_next["Weighted_Exp_Return_%"] = df_signal_next["Expected_Return_%"] * df_signal_next["Weight_Sharpe"]

# # ===== 5️⃣ Export & Display =====
# cols_show = [
#     "Stock", "Sector", "Sharpe_Ratio", "Weight_Sharpe",
#     "Pred_dLogclose", "Expected_Return_%",
#     "Raw_Signal", "Final_Signal", "Flip_Status", "Comment"
# ]

# # 🔹 เรียงตาม Expected_Return_% จากมากไปน้อย
# df_signal_next = df_signal_next[cols_show].sort_values("Expected_Return_%", ascending=False)

# df_signal_next.to_excel("NextMonth_SignalPlan_SharpeWeighted_FlipInfo.xlsx", index=False)
# print("✅ Exported → NextMonth_SignalPlan_SharpeWeighted_FlipInfo.xlsx")

# # ===== 6️⃣ PRINT TABLE =====
# print("\n=== 📈 Next-Month Trading Signals (Sharpe-weighted + Flip Info, Sorted by Expected Return) ===")
# display(df_signal_next)

# # ===== 7️⃣ Visualization =====
# plt.figure(figsize=(10,6))
# sns.barplot(
#     data=df_signal_next,
#     x="Stock",
#     y="Expected_Return_%",
#     hue="Final_Signal",
#     dodge=False,
#     order=df_signal_next["Stock"]  # ใช้ลำดับเดียวกับที่เรียงในตาราง
# )
# plt.axhline(0, color="black", lw=1)
# plt.title("Next-Month Expected Return by Stock (Sharpe-weighted + Flip)", weight="bold")
# plt.xticks(rotation=45, ha='right')
# plt.show()


1. สร้างโมเดล Deep Learning (Technical Side) [งานหลักของคุณ]ส่วนนี้คือการสร้าง "ตา" ข้างที่สองให้ AI (ข้างแรกคือ ARDL ของเพื่อน)สร้างโมเดล LSTM หรือ GRU: เขียน Code แยกออกมาเพื่อรับกราฟราคาและ Technical Indicators (RSI, MACD) 1กำหนดค่า Output ($b_t$): ต้องเทรนให้โมเดลทำนายออกมาเป็น ความน่าจะเป็น (Probability) ว่าเดือนถัดไปตลาดจะ Bull, Bear หรือ Neutral 2สิ่งที่ต้องระวัง: อย่าเผลอทำนายเป็น "ราคาปิด" (Price Regression) เพราะใน Proposal ระบุชัดเจนว่าต้องเป็นสัญญาณความน่าจะเป็น (Classification/Probability) เพื่อส่งต่อให้ RL2. สร้างระบบ Reinforcement Learning (The Brain) [งานหลักของคุณ]นี่คือ "สมอง" ของระบบที่ต้องเอาข้อมูลทุกอย่างมารวมกันออกแบบ Environment: เขียน Class Env ใน Python ให้รับข้อมูล 2 ทาง:Macro Signals ($m_t$): รับค่าที่เพื่อนคุณทำเสร็จแล้ว (จาก ARDL)Technical Signals ($b_t$): รับค่าจากโมเดล Deep Learning ที่คุณทำในข้อ 1รวม State ($S_t$): จับสองค่าข้างบนมารวมกันเป็น State เดียว 3Train PPO Agent: รันการเรียนรู้เพื่อให้ AI ตัดสินใจ (Buy/Hold/Sell) โดยใช้ Reward Function ที่หักลบค่า Drawdown 43. การเชื่อมต่อข้อมูลกับเพื่อน (Data Interface)เพื่อให้งานประกอบกันได้ง่าย คุณต้องตกลงกับเพื่อนให้ชัดเจนครับ:Request: บอกเพื่อนว่า "ขอไฟล์ Excel/CSV ที่มีคอลัมน์ Predicted Return หรือ Macro Signal จาก ARDL รายเดือน มาให้ฉันหน่อย"Action: คุณจะเอาคอลัมน์นั้นมาเป็น Input ส่วน $m_t$ ใน Code RL ของคุณครับ4. Backtesting & Reportนำ AI ที่เทรนเสร็จแล้ว มาเทรดจำลองกับข้อมูลปี 2025 และสรุปผลวัดค่า Sharpe Ratio, Drawdown เทียบกับตลาด 5สรุป: งานคุณตอนนี้ลดลงเหลือแค่สาย Programming & AI ล้วนๆ คือ "ทำ DL ให้เสร็จ -> เอา ARDL ของเพื่อนมายัดใส่ RL -> เทรน AI -> วัดผล" ครับ

In [ ]:
!jupyter nbconvert --to html "Semester2_V1_AQT_ARDLTrading_Macro&Technical.ipynb" --output "AQT_Semester1_Final.html"
